# 1. import các thưu viện cho việc tiền xử lý

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import string
import re


from underthesea import word_tokenize, pos_tag, ner

c:\Users\VICTUS\anaconda3\envs\aio\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.1 Danh sách cách từ stopword, viết tắt, từ phủ định

In [2]:
vietnamese_sentiment_negation = [
    "không",           
    "chẳng",           
    "chưa",           
    "đừng",            
    "chả",             
]

In [3]:
vietnamese_teen_slang = {
    # Từ viết tắt cơ bản
    "đc": "được",
    "dc": "được", 
    "k": "không",
    "ko": "không",
    "kg": "không",
    "kh": "không",
    "khg": "không",
    "cx": "cũng",
    "cg": "cũng",
    "nx": "nữa", 
    "nz": "nữa",
    "r": "rồi",
    "z": "rồi",
    "zồi": "rồi",
    "zì": "gì",
    "j": "gì",
    "gj": "gì",
    "mk": "mình",
    "vk": "vợ",
    "vc": "vãi cả",
    "m": "mình",
    "mìk": "mình",
    "t": "tao",
    "tao": "tao", 
    "m": "mày",
    "may": "mày",
    "b": "bạn",
    "tn": "thế này",
    "bn": "bạn",
    "e": "em",
    "a": "anh",
    "c": "chị",
    "vs": "với",
    "w": "với",
    "v": "vậy",
    "chs": "chơi",
    "ch": "chưa",
    "chx": "chưa",
    "ntn": "như thế nào",
    "stn": "sao thế này",
    "tbn": "thế bạn",
    "h": "giờ",
    "bh": "bây giờ",
    "lh": "liên hệ",
    "ns": "nói",
    "nc": "nước",
    "nch": "nói chuyện",
    "ncl": "nói chung là",
    "nc": "nói chuyện",
    "xl": "xin lỗi",
    "sr": "sorry",
    "tks": "thanks",
    "cảm ơn": "cảm ơn",
    "ty": "thank you",
    "thx": "thanks",
    "aui": "anh yêu",
    "ô": "ông",
    "csgt": "cảnh sát giao thông",
    "hv": "học viện",
    "cntt": "công nghệ thông tin",
    "mn": "mọi người",
    "mng": "mọi người",
    
    # Từ teen phổ biến
    "tml": "thằng mặt lồn",
    "đm": "địt mẹ",
    "dm": "đijt mẹ",
    "wtf": "cái địt",
    "omg": "ôi trời ơi",
    
    
    # Từ toxic/chửi thề (cho bài toán phân loại toxic)
    "vcl": "vãi cả lồn",
    "vl": "vãi lồn", 
    "cc": "cặc",
    "cl": "cái lồn",
    "clm": "cái lồn mẹ",
    "cmm": "con mẹ mày",
    "cmn": "con mẹ nó",
    "cmnr": "con mẹ nó rồi",
    "dcm": "địt con mẹ",
    "dcmm": "địt con mẹ mày",
    "dcmn": "địt con mẹ nó", 
    "đcm": "địt con mẹ",
    "đcmm": "địt con mẹ mày",
    "lz": "lồn",
    "l": "lồn",
    "lol": "lồn",
    "nml": "não mẹ lồn",
    "nmn": "não mẹ nó",
    "nmm": "não mẹ mày",
    "vkl": "vãi ka lồn",
    "voz": "vãi lồn", 
    "shit": "địt",
    "fuck": "địt",
    "fk": "địt",
    "damn": "chết tiệt",
    "bitch": "con đĩ",
    "asshole": "thằng lồn",
    "stupid": "ngu",
    "idiot": "ngu",
    "retard": "khùng",
    "gay": "đồng tính",
    "die": "chết đi",
    "kill": "giết",
    "hate": "ghét",
    "ugly": "xấu",
    "fat": "béo",
    "loser": "thất bại",
    "noob": "thua kém",
    "trash": "rác",
    "cancer": "ung thư",
    "toxic": "độc hại",
    
    # Từ toxic tiếng Việt
    "ngu": "ngu",
    "khùng": "khùng",
    "điên": "điên", 
    "đần": "đần",
    "tởm": "tởm",
    "ghê": "ghê",
    "dở": "dở",
    "tệ": "tệ",
    "tồi": "tồi",
    "rác": "rác",
    "shit": "cứt",
    "fck": "địt",
    "fuk": "địt",
    "dam": "chết tiệt",
    "hell": "địa ngục",
    
    # Từ viết tắt toxic
    "fku": "địt mẹ mày",
    "fucku": "địt mẹ mày",
    "fuck you": "địt mẹ mày",
    "suck my dick": "bú cặc tao",

    
    # Từ xúc phạm nhẹ
    "suck": "tệ",
    "đúng z": "đúng rồi",
    "oke": "okay",
    "okie": "okay",
    "ok": "okay",
    "okela": "okay",
    "roài": "rồi",
    "zậy": "vậy",
    "zị": "gì",
    "zàng": "gì vậng",
    "zề": "gì thế",
    "zay": "vậy",
    "uk": "ừ",
    "uh": "ừ",
    "uhm": "ừm",
    "hm": "hmm",
    "hihi": "cười",
    "hehe": "cười",
    "huhu": "khóc",
    "hoho": "cười",
    
    # Từ mạng xã hội
    "flex": "khoe khoang",
    "chill": "thư giãn",
    "slay": "xuất sắc",
    "ship": "ghép đôi",
    "crush": "người thích",
    "babe": "người yêu",
    "bff": "bạn thân mãi mãi",
    "simp": "dại gái",
    "toxic": "độc hại",
    "red flag": "tín hiệu nguy hiểm",
    "green flag": "tín hiệu tích cực",
    
    # Từ game/internet
    "gg": "good game",
    "ez": "dễ ợt",
    "noob": "nghiệp dư",
    "pro": "chuyên nghiệp", 
    "lag": "giật",
    "bug": "lỗi",
    "hack": "gian lận",
    "tryhard": "cố gắng hết mức",

    
    # Từ mới xuất hiện
    "fire": "xuất sắc",
    
    # Từ viết tắt khác
    "nt": "như thế",
    "bt": "bình thường",
    "vt": "vậy thì",
    "đt": "điện thoại",
    "fb": "facebook",
    "ig": "instagram", 
    "tt": "tiktok",
    "yt": "youtube",
    "zl": "zalo",
    "mess": "messenger",
    "vib": "vietinbank",
    "tech": "techcombank",
    "vcb": "vietcombank",
    "mb": "mbbank",
    "tp": "thành phố",
    "q": "quận",
    "p": "phường",
    "st": "street",
    "rd": "road",
    "snvv": "sinh nhật vui vẻ",
    
    # Từ cảm xúc
    "uwu": "dễ thương",
    "owo": "ngạc nhiên",

    
    # Từ thời gian
    "gl": "good luck",

}

In [4]:
vietnamese_stopwords = [
    # Đại từ nhân xưng
    "tôi", "bạn", "anh", "chị", "em", "ông", "bà", "cô", "chú", "cậu",
    "mình", "ta", "chúng ta", "chúng tôi", "các bạn", "họ", "nó",
    "tự", "ai", "gì", "nào", "sao", "thế nào", "như thế nào", "mày",
    "tao",

    
    # Từ chỉ định
    "này", "đó", "kia", "đây", "đấy", "kìa", "nọ",
    
    # Giới từ
    "của", "với", "cho", "tại", "trong", "ngoài", "trên", "dưới",
    "về", "đến", "từ", "theo", "sau", "trước", "giữa", "bên",
    "cùng", "nhờ", "bởi", "do", "vì", "để", "mà", "như",
    
    # Liên từ
    "và", "hay", "hoặc", "nhưng", "song", "mặc dù", "tuy", "dù",
    "nếu", "giả sử", "khi", "lúc", "hồi", "thuở", "tức là",
    "nghĩa là", "tức", "tuy nhiên", "tuy thế", "thế nhưng",
    
    # Từ hỏi
    "làm sao", "ra sao", "thế nào", "như nào", "bao giờ", "khi nào",
    "ở đâu", "tại sao", "vì sao", "tại làm sao", "bằng cách nào",
    
    # Trợ động từ
    "là", "thì", "mà", "rồi", "đã", "sẽ", "có thể", "phải",
    "nên", "được", "bị", "vẫn", "vẫn còn", "còn", "đang",
    "sắp", "vừa", "mới", "cũng", "cùng", "chỉ", "chỉ có",
    
    # Từ định lượng
    "nhiều", "ít", "vài", "mấy", "bao nhiêu", "tất cả", "toàn bộ",
    "cả", "mọi", "từng", "mỗi", "một số", "phần lớn", "đa số",
    "thiểu số", "một phần", "hầu hết", "gần như", "hơn", "dưới",
    
    # Từ thời gian
    "hôm nay", "ngày mai", "hôm qua", "tuần này", "tháng này",
    "năm này", "hiện tại", "bây giờ", "lúc này", "khi đó",
    "lúc đó", "thuở đó", "ngày xưa", "xưa kia", "sau này",
    
    # Từ nối, từ tình thái
    "thật", "thực", "quả", "đúng", "rõ", "rất", "khá", "hơi",
    "tương đối", "khôn", "lắm", "quá", "cực", "cực kỳ",
    "vô cùng", "hết sức", "đặc biệt", "nhất là", "đúng là",
    
    # Các từ khác thường gặp
    "về phần", "về mặt", "theo như", "như vậy", "thế thôi",
    "vậy thôi", "chỉ thế", "có lẽ", "chắc", "chắc chắn",
    "dĩ nhiên", "tất nhiên", "đương nhiên", "hiển nhiên",
    "rõ ràng", "hẳn", "nhất định", "chắc hẳn",
    
    # Từ kết thúc câu
    "thôi", "vậy", "đó", "mà thôi", "à", "ạ", "ư", "hử",
    "nhé", "nha", "ha", "hehe", "hihi",
    
    # Từ phổ biến khác
    "việc", "chuyện", "công việc", "vấn đề", "vần đề", "điều",
    "cái", "con", "chiếc", "cuốn", "quyển", "bức", "bài",
    "người", "thằng", "con người", "ai đó", "có người",
    "gia đình", "nhà", "quê", "quê hương", "đất nước",
    
    # Từ cảm thán không phủ định
    "ôi", "ối", "ơi", "ừ", "ờ", "à", "ạ", "ư", "hử",
    "wow", "wao", "chà", "trời", "trời ơi", "chao ôi",
    
    # Từ chỉ mức độ
    "hơn hẳn", "kém", "bằng", "ngang", "thua", "thắng",
    "cao", "thấp", "lớn", "nhỏ", "to", "bé", "dài", "ngắn",
    
    # Từ chỉ phương hướng
    "lên", "xuống", "vào", "ra", "qua", "lại", "lui",
    "tới", "đi", "về", "lại đây", "ra đó", "vào trong",
    
    # Số đếm cơ bản
    "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín", "mười",
    "trăm", "nghìn", "triệu", "tỷ", "đầu tiên", "thứ hai", "thứ ba",
    
    # Từ khẳng định (không phủ định)
    "ừ", "ờ", "đúng", "phải", "được", "ok", "oke", "okie",
    
    # Từ chỉ thời điểm
    "sáng", "chiều", "tối", "đêm", "trưa", "sớm", "muộn",
    "Thứ Hai", "Thứ Ba", "Thứ Tư", "Thứ Năm", "Thứ Sáu", "Thứ Bảy", "Chủ Nhật"
]

# 2. Tiền xử lý data (chung)

In [5]:
def lowercase(text):
    # Đưa về viết thường
    return text.lower()

def punctuation_removal(text):
    # Loại bỏ các dấu câu
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator) 

def remove_number(text):
    return re.sub(r'\d+', '', text)

def remove_emoji(text):
    # Loại bỏ emoji như 😂😂
    clean_text = re.sub(r'[^\w\s,.!?-]', '', text)
    # Loại bỏ emoji như :))
    clean_text = re.sub(r'[:;=8xX][-~]?[)(DPpOo/\\]+', '', clean_text)
    # Loại bỏ emoji đặc biệt như ʚöɞ
    clean_text = re.sub(r'[^a-zA-Z0-9\s,.!?àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđĐ-]', '', clean_text)
    
    return clean_text

def normalize_abbreviations(text):
    # Chuẩn hóa viết tắt
    words = word_tokenize(text)
    nor_text = []
    for word in words:
        if word in vietnamese_teen_slang.keys():
            nor_text.append(vietnamese_teen_slang[word])
        else:
            nor_text.append(word)
    return ' '.join(nor_text)

def remove_repeated_chars(text):
    # Thay thế ký tự lặp như "vậyyyy" thành "vậy", nhược điểm là với từ như "pass" sẽ thành "pas"
    return re.sub(r'(.)\1+', r'\1', text)

def remove_stopword(text):
    # Loại bỏ tử dừng
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in vietnamese_stopwords])

def pipeline_preprocess(text):
    text = lowercase(text)
    text = punctuation_removal(text)
    text = remove_number(text)
    text = remove_emoji(text)
    text = normalize_abbreviations(text)
    text = remove_repeated_chars(text)
    text = remove_stopword(text)
    return text


In [6]:
def preprocess_data(data):
    """Hàm tiền xử lý cho dữ liệu cho trước
    Args:
      - data: Input data đã cho trước
    Returns:
      - Data đã qua tiền xử lý
    """
    

    data = data.copy()
    first_column = data.columns[0]


    """
    - Xóa bỏ các sample có số từ word lớn hơn 50 (tức là bài đăng hoặc quá dài)
    Làm tăng thêm chiều, giảm tốc độ học của mô hình
    - Xỏa bỏ các sample có số từ bé hơn 3 vì có khả năng là tag tên nhau"""

    data = data[[len(i.split()) < 50 for i in data[first_column]]]
    data = data[[len(i.split()) > 2 for i in data[first_column]]]

    data[first_column] = data[first_column].apply(pipeline_preprocess)

    data = data.reset_index(drop=True)

    return data

## 2.1. Tiền xử lý cho tập dữ liệu đã có sẵn label

In [7]:
data = pd.read_csv('train_spam.csv')
display(data)

,comment,label
0,Shop ơi có mẫu này size 38 không ạ? Dr cho mìn...,0
1,"Nhận pass lại giáo trình Kinh tế vĩ mô, sách c...",1
2,Clb guitar tuần này có buổi sinh hoạt không ad...,0
3,BẠN CÓ MUỐN KIẾM 300K MỖI NGÀY? Chỉ cần điện t...,1
4,Vcl quả drama mới. Con bé A cặp với thầy B á? ...,0
...,...,...
1655,"Thằng main trong bộ phim này ngu lồn vcl, xem ...",0
1656,Có ai biết cách stream phim từ laptop lên TV k...,0
1657,Bán tài khoản VIP cho các trang học tiếng Anh ...,1
1658,Có ai biết quán cafe phim nào ở Hà Nội không?,0


In [8]:
data_pre = preprocess_data(data)
data_pre

,comment,label
0,shop có mẫu size không dr xin giá,0
1,nhận pas giáo trình kinh tế vĩ mô sách chưa gh...,1
2,clb guitar tuần có buổi sinh hoạt không ad,0
3,có muốn kiếm không ngày cần điện thoại thời gi...,1
4,vãi lồn drama con bé a cặp thầy á hóng link,0
...,...,...
1655,main bộ phim ngu lồn vãi lồn xem,0
1656,có biết cách stream phim laptop tv không,0
1657,bán tài khoản vip các trang học tiếng phim ejo...,1
1658,có biết quán cafe phim ở hà nội không,0


## 2.2 Tiền xử lý cho dữ liệu craw không nhãn các comment của sinh viên ptit

In [9]:
comment_data = pd.read_csv("comment_data.csv")
comment_data

,text,length
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...,2865
1,Chúc mừng zai mẹ nhé,20
2,Xin chúc mừng các bạn trẻ,25
3,1 bữa lẩu ở iec nhé\nNguyễn Trang,32
4,Nguyễn Trang\nchờ quà hcm lâu quá,32
...,...,...
4744,Đâu:)?,6
4745,Hoang Anh\n?,11
4746,Chi Linh,8
4747,Quynh Anh\nok,12


In [10]:
comment_data['num_word'] = [len(i.split()) for i in comment_data['text']]
comment_data

,text,length,num_word
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...,2865,574
1,Chúc mừng zai mẹ nhé,20,5
2,Xin chúc mừng các bạn trẻ,25,6
3,1 bữa lẩu ở iec nhé\nNguyễn Trang,32,8
4,Nguyễn Trang\nchờ quà hcm lâu quá,32,7
...,...,...,...
4744,Đâu:)?,6,1
4745,Hoang Anh\n?,11,3
4746,Chi Linh,8,2
4747,Quynh Anh\nok,12,3


In [12]:
data_pre_n = preprocess_data(comment_data)
display(data_pre_n)

,text,length,num_word
0,chúc mừng zai mẹ,20,5
1,xin chúc mừng các trẻ,25,6
2,bữa lẩu ở iec nguyễn trang,32,8
3,nguyễn trang chờ quà hcm lâu,32,7
4,chúc mừng sinh viên học giỏi,28,6
...,...,...,...
4125,fa nhể,30,9
4126,lua ga,10,3
4127,hoang,11,3
4128,quynh okay,12,3


In [13]:
comment_data['text'].loc[4725]

'Đinh Gia Bảo\nkèo thơm bro'

# 3. Huấn luyện mô hình Navie Bayes nhận diện toxic/non bằng thư viện có sẵn sklearn

## 3.1. import các thưu viện liên quan

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.pipeline import Pipeline

import torch
from transformers import AutoTokenizer, AutoModel

## 3.2. Tách riêng nhãn ra khỏi dữ liệu có nhãn

In [16]:
X = data_pre['comment']
y = data_pre['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


X_n = data_pre_n['text']

## 3.3. Huấn luyện mô hình Navie Bayes kèm với pipeline BoW

In [24]:
def train_model_NB_with_BoW_pipeline(X_train, y_train, x_test, y_test, X_n, threshold=0.8):
    pipeline = Pipeline([
        ("BoW", CountVectorizer()),
        ("NB", MultinomialNB())
    ])
    pipeline.fit(X_train, y_train)

    # Kết quả đánh giá ma trận trước khi self training
    y_pred_before = pipeline.predict(X_test)


    # Dự đoán cho dữ liệu không nhãn
    y_n = pd.Series(pipeline.predict(X_n))
    # Dự đoán xác suất cho dữ liệu không nhãn, sẽ trả về dạng list lồng với mỗi phần tử là xác suất nhãn 0/1 xuất hiện
    y_n_proba = pipeline.predict_proba(X_n)
    # Chuyển về pd.SeriesSeries, lấy ra xác xuất gán nhãn hiện tại của mô hình cho từng sample
    y_n_proba = pd.Series(np.max(y_n_proba, axis=1))

    # Tạo ra mask chứa các sample được dự đoán với xác suất trên ngưỡng threshold
    mask = y_n_proba > threshold

    # # Lọc ra các sample ở dataset không nhãn khi được dự đoán có xác suất trên ngưỡng threshold
    X_n = X_n[mask]
    y_n = y_n[mask]

    # Gộp chung ds 
    X_train_new = pd.concat([X_train, X_n], axis=0)

    # Gộp chung label
    y_train_new = pd.concat([y_train, y_n], axis=0)

    # Huấn luyện lại mô hình với ds đã gộp
    pipeline.fit(X_train_new, y_train_new)

    # Dự đoán cho tập test và đưa ra kết quả đánh giá ma trận
    y_pred_after = pipeline.predict(X_test)

    test = 0
    print('-' * 30)

    print(f"Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_before))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_before, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_before, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_before, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_before))


    print(f"\nMa trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_after))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_after, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_after, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_after, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_after))


In [25]:
train_model_NB_with_BoW_pipeline(X_train, y_train, X_test, y_test, X_n)

------------------------------
Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9668674698795181
Độ chính xác theo nhãn (Precision - macro): 0.9565772669220944
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9706463158760851
Điểm F1 (F1-score - macro): 0.9630270817514553
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       223
           1       0.92      0.98      0.95       109

    accuracy                           0.97       332
   macro avg       0.96      0.97      0.96       332
weighted avg       0.97      0.97      0.97       332


Ma trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9608433734939759
Độ chính xác theo nhãn (Precision - macro): 0.9499521072796935
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9638170074464146
Điểm F1 (F1-score - macro): 0.9563047329789927
Báo cáo chi tiết các chỉ số:

## 3.4. Huấn luyện mô hình Navie Bayes kèm với pipeline TF-IDF

In [41]:
def train_model_NB_with_tfidf_pipeline(X_train, y_train, x_test, y_test, X_n, threshold=0.8):
    pipeline = Pipeline([
        ("TF_IDF", TfidfVectorizer()),
        ("NB", MultinomialNB())
    ])
    pipeline.fit(X_train, y_train)

    # Kết quả đánh giá ma trận trước khi self training
    y_pred_before = pipeline.predict(X_test)


    # Dự đoán cho dữ liệu không nhãn
    y_n = pd.Series(pipeline.predict(X_n))
    # Dự đoán xác suất cho dữ liệu không nhãn, sẽ trả về dạng list lồng với mỗi phần tử là xác suất nhãn 0/1 xuất hiện
    y_n_proba = pipeline.predict_proba(X_n)
    # Chuyển về pd.SeriesSeries, lấy ra xác xuất gán nhãn hiện tại của mô hình cho từng sample
    y_n_proba = pd.Series(np.max(y_n_proba, axis=1))

    # Tạo ra mask chứa các sample được dự đoán với xác suất trên ngưỡng threshold
    mask = y_n_proba > threshold

    # # Lọc ra các sample ở dataset không nhãn khi được dự đoán có xác suất trên ngưỡng threshold
    X_n = X_n[mask]
    y_n = y_n[mask]

    # Gộp chung ds 
    X_train_new = pd.concat([X_train, X_n], axis=0)

    # Gộp chung label
    y_train_new = pd.concat([y_train, y_n], axis=0)

    # Huấn luyện lại mô hình với ds đã gộp
    pipeline.fit(X_train_new, y_train_new)

    # Dự đoán cho tập test và đưa ra kết quả đánh giá ma trận
    y_pred_after = pipeline.predict(X_test)

    test = 0
    print('-' * 30)

    print(f"Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_before))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_before, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_before, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_before, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_before))


    print(f"\nMa trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_after))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_after, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_after, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_after, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_after))

    return pipeline

In [47]:
trained_pipeline = train_model_NB_with_tfidf_pipeline(X_train, y_train, X_test, y_test, X_n)

------------------------------
Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9427710843373494
Độ chính xác theo nhãn (Precision - macro): 0.9448275862068966
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9245690541819229
Điểm F1 (F1-score - macro): 0.9336663336663337
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       223
           1       0.95      0.87      0.91       109

    accuracy                           0.94       332
   macro avg       0.94      0.92      0.93       332
weighted avg       0.94      0.94      0.94       332


Ma trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9006024096385542
Độ chính xác theo nhãn (Precision - macro): 0.9305976176054916
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.8509688567079442
Điểm F1 (F1-score - macro): 0.8771735109137995
Báo cáo chi tiết các chỉ số:

In [53]:
# Bây giờ bạn có thể sử dụng trained_pipeline để dự đoán trên các mẫu mới
new_sample_1 = "dịch vụ rất tệ"
new_sample_2 = "sản phẩm đáng tiền"
new_sample_3 = "khóa học làm giàu giảm giá 50%"

# Dự đoán nhãn
prediction_1 = trained_pipeline.predict([new_sample_1])
prediction_2 = trained_pipeline.predict([new_sample_2])
prediction_3 = trained_pipeline.predict([new_sample_3])

print(f"\nDự đoán cho '{new_sample_1}': {prediction_1[0]}")
print(f"Dự đoán cho '{new_sample_2}': {prediction_2[0]}")
print(f"Dự đoán cho '{new_sample_3}': {prediction_3[0]}")


Dự đoán cho 'dịch vụ rất tệ': 0
Dự đoán cho 'sản phẩm đáng tiền': 0
Dự đoán cho 'khóa học làm giàu giảm giá 50%': 0


Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9427710843373494
Độ chính xác theo nhãn (Precision - macro): 0.9448275862068966
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9245690541819229
Điểm F1 (F1-score - macro): 0.9336663336663337
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       223
           1       0.95      0.87      0.91       109

    accuracy                           0.94       332
   macro avg       0.94      0.92      0.93       332
weighted avg       0.94      0.94      0.94       332


Ma trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.8704819277108434
Độ chính xác theo nhãn (Precision - macro): 0.9131016042780749
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.8050972970749166
Điểm F1 (F1-score - macro): 0.8343832295038225
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       223
           1       0.99      0.61      0.76       109

    accuracy                           0.87       332
   macro avg       0.91      0.81      0.83       332
weighted avg       0.89      0.87      0.86       332

## 3.5. Huấn luyện mô hình Navie Bayes kèm với pipeline embedding với Phobert

In [ ]:
def train_model_with_embedding_pipeline(X_train, y_train, X_test, y_test, X_n):
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
    model = AutoModel.from_pretrained("vinai/phobert-base-v2")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    X_n_tokens = tokenizer(X_n.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        X_train_embeddings = model(**X_train_tokens).last_hidden_state[:, 0, :]
        X_test_embeddings = model(**X_test_tokens).last_hidden_state[:, 0, :]
        X_n_embeddings = model(**X_n_tokens).last_hidden_state[:, 0, :]
        
        
    sentence_embedding = X_train_embeddings

In [ ]:
model = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# 4. Huấn luyện mô hình SVM nhận diện toxic/non bằng sklearn

## 4.1. Huấn luyện mô hình SVC với chiến lược BoW

In [33]:
def train_model_SVC_with_BoW_pipeline(X_train, y_train, x_test, y_test, X_n, threshold=0.8):
    pipeline = Pipeline([
        ("BoW", CountVectorizer()),
        ("SVC", SVC(probability=True))
    ])
    pipeline.fit(X_train, y_train)
    # Kết quả đánh giá ma trận trước khi self training
    y_pred_before = pipeline.predict(X_test)


    # Dự đoán cho dữ liệu không nhãn
    y_n = pd.Series(pipeline.predict(X_n))
    # Dự đoán xác suất cho dữ liệu không nhãn, sẽ trả về dạng list lồng với mỗi phần tử là xác suất nhãn 0/1 xuất hiện
    y_n_proba = pipeline.predict_proba(X_n)
    # Chuyển về pd.SeriesSeries, lấy ra xác xuất gán nhãn hiện tại của mô hình cho từng sample
    y_n_proba = pd.Series(np.max(y_n_proba, axis=1))

    # Tạo ra mask chứa các sample được dự đoán với xác suất trên ngưỡng threshold
    mask = y_n_proba > threshold

    # # Lọc ra các sample ở dataset không nhãn khi được dự đoán có xác suất trên ngưỡng threshold
    X_n = X_n[mask]
    y_n = y_n[mask]

    # Gộp chung ds 
    X_train_new = pd.concat([X_train, X_n], axis=0)

    # Gộp chung label
    y_train_new = pd.concat([y_train, y_n], axis=0)

    # Huấn luyện lại mô hình với ds đã gộp
    pipeline.fit(X_train_new, y_train_new)

    # Dự đoán cho tập test và đưa ra kết quả đánh giá ma trận
    y_pred_after = pipeline.predict(X_test)

    test = 0
    print('-' * 30)

    print(f"Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_before))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_before, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_before, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_before, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_before))


    print(f"\nMa trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_after))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_after, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_after, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_after, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_after))

In [34]:
train_model_SVC_with_BoW_pipeline(X_train, y_train, X_test, y_test, X_n)

------------------------------
Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9789156626506024
Độ chính xác theo nhãn (Precision - macro): 0.9713906325446644
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9819599292384911
Điểm F1 (F1-score - macro): 0.976368424798918
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       223
           1       0.95      0.99      0.97       109

    accuracy                           0.98       332
   macro avg       0.97      0.98      0.98       332
weighted avg       0.98      0.98      0.98       332


Ma trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9728915662650602
Độ chính xác theo nhãn (Precision - macro): 0.9663961038961039
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9727856173118855
Điểm F1 (F1-score - macro): 0.9694799955057557
Báo cáo chi tiết các chỉ số:


## 4.2. Train model SVC với chiến lược TF-IDF

In [39]:
def train_model_SVC_with_tfidf_pipeline(X_train, y_train, X_test, y_test, X_n, threshold=0.8):
    pipeline = Pipeline([
        ("TF_IDF", TfidfVectorizer()),
        ("SVC", SVC(probability=True))
    ])
    pipeline.fit(X_train, y_train)

        # Kết quả đánh giá ma trận trước khi self training
    y_pred_before = pipeline.predict(X_test)


    # Dự đoán cho dữ liệu không nhãn
    y_n = pd.Series(pipeline.predict(X_n))
    # Dự đoán xác suất cho dữ liệu không nhãn, sẽ trả về dạng list lồng với mỗi phần tử là xác suất nhãn 0/1 xuất hiện
    y_n_proba = pipeline.predict_proba(X_n)
    # Chuyển về pd.SeriesSeries, lấy ra xác xuất gán nhãn hiện tại của mô hình cho từng sample
    y_n_proba = pd.Series(np.max(y_n_proba, axis=1))

    # Tạo ra mask chứa các sample được dự đoán với xác suất trên ngưỡng threshold
    mask = y_n_proba > threshold

    # # Lọc ra các sample ở dataset không nhãn khi được dự đoán có xác suất trên ngưỡng threshold
    X_n = X_n[mask]
    y_n = y_n[mask]

    # Gộp chung ds 
    X_train_new = pd.concat([X_train, X_n], axis=0)

    # Gộp chung label
    y_train_new = pd.concat([y_train, y_n], axis=0)

    # Huấn luyện lại mô hình với ds đã gộp
    pipeline.fit(X_train_new, y_train_new)

    # Dự đoán cho tập test và đưa ra kết quả đánh giá ma trận
    y_pred_after = pipeline.predict(X_test)

    test = 0
    print('-' * 30)

    print(f"Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_before))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_before, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_before, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_before, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_before))


    print(f"\nMa trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_after))
    print("Độ chính xác theo nhãn (Precision - macro):", precision_score(y_test, y_pred_after, average='macro'))
    print("Tỉ lệ thu hồi theo nhãn (Recall - macro):", recall_score(y_test, y_pred_after, average='macro'))
    print("Điểm F1 (F1-score - macro):", f1_score(y_test, y_pred_after, average='macro'))
    print("Báo cáo chi tiết các chỉ số:")
    print(classification_report(y_test, y_pred_after))

In [37]:
def train_model_SVC_with_tfidf_pipeline(X_train, y_train, X_test, y_test, X_n):
    pipeline = Pipeline([
        ("TF_IDF", TfidfVectorizer()),
        ("SVC", SVC())
    ])
    pipeline.fit(X_train, y_train)

    # Dự đoán cho dữ liệu không nhãn
    y_n = pd.Series(pipeline.predict(X_n))

    # Gộp chung ds 
    X_train_new = pd.concat([X_train, X_n], axis=0)

    # Gộp chung label
    y_train_new = pd.concat([y_train, y_n], axis=0)

    # Huấn luyện lại mô hình với ds đã gộp
    pipeline.fit(X_train_new, y_train_new)

    # Dự đoán cho tập test và đưa ra kết quả đánh giá ma trận
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {acc}")

    print("Classification Report")
    print(classification_report(y_test, y_pred))
    

In [40]:
train_model_SVC_with_tfidf_pipeline(X_train, y_train, X_test, y_test, X_n)

------------------------------
Ma trận đánh giá trước khi self-training (trước khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9849397590361446
Độ chính xác theo nhãn (Precision - macro): 0.9840443121693122
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.9817542271773563
Điểm F1 (F1-score - macro): 0.9828864215095001
Báo cáo chi tiết các chỉ số:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       223
           1       0.98      0.97      0.98       109

    accuracy                           0.98       332
   macro avg       0.98      0.98      0.98       332
weighted avg       0.98      0.98      0.98       332


Ma trận đánh giá sau khi self-training (sau khi thêm dữ liệu tự gán nhãn):
Độ chính xác (Accuracy): 0.9759036144578314
Độ chính xác theo nhãn (Precision - macro): 0.9798617882732014
Tỉ lệ thu hồi theo nhãn (Recall - macro): 0.965647755790513
Điểm F1 (F1-score - macro): 0.9722825179495742
Báo cáo chi tiết các chỉ số:


## 4.3. Train model SVC với chiến lược embedding câu dugf PhoBert

In [ ]:
def train_model_SVC_with_embedding_pipeline(X_train, y_train, X_test, y_test, X_n):
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
    model = AutoModel.from_pretrained("vinai/phobert-base-v2")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    X_n_tokens = tokenizer(X_n.tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        X_train_embeddings = model(**X_train_tokens).last_hidden_state[:, 0, :]
        X_test_embeddings = model(**X_test_tokens).last_hidden_state[:, 0, :]
        X_n_embeddings = model(**X_n_tokens).last_hidden_state[:, 0, :]
        
        
    sentence_embedding = X_train_embeddings

# -1.Khó khăn trong quá trình xử lý, huấn luyện nhận diện Toxic/Non

**Trong chuẩn hóa viết tắt**
- Ngữ nghĩa phân biệt từ trong ngữ cảnh khác nhau (cân nhắc dùng pos, deoendency parsing để đoán vì từ phải đi với nghĩa ở trước)
    - vãi cả - việc - vc

**Phân biệt câu reverse toxic**
- Khó khăn trong việc phân biệt câu toxic/non
    - Mày là đồ ngu - Toxic
    - Mày không phải là ngu - Non Toxic
    - Anh không ngu đâu - Non Toxic
    - Cái này rất tệ - Toxic
    - Cái này tệ - Ít toxic hơn, có khi là không

**Câu chê bai, vẫn coi là toxic**
- Mày không tài đâu con ạ

**Câu có cả 2 loại**
- Mày là đồ ngu, nhưng nó thì không
- Mày không ngu, nhưng nó ngược lại


**Câu ẩn ý:**
- Mày thì khôn
- Mày khôn, tao thì ngu
- Mày thì không

**Vấn đề chưa loại bỏ được từ tag tên như "Hải Yến", chưa phân biệt được lỗi sai như "đungs"**
- Nếu đưa vào mô hình khi tfidf sẽ làm tăng số chiều, không đem lại giá trị lớn và làm giảm tốc độ huấn luyện, tăng thời gian xử lý

# test dataset spam/ham

In [ ]:
import pandas as pd


data = pd.read_csv("spam/vispamdetectionv2/train.csv")

In [ ]:
data

In [ ]:
test = pd.DataFrame()
test['comment'] = data['comment']
test['description'] = data['description']
test['label'] = data['label']
test['spam_label'] = data['spam_label']

In [ ]:
test

In [ ]:
test[(test['label'] == 1) & (test['spam_label'] == 3)]

In [ ]:
test = pd.DataFrame([data['comment'], data['description'], data['label'], data['spam_label']])

In [ ]:
data.describe()

In [ ]:
ds_dict = [
    {'comment': 'Shop ơi có mẫu này size 38 không ạ? Dr cho mình xin giá với nhé.', 'label': 0},
    {'comment': 'Nhận pass lại giáo trình Kinh tế vĩ mô, sách còn mới 99% chưa ghi chép gì. Giá 50k, ai cần ib.', 'label': 1},
    {'comment': 'Clb guitar tuần này có buổi sinh hoạt không ad ơi?', 'label': 0},
    {'comment': 'BẠN CÓ MUỐN KIẾM 300K MỖI NGÀY? Chỉ cần điện thoại và thời gian rảnh. Công việc đơn giản, không vốn, không lừa đảo. Liên hệ Zalo: 09xxyyzz', 'label': 1},
    {'comment': 'Vcl quả drama mới. Con bé A cặp với thầy B á? Hóng link.', 'label': 0},
    {'comment': 'Cần tìm người học chung Ielts mục tiêu 7.0. Mình ở khu vực Cầu Giấy.', 'label': 0},
    {'comment': 'Thanh lý toàn bộ đồ dùng sinh viên: bàn học, giá sách, nồi cơm điện. Đồng giá 100k. Mình sắp ra trường nên pass rẻ.', 'label': 1},
    {'comment': 'Mấy đứa ơi có ai biết pass wifi thư viện là gì không?', 'label': 0},
    {'comment': 'Bên mình chuyên nhận làm assignment, thesis, đảm bảo qua môn, điểm cao. Uy tín, bảo mật info.', 'label': 1},
    {'comment': 'Nói thật học ngành này ra trường không biết làm gì luôn á.', 'label': 0},
    {'comment': 'Sao quả ảnh này nhìn như pts thế nhỉ?', 'label': 0},
    {'comment': '🔥 XẢ KHO CUỐI NĂM - LAPTOP GAMING GIẢM SỐC 50% 🔥 FREESHIP TOÀN QUỐC. Check ngay tại link bio!', 'label': 1},
    {'comment': 'Thầy dạy môn này chán vcl, buồn ngủ chết đi được.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào ăn buffet lẩu nướng ngon bổ rẻ cho sv không?', 'label': 0},
    {'comment': 'Bạn nào rớt môn GDTC thì về đội của mình :))', 'label': 0},
    {'comment': 'Tuyển 10 bạn nữ làm PG cho sự kiện, ngoại hình ưa nhìn, cao >1m65. Lương 700k/ngày.', 'label': 1},
    {'comment': 'Đm quả mạng ktx như cc, load mãi ko xong.', 'label': 0},
    {'comment': 'Ai có info bạn nam mặc áo hoodie đen ngồi bàn đầu lớp Triết sáng nay không?', 'label': 0},
    {'comment': 'Chỉ 1 liệu trình, da trắng bật tông, mờ thâm nám. Cam kết hoàn tiền nếu không hiệu quả. Ib để được tư vấn.', 'label': 1},
    {'comment': 'Mọi người cho em hỏi, học phí trường mình kỳ này tăng hay sao ấy ạ?', 'label': 0},
    {'comment': 'Đăng ký học lại ở đâu vậy chúng m?', 'label': 0},
    {'comment': 'Bạn nào đang tìm việc làm thêm không? Quán trà chanh gần trường mình đang cần người.', 'label': 0},
    {'comment': 'VAY TIỀN ONLINE - KHÔNG GẶP MẶT - CÓ TIỀN SAU 5 PHÚT. Chỉ cần CMND/CCCD. Lãi suất thấp nhất thị trường.', 'label': 1},
    {'comment': 'Bài này copy trên mạng chứ sáng tạo cc gì.', 'label': 0},
    {'comment': 'Ai có tài khoản ChatGPT Plus không cho mình dùng ké với.', 'label': 0},
    {'comment': 'Nhận order đồ ăn vặt Hàn Quốc, tối nay mình chốt đơn nhé mng.', 'label': 1},
    {'comment': 'Học bổng của trường mình yêu cầu GPA bao nhiêu vậy ạ?', 'label': 0},
    {'comment': 'Thằng báo team đây rồi, game nào cũng afk.', 'label': 0},
    {'comment': 'Cần pass gấp con iphone 11 lock, máy còn ngon, pin 90%. Giá 4tr có fix.', 'label': 1},
    {'comment': 'Mai thi rồi mà giờ chữ nào cũng chưa vào đầu. Toang.', 'label': 0},
    {'comment': 'Ad duyệt bài em với, em bị lừa mất 500k tiền cọc phòng trọ, muốn cảnh báo mng.', 'label': 0},
    {'comment': 'Bạn nào cần tìm người yêu đi chơi noel thì dr mình nhé. Mình FA lâu năm rồi.', 'label': 0},
    {'comment': 'CUNG CẤP TÀI KHOẢN NETFLIX, YOUTUBE PREMIUM GIÁ RẺ. Bảo hành suốt thời gian sử dụng.', 'label':1},
    {'comment': 'Lại phải lên thư viện cày deadline thôi.', 'label': 0},
    {'comment': 'Ngu thì chết chứ bệnh tật gì.', 'label': 0},
    {'comment': 'Tham gia group chia sẻ tài liệu học tập free cho sv. Link ở bio nhé.', 'label': 1},
    {'comment': 'Có ai đi xe bus 32 không cho mình hỏi mấy giờ có chuyến cuối ạ?', 'label': 0},
    {'comment': 'Góc tìm đồ: Em có làm rơi 1 cái thẻ sinh viên ở căng tin, ai nhặt được cho em xin lại với ạ.', 'label': 0},
    {'comment': 'Tuyển mẫu nail cho học viên, free 100%. Nhanh tay đặt lịch kẻo hết slot.', 'label': 1},
    {'comment': 'Ôi idol của tôi, đẹp trai quá.', 'label': 0},
    {'comment': 'Chỗ này gửi xe qua đêm có an toàn không mọi người?', 'label': 0},
    {'comment': 'Bán data 1000 khách hàng tiềm năng, phù hợp cho các bạn sale. Giá rẻ.', 'label': 1},
    {'comment': 'Học xong 4 năm mà cảm thấy mông lung quá.', 'label': 0},
    {'comment': 'Đứa nào có link xem phim này full HD không?', 'label': 0},
    {'comment': 'BÍ KÍP ĐẠT 900+ TOEIC TRONG 3 THÁNG. Tài liệu độc quyền, cam kết đầu ra. Inbox để nhận ngay!', 'label': 1},
    {'comment': 'Môn này có được dùng tài liệu lúc thi không nhỉ?', 'label': 0},
    {'comment': 'Cmt này mà được 100 like tôi tỏ tình crush.', 'label': 0},
    {'comment': 'Bạn nào cần thuê trọ gần trường thì liên hệ mình nhé, chính chủ không qua trung gian.', 'label': 1},
    {'comment': 'Đang đi học mà mẹ gọi hỏi có cần tiền không. Hạnh phúc giản đơn.', 'label': 0},
    {'comment': 'Logic như cc mà cũng nói được.', 'label': 0},
    {'comment': 'Tuyển CTV viết content, chỉ cần online, không yêu cầu kinh nghiệm, thu nhập 2-4tr/tháng.', 'label': 1},
    {'comment': 'Có ai nhận làm hộ chiếu nhanh không ạ?', 'label': 0},
    {'comment': 'Quán này đồ uống cũng bth mà sao đông thế nhỉ?', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Google, Facebook hiệu quả, cam kết ra đơn.', 'label': 1},
    {'comment': 'Lại là một ngày không biết ăn gì.', 'label': 0},
    {'comment': 'Báo thủ, báo cha báo mẹ.', 'label': 0},
    {'comment': 'Cài win, sửa laptop lấy ngay giá sinh viên. Kèm vệ sinh máy miễn phí.', 'label': 1},
    {'comment': 'Thèm trà sữa quá mà hết tiền rồi.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào làm thêm ca tối không ạ?', 'label': 0},
    {'comment': 'GIẢM CÂN CẤP TỐC - AN TOÀN - HIỆU QUẢ. Không cần ăn kiêng, không cần tập luyện. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Chủ thớt viết bài hay quá.', 'label': 0},
    {'comment': 'Sắp ra trường rồi, có ai có kinh nghiệm apply intern cho mình xin với.', 'label': 0},
    {'comment': 'Mình có voucher ahamove giảm 50%, ai cần mình tặng lại nè.', 'label': 0},
    {'comment': 'GOM ORDER GÀ RÁN 120K/COMBO 2 NGƯỜI. FREESHIP BÁN KÍNH 3KM.', 'label': 1},
    {'comment': 'Vc quả review, đọc xong không dám đi làm luôn.', 'label': 0},
    {'comment': 'Nói ít thôi, làm được không?', 'label': 0},
    {'comment': 'Nhận đi chợ hộ, nấu ăn cho các bạn sinh viên lười. Đảm bảo ngon, sạch.', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy xác nhận sinh viên online không?', 'label': 0},
    {'comment': 'Tự nhiên bị con bạn thân unfollow. Buồn vcl.', 'label': 0},
    {'comment': 'Bạn nào muốn cải thiện vòng 1 thì ib em nhé, sản phẩm nhà em an toàn tuyệt đối.', 'label': 1},
    {'comment': 'Điểm giữa kỳ môn này của mng sao rồi?', 'label': 0},
    {'comment': 'Ad cho em hỏi sao bài em đăng mãi chưa được duyệt ạ?', 'label': 0},
    {'comment': 'Pass lại vé xem phim cuối tuần này do bận việc đột xuất. Mua 120k pass 80k.', 'label': 1},
    {'comment': 'Mọi người ơi cứu, em lỡ tay format cái ổ cứng chứa đồ án tốt nghiệp rồi.', 'label': 0},
    {'comment': 'Thằng này nói chuyện xàm l thật sự.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page, trả lời tin nhắn khách hàng. Có thể làm tại nhà.', 'label': 1},
    {'comment': 'Có ai thi lại môn Xác suất thống kê không, lập nhóm ôn chung đi.', 'label': 0},
    {'comment': 'Trường mình có ma không chúng mày?', 'label': 0},
    {'comment': 'Balo chống nước, đựng vừa laptop 15.6 inch, siêu bền, đang sale 30%. Link shopee ở cmt.', 'label': 1},
    {'comment': 'Đẹp trai không bằng chai mặt. Cố lên bạn ơi.', 'label': 0},
    {'comment': 'Cần tìm 1 bạn nam ở ghép, phòng rộng 20m2, có nóng lạnh, điều hòa. Điện nước giá dân.', 'label': 0},
    {'comment': 'Chỉ cần bỏ ra 2 TRIỆU, thu về 20 TRIỆU sau 1 tháng. Cơ hội đầu tư có 1 không 2. Ai quan tâm chấm mình gửi info.', 'label': 1},
    {'comment': 'Đi thực tập có lương không mng?', 'label': 0},
    {'comment': 'Bài viết này thiếu khách quan quá.', 'label': 0},
    {'comment': 'MÁY CHIẾU MINI CHỈ 599K. Biến phòng trọ thành rạp phim. Nhanh tay số lượng có hạn.', 'label': 1},
    {'comment': 'Hè này có ai đi mùa hè xanh không?', 'label': 0},
    {'comment': 'Nhìn cái mặt đã thấy ghét.', 'label': 0},
    {'comment': 'Nhận làm bằng lái xe máy, ô tô các hạng. Bao đậu 100%.', 'label': 1},
    {'comment': 'Có ai ship đồ ăn đêm không ạ? Em đói quá.', 'label': 0},
    {'comment': 'Mọi người ơi, cho em hỏi xin bảng điểm ở phòng đào tạo có nhanh không ạ?', 'label': 0},
    {'comment': 'BẠN MUỐN TĂNG THU NHẬP? Trở thành đối tác của chúng tôi ngay hôm nay. Chiết khấu cao, không cần vốn. Chấm (.) để nhận info.', 'label': 1},
    {'comment': 'Đứa nào lấy con chuột máy tính của tao ở phòng tự học tầng 3 trả lại đây.', 'label': 0},
    {'comment': 'Mình nhận des ảnh, làm video kỷ yếu, video tiktok. Giá cả học sinh, sản phẩm chất lượng. Dr mình nhé.', 'label': 1},
    {'comment': 'Cần tìm người share tiền phòng trọ. Phòng 2tr2/tháng ở gần trường, điện nước giá dân.', 'label': 0},
    {'comment': 'Chán vcl, lại rớt môn nữa rồi. Có ai giống t ko?', 'label': 0},
    {'comment': 'KHÓA HỌC PHOTOSHOP CẤP TỐC ONLINE. Giảm 70% học phí cho 50 bạn đăng ký đầu tiên. Cam kết thành thạo sau 7 ngày.', 'label': 1},
    {'comment': 'Review công ty XYZ có ổn không mng? Em mới nhận được offer intern ở đó.', 'label': 0},
    {'comment': 'Nhận edit CV, cover letter chuẩn chỉnh, chuyên nghiệp. Tăng 90% cơ hội được gọi phỏng vấn.', 'label': 1},
    {'comment': 'Cái thằng cha gửi xe ở nhà B khó tính vãi.', 'label': 0},
    {'comment': 'Flex nhẹ con ảnh thẻ mới chụp.', 'label': 0},
    {'comment': 'Xả hàng tai nghe bluetooth, đồng hồ thông minh đồng giá 99k. Số lượng có hạn, đặt hàng ngay tại link bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào yên tĩnh để học bài không ạ?', 'label': 0},
    {'comment': 'Đứa nào rảnh đi hiến máu với t không?', 'label': 0},
    {'comment': 'Mày nói thế thì chịu rồi, cãi cùn vcl.', 'label': 0},
    {'comment': 'Tuyển 20 nhân viên nam/nữ dán tem sản phẩm tại nhà. Việc nhẹ lương cao, trả tiền theo ngày.', 'label': 1},
    {'comment': 'Thư viện trường mình mở cửa đến mấy giờ thế các bác?', 'label': 0},
    {'comment': 'Ad ơi sao bài của em bị xóa vậy? Có vi phạm gì đâu ạ?', 'label': 0},
    {'comment': 'Bạn nào có mã freeship Shopee không dùng đến cho mình xin với. Cảm ơn nhiều.', 'label': 0},
    {'comment': 'HACK FOLLOW, HACK TYM TIKTOK, FACEBOOK. Giá rẻ nhất hệ mặt trời. Ib để được báo giá.', 'label': 1},
    {'comment': 'Lịch thi cuối kỳ có chưa mng ơi?', 'label': 0},
    {'comment': 'Bài viết sâu sắc quá, cảm ơn tác giả.', 'label': 0},
    {'comment': 'Bán tài liệu, đề cương ôn thi cuối kỳ full các môn đại cương. Đảm bảo trúng tủ 80%.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký tham gia nghiên cứu khoa học không ạ?', 'label': 0},
    {'comment': 'Đm quả tắc đường ở ngã tư sở, đi học muộn cmnr.', 'label': 0},
    {'comment': 'Tham gia ngay nhóm share link khóa học Udemy, Coursera miễn phí. Link ở comment dưới.', 'label': 1},
    {'comment': 'Học lại môn này với thầy nào thì dễ qua hơn nhỉ?', 'label': 0},
    {'comment': 'Thằng này bị ảo à, phát biểu ngu vcl.', 'label': 0},
    {'comment': 'Pass lại đôi boot da mới đi 1 lần do không hợp style. Size 37, giá 200k.', 'label': 1},
    {'comment': 'Mai thuyết trình rồi mà slide còn chưa làm xong. Ai cứu tui với.', 'label': 0},
    {'comment': 'Check uy tín của shop này giúp mình với mng ơi.', 'label': 0},
    {'comment': 'Cần tìm 1 bạn nữ chụp ảnh lookbook cho shop thời trang. Yêu cầu ăn ảnh, tự tin. Lương 400-600k/buổi.', 'label': 1},
    {'comment': 'Quên mật khẩu trang tín chỉ thì làm sao lấy lại được ạ?', 'label': 0},
    {'comment': 'Idol tiktok cc gì, toàn mấy trò xàm xí.', 'label': 0},
    {'comment': 'Nhận cài đặt phần mềm đồ họa (PTS, AI,...) full crack vĩnh viễn. Giá 30k/phần mềm.', 'label': 1},
    {'comment': 'Tự nhiên thèm bún đậu mắm tôm quá chúng mày.', 'label': 0},
    {'comment': 'Cmt dạo thôi chứ đọc chả hiểu cc gì.', 'label': 0},
    {'comment': 'Bạn nào đang bị TRĨ, TÁO BÓN thì về đội của em. Thuốc gia truyền cam kết khỏi dứt điểm.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào sửa điện thoại uy tín mà giá sv không?', 'label': 0},
    {'comment': 'Đội bóng trường mình đá giải này có hay không mng?', 'label': 0},
    {'comment': 'GOM ORDER SỮA CHUA TRÂN CHÂU HẠ LONG. 25K/CỐC. 5H CHIỀU MÌNH SHIP NHÉ.', 'label': 1},
    {'comment': 'Cãi nhau với người nhà mệt thật sự.', 'label': 0},
    {'comment': 'Cần tìm chủ cho bé mèo con bị bỏ rơi. Ai nuôi được dr mình nhé.', 'label': 0},
    {'comment': 'Bạn có muốn da mặt hết mụn, căng bóng như idol Hàn? Hãy ib mình để được tư vấn bộ sản phẩm phù hợp.', 'label': 1},
    {'comment': 'Học xong tín chỉ GDQP có được cộng điểm rèn luyện không ạ?', 'label': 0},
    {'comment': 'Đang yên đang lành bị con ex nó tag vào bài nyc của nó. vc.', 'label': 0},
    {'comment': 'Tuyển cộng tác viên đăng bài cho các group, page. Thu nhập thụ động 1-3tr/tháng.', 'label': 1},
    {'comment': 'Mấy môn tự chọn nên chọn môn nào dễ thở nhỉ mng?', 'label': 0},
    {'comment': 'Nói chuyện kiểu này thì next dùm.', 'label': 0},
    {'comment': 'Chuyên sỉ lẻ quần áo hot trend, không qua trung gian. Mẫu mới cập nhật liên tục. Add zalo để xem mẫu.', 'label': 1},
    {'comment': 'Có ai biết thủ tục xin ra khỏi KTX không?', 'label': 0},
    {'comment': 'Sắp tới sinh nhật ny, nên tặng quà gì được nhỉ?', 'label': 0},
    {'comment': 'Cung cấp acc game có sẵn rank, tướng, skin. Giá rẻ hơn nạp thẻ. Giao dịch trực tiếp hoặc trung gian.', 'label': 1},
    {'comment': 'Đói quá, có ai ship cơm đêm không?', 'label': 0},
    {'comment': 'Nhìn mặt thằng này hãm vãi.', 'label': 0},
    {'comment': 'Bán key Windows, Office bản quyền giá rẻ cho sinh viên. Dùng vĩnh viễn.', 'label': 1},
    {'comment': 'Có ai đăng ký đi tiếp sức mùa thi không?', 'label': 0},
    {'comment': 'Điểm thi của bạn này ảo thật sự, chắc con ông cháu cha.', 'label': 0},
    {'comment': 'Bạn nào bị hôi nách, hôi chân kinh niên thì đừng bỏ qua sản phẩm này. Hiệu quả ngay lần đầu sử dụng.', 'label': 1},
    {'comment': 'Quên không đóng tiền điện, phòng bị cắt mạng rồi. Đen vcl.', 'label': 0},
    {'comment': 'Mình thấy quan điểm của bạn khá phiến diện.', 'label': 0},
    {'comment': 'Nhận làm con dấu tên, chức danh, shop... Lấy ngay sau 15 phút. Ship cod toàn quốc.', 'label': 1},
    {'comment': 'Có ai biết app nào học tiếng Trung hiệu quả không ạ?', 'label': 0},
    {'comment': 'Cần tìm người đi xem phim chung. Mình bao vé.', 'label': 0},
    {'comment': 'Bạn muốn nói tiếng Anh như người bản xứ? Đăng ký ngay khóa học giao tiếp với GV nước ngoài, ưu đãi 50% cho SV.', 'label': 1},
    {'comment': 'Môn Tư tưởng Hồ Chí Minh có khó không mng?', 'label': 0},
    {'comment': 'Đm cái đứa đi vệ sinh không dội nước trong ktx. Vô ý thức vcl.', 'label': 0},
    {'comment': 'Pass lại thẻ tập gym 3 tháng do mình không có thời gian đi. Giá 500k.', 'label': 1},
    {'comment': 'Cứu, mai thi mà giờ vẫn đang cày phim.', 'label': 0},
    {'comment': 'Đẹp trai quá, cho em xin info với ạ.', 'label': 0},
    {'comment': 'Nhận order hàng US, UK không qua trung gian. Cam kết hàng auth, bill đầy đủ.', 'label': 1},
    {'comment': 'Điểm rèn luyện thấp có ảnh hưởng đến việc xét tốt nghiệp không ạ?', 'label': 0},
    {'comment': 'Thằng bạn mượn tiền mấy tháng không trả. Giờ làm sao đòi được nhỉ?', 'label': 0},
    {'comment': 'VÍ DA NAM CAO CẤP - QUÀ TẶNG Ý NGHĨA CHO NGƯỜI THƯƠNG. Khắc tên miễn phí. Kiểm tra hàng trước khi thanh toán.', 'label': 1},
    {'comment': 'Có ai có link tải sách chuyên ngành này bản pdf không cho mình xin với.', 'label': 0},
    {'comment': 'Thua game cay cú à?', 'label': 0},
    {'comment': 'Tuyển nhân viên bán hàng shop quần áo. Lương 20k/h + thưởng. Ưu tiên sv làm xoay ca.', 'label': 1},
    {'comment': 'Học ngành ngôn ngữ Anh ra có dễ xin việc không các anh chị?', 'label': 0},
    {'comment': 'Cmt này đủ 50 like mình đi nhuộm tóc màu xanh.', 'label': 0},
    {'comment': 'Chỉ 3K mỗi ngày để sở hữu mái tóc dày, óng mượt. Dầu gội thảo dược thiên nhiên, an toàn cho mọi da đầu.', 'label': 1},
    {'comment': 'Có ai bị dị ứng với hải sản không?', 'label': 0},
    {'comment': 'Chỗ này chụp ảnh kỷ yếu có đẹp không mng?', 'label': 0},
    {'comment': 'Tìm đồng đội chơi Pubg tối nay.', 'label': 0},
    {'comment': 'Bán tài khoản Canva Pro, Grammarly Premium giá sinh viên. Dùng chung ổn định.', 'label': 1},
    {'comment': 'Trời mưa, lại lười đi học.', 'label': 0},
    {'comment': 'Mày biết cc gì mà nói.', 'label': 0},
    {'comment': 'Nhận làm móng, nối mi tại nhà. Giá rẻ hơn ngoài tiệm. Book lịch ngay nhé các nàng.', 'label': 1},
    {'comment': 'Thủ tục làm lại thẻ sinh viên bị mất như thế nào vậy ạ?', 'label': 0},
    {'comment': 'Có ai đi xem concert của band này cuối tháng không?', 'label': 0},
    {'comment': 'Bạn nào cần vay nóng 1-5 triệu trong ngày thì dr mình. Thủ tục đơn giản.', 'label': 1},
    {'comment': 'Lại sắp đến hạn đóng học phí rồi. Áp lực ghê.', 'label': 0},
    {'comment': 'Có ai biết thủ tục xin bảo lưu kết quả học tập ntn không ạ?', 'label': 0},
    {'comment': '✨ KÍNH ÁP TRÒNG 1 NGÀY CHỈ 15K/CẶP ✨ Đa dạng mẫu mã, đeo êm như không đeo. Sỉ ib để có giá tốt nhất.', 'label': 1},
    {'comment': 'Cần tìm người đi Đà Lạt cuối tháng này, share tiền phòng, tiền xe cho rẻ.', 'label': 0},
    {'comment': 'Nhận gỡ rối tơ lòng, tư vấn tình cảm. Có duyên mới gặp, phí tùy tâm. Dr mình nếu bạn cần một người lắng nghe.', 'label': 1},
    {'comment': 'Đứa nào đi qua lớp D301 sáng nay có nhặt được cái ô màu đen không cho mình xin lại với.', 'label': 0},
    {'comment': 'Môn Pháp luật đại cương có điểm danh gắt không mọi người?', 'label': 0},
    {'comment': 'TÚI TOTE VẢI CANVAS SIÊU BỀN, ĐỰNG VỪA LAPTOP 14 INCH. Mẫu độc quyền, giá chỉ 79k. Link Shopee ở bio.', 'label': 1},
    {'comment': 'Review đồ ăn ở canteen mới sửa lại đi anh em.', 'label': 0},
    {'comment': 'Mình có nhận làm freelancer mảng content marketing, SEO. Bạn nào có job giới thiệu mình với nhé.', 'label': 0},
    {'comment': 'Đã ngu còn hay lý sự.', 'label': 0},
    {'comment': 'Tuyển 5 bạn livestream bán quần áo. Không yêu cầu kinh nghiệm, chỉ cần hoạt ngôn, tự tin. Lương cứng + % doanh thu.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào photocopy giá rẻ gần trường không ạ?', 'label': 0},
    {'comment': 'Thằng bạn cùng phòng ở bẩn vcl, nói mãi không thay đổi.', 'label': 0},
    {'comment': '💰 BẠN MUỐN TỰ CHỦ TÀI CHÍNH? Tham gia đội nhóm đầu tư của mình, cam kết lợi nhuận 20-30%/tháng. Không đa cấp!', 'label': 1},
    {'comment': 'Góc cảnh báo: Mọi người cẩn thận với shop X nhé, mình mua hàng bị gửi sai mẫu mà shop không cho đổi trả.', 'label': 0},
    {'comment': 'Điểm thi giữa kỳ môn này có trên trang tín chỉ chưa mng?', 'label': 0},
    {'comment': 'Pass lại con chuột gaming không dây do được tặng không có nhu cầu dùng. Mới 100%, giá 400k.', 'label': 1},
    {'comment': 'Sắp tới 20/10, mua quà gì tặng mẹ được nhỉ?', 'label': 0},
    {'comment': 'Nói chuyện không có tí logic nào cả.', 'label': 0},
    {'comment': 'Nhận diệt virus, dọn rác, tăng tốc laptop. Lấy ngay sau 30 phút, giá chỉ 50k.', 'label': 1},
    {'comment': 'Có ai có link đề thi Olympic Tin học sinh viên năm ngoái không?', 'label': 0},
    {'comment': 'Chán, lại phải đi học quân sự rồi.', 'label': 0},
    {'comment': 'Khóa học làm giàu từ bất động sản cho người mới bắt đầu. Bí quyết sở hữu nhà trước tuổi 30. Đăng ký để giữ slot.', 'label': 1},
    {'comment': 'Tự nhiên bị block facebook không rõ lý do. Cay.', 'label': 0},
    {'comment': 'Bạn nào có voucher ahamove không dùng đến cho mình xin với được không ạ?', 'label': 0},
    {'comment': 'Bạn nào cần người mẫu makeup không ạ? Em nhận làm mẫu free để lấy kinh nghiệm.', 'label': 0},
    {'comment': 'Tuyển cộng tác viên bán vé xem phim, chiết khấu cao. Làm online, không cần bỏ vốn.', 'label': 1},
    {'comment': 'Môn này học lại lần thứ 3 rồi. Nản vcl.', 'label': 0},
    {'comment': 'Thằng này trong ảnh nhìn khác hẳn bên ngoài.', 'label': 0},
    {'comment': 'Nhận order trà sữa, đồ ăn vặt ship tận KTX. Freeship cho đơn từ 50k.', 'label': 1},
    {'comment': 'Thủ tục xin rút học phần có phức tạp không mng?', 'label': 0},
    {'comment': 'Thớt cho xin link nhạc được không ạ?', 'label': 0},
    {'comment': 'Cần tìm gia sư môn Hóa cho em lớp 10. Yêu cầu sv chuyên ngành Hóa, có kinh nghiệm. Lương thỏa thuận.', 'label': 1},
    {'comment': 'Đồ án tốt nghiệp có được chọn giảng viên hướng dẫn không ạ?', 'label': 0},
    {'comment': 'Mày là nhất rồi, không ai bằng mày đâu.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ mã giảm giá, voucher Now, Grabfood. Link nhóm ở dưới.', 'label': 1},
    {'comment': 'Có ai biết cách fix lỗi font trong word không?', 'label': 0},
    {'comment': 'Cần tìm người chụp 1 bộ ảnh dã ngoại cuối tuần này. Ai nhận chụp ib báo giá mình với.', 'label': 0},
    {'comment': 'Bán tài khoản Fshare VIP, get link max tốc độ. Giá 20k/tháng.', 'label': 1},
    {'comment': 'Đang học thì mất điện. Số nhọ vcl.', 'label': 0},
    {'comment': 'Cái kết hay quá, xem mà nổi da gà.', 'label': 0},
    {'comment': 'Dịch vụ giặt sấy quần áo, chăn màn lấy ngay. Giá sinh viên, có ship tận nơi.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán bánh tráng trộn ngon không?', 'label': 0},
    {'comment': 'Lại sắp phải chạy deadline rồi, mệt mỏi ghê.', 'label': 0},
    {'comment': 'Bạn nào muốn xăm hình nghệ thuật, an toàn, giá rẻ thì dr cho mình nhé. Đang có chương trình giảm giá 20%.', 'label': 1},
    {'comment': 'Check info giúp em bạn này với ad ơi.', 'label': 0},
    {'comment': 'Cmt vô tri nhưng muốn lên top.', 'label': 0},
    {'comment': 'Tuyển nhân viên nhập liệu cho các sàn TMĐT. Công việc đơn giản, làm tại nhà, thu nhập ổn định.', 'label': 1},
    {'comment': 'Học xong ngành này có sợ thất nghiệp không các tiền bối?', 'label': 0},
    {'comment': 'Đm cái đứa nào lấy trộm sạc dự phòng của tao.', 'label': 0},
    {'comment': 'Bán 1000 file thiết kế có sẵn cho dân đồ họa. Full định dạng, chỉ 99k. Link tải ở bio.', 'label': 1},
    {'comment': 'Có ai chung phòng KTX mà gặp phải đứa ở bẩn, lười biếng không?', 'label': 0},
    {'comment': 'Nên mua laptop hãng nào cho bền nhỉ mng?', 'label': 0},
    {'comment': 'Mở thẻ tín dụng VPBank online, nhận ngay voucher 500k. Không cần chứng minh thu nhập, duyệt trong 15 phút.', 'label': 1},
    {'comment': 'Lại mưa, chán chả muốn đi học.', 'label': 0},
    {'comment': 'Thằng này nói chuyện đạo lý vcl.', 'label': 0},
    {'comment': 'Nhận order giày sneaker rep 1:1, full box, bill. Cam kết hàng đẹp như auth. Check dr để xem mẫu.', 'label': 1},
    {'comment': 'Thư viện có cho mượn sạc laptop không ạ?', 'label': 0},
    {'comment': 'Tìm bạn chơi cầu lông buổi chiều ở sân vận động trường.', 'label': 0},
    {'comment': 'Chỉ cần online 2-3h/ngày, thu nhập 100-200k. Công việc cho các bạn sv muốn kiếm thêm. Ib mình để được hướng dẫn.', 'label': 1},
    {'comment': 'Đồ ăn ở căn tin dạo này lên giá hay sao ấy nhỉ?', 'label': 0},
    {'comment': 'Bài viết này nông cạn quá.', 'label': 0},
    {'comment': 'Pass lại sách tiếng Anh chuyên ngành còn mới. Mua 150k pass 70k.', 'label': 1},
    {'comment': 'Có ai biết lịch nghỉ Tết của trường mình chưa?', 'label': 0},
    {'comment': 'Điểm GPA thấp có ảnh hưởng đến việc apply học bổng không ạ?', 'label': 0},
    {'comment': 'Bạn nào cần người tâm sự, trò chuyện đêm khuya không? Mình FA, rảnh lắm.', 'label': 0},
    {'comment': 'Sim data 4G không giới hạn dung lượng, giá chỉ 50k/tháng. Mua ngay kẻo hết.', 'label': 1},
    {'comment': 'Vc, crush vừa rep story mình. Vui quá.', 'label': 0},
    {'comment': 'Mày thì biết cc gì mà phán.', 'label': 0},
    {'comment': 'Nhận trông nhà, chăm thú cưng dịp lễ Tết. Sinh viên Bách Khoa, thật thà, chăm chỉ. Ib để trao đổi.', 'label': 1},
    {'comment': 'Mọi người cho em hỏi cách liên hệ với văn phòng Đoàn trường với ạ.', 'label': 0},
    {'comment': 'Cần tìm người yêu đi chơi trung thu.', 'label': 0},
    {'comment': 'Bạn nào bị viêm xoang, viêm mũi dị ứng lâu năm không khỏi thì dr mình nhé. Thuốc gia truyền đảm bảo hiệu quả.', 'label': 1},
    {'comment': 'Môn Kỹ năng giao tiếp có phải thuyết trình nhiều không mng?', 'label': 0},
    {'comment': 'Lại một kỳ nữa sắp qua mà vẫn chưa có mảnh tình vắt vai.', 'label': 0},
    {'comment': 'GOM ORDER BÁNH TRÁNG PHƠI SƯƠNG TÂY NINH CHÍNH GỐC. Đầy đủ topping, 30k/phần.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào sửa khóa balo uy tín không?', 'label': 0},
    {'comment': 'Nhìn cái avatar đã không muốn nói chuyện rồi.', 'label': 0},
    {'comment': 'Tuyển 10 bạn làm Airdrop, Bounty cho các dự án Crypto. Không cần vốn, chỉ cần siêng năng. Thu nhập không giới hạn.', 'label': 1},
    {'comment': 'Có ai có file excel tính điểm GPA tự động không cho mình xin với.', 'label': 0},
    {'comment': 'Chỗ này gửi xe có đắt không mọi người?', 'label': 0},
    {'comment': 'Pass lại khóa học lập trình Python trên Udemy. Mua 500k pass 100k.', 'label': 1},
    {'comment': 'Học xong mệt quá, có ai đi ăn gì không?', 'label': 0},
    {'comment': 'Ad duyệt bài nhanh giúp em với ạ.', 'label': 0},
    {'comment': 'Bán bộ bài tarot cho người mới bắt đầu, kèm sách hướng dẫn chi tiết. Giá chỉ 150k.', 'label': 1},
    {'comment': 'Có ai biết cách cải thiện điểm tiếng Anh không?', 'label': 0},
    {'comment': 'Thằng này đẹp trai mà nói chuyện vô duyên vcl.', 'label': 0},
    {'comment': 'Nhận vẽ chibi, tranh chân dung theo yêu cầu. Giá hạt dẻ, làm quà tặng siêu ý nghĩa.', 'label': 1},
    {'comment': 'Lịch học lại đã có chưa vậy mng?', 'label': 0},
    {'comment': 'Mọi người cho em hỏi học cải thiện điểm thì điểm cũ có bị xóa đi không ạ?', 'label': 0},
    {'comment': '🔥 VOUCHER GRAB/BE GIẢM 50% - SỐ LƯỢNG CÓ HẠN 🔥 Chỉ 10k/voucher. Mua 5 tặng 1. Ib ngay để chốt đơn.', 'label': 1},
    {'comment': 'Cần tìm 1 bạn edit video Capcut cho bài tập nhóm. Có trả phí ạ.', 'label': 0},
    {'comment': 'Nhận báo cáo thực tập, khóa luận tốt nghiệp trọn gói. Đảm bảo đúng hạn, chuẩn yêu cầu, không đạo văn. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Đứa nào đi xe wave trắng để ở nhà xe D sáng nay quên gạt chân chống đấy.', 'label': 0},
    {'comment': 'Có ai biết cách xin giấy tạm hoãn nghĩa vụ quân sự không, chỉ mình với.', 'label': 0},
    {'comment': 'SERUM MỌC LÔNG MÀY, LÔNG MI. Hiệu quả sau 2 tuần sử dụng. Cam kết hàng chính hãng, check mã vạch thoải mái.', 'label': 1},
    {'comment': 'Review CLB Marketing của trường mình đi mng. Em là k67 muốn apply.', 'label': 0},
    {'comment': 'Mình có nhận order cơm văn phòng, cơm suất sinh viên. Ship tận nơi, thực đơn thay đổi mỗi ngày. Link page ở bio.', 'label': 1},
    {'comment': 'Đm cái đứa nào hát karaoke trong ktx giờ này vậy?', 'label': 0},
    {'comment': 'Đẹp trai/xinh gái mà vô duyên thì cũng vứt.', 'label': 0},
    {'comment': 'Bán tài khoản Netflix Premium xem phim 4K không giới hạn, giá chỉ 40k/tháng. Bảo hành full time.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán sạc laptop Dell chính hãng không?', 'label': 0},
    {'comment': 'Cần tìm người chung tiền mua tài khoản Grammarly Premium dùng chung.', 'label': 0},
    {'comment': 'Mày nghĩ mày là ai mà nói chuyện kiểu đấy?', 'label': 0},
    {'comment': 'Tuyển 20 bạn nam nữ phát tờ rơi cho trung tâm tiếng Anh. Lương 150k/ca 3 tiếng. Làm ngay trong tuần.', 'label': 1},
    {'comment': 'Học phí của trường mình so với các trường khác có cao không ạ?', 'label': 0},
    {'comment': 'Ad ơi, group mình có nên lập 1 bài ghim các câu hỏi thường gặp cho tân sinh viên không ạ?', 'label': 0},
    {'comment': 'Pass lại cây đàn guitar classic do không có thời gian tập. Tặng kèm bao da, capo. Giá 700k.', 'label': 1},
    {'comment': 'Căng tin có món gì mới không chúng mày?', 'label': 0},
    {'comment': 'Cần tìm một bạn nữ hiền lành, sạch sẽ ở ghép. Phòng đầy đủ đồ, gần trạm xe bus, giá 1tr/người.', 'label': 0},
    {'comment': 'Nhận tăng điểm Google Maps, review 5 sao cho quán ăn, shop. Giúp tăng uy tín, kéo khách hàng.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán hoa sáp đẹp mà giá rẻ không?', 'label': 0},
    {'comment': 'Thằng bạn thân có người yêu xong quên mẹ mình luôn. Buồn vcl.', 'label': 0},
    {'comment': 'Dịch vụ SMS Banking, xác nhận OTP, tạo tài khoản ảo. Nhanh chóng, bảo mật. Ib để biết thêm chi tiết.', 'label': 1},
    {'comment': 'Môn Kinh tế chính trị có bài tập nhóm không mng?', 'label': 0},
    {'comment': 'Nói chuyện như này thì bạn nên đi học lại lớp 1.', 'label': 0},
    {'comment': 'Bán 500GB tài liệu marketing, chạy quảng cáo Facebook, Google, Tiktok. Giá chỉ 149k trọn bộ.', 'label': 1},
    {'comment': 'Lịch nghỉ lễ 30/4-1/5 của trường có chưa ạ?', 'label': 0},
    {'comment': 'Có ai biết cách kết nối máy in ở thư viện không?', 'label': 0},
    {'comment': 'Bạn nào cần người chạy deadline code, fix bug thì hú mình nhé. Nhận các job Python, Java, C++.', 'label': 1},
    {'comment': 'Mới đầu tháng mà đã hết tiền rồi. Đau khổ.', 'label': 0},
    {'comment': 'Bài này mà cũng được duyệt lên page á ad?', 'label': 0},
    {'comment': 'Ốp lưng, cường lực điện thoại đồng giá 19k. Rẻ nhất Vịnh Bắc Bộ. Freeship cho đơn từ 50k.', 'label': 1},
    {'comment': 'Có ai đi xem giải bóng rổ sinh viên chiều nay không?', 'label': 0},
    {'comment': 'Mày tưởng mày hay lắm à?', 'label': 0},
    {'comment': 'Nhận unlock, mở khóa iCloud iPhone, iPad. Giá cả tùy tình trạng máy. Uy tín, làm xong mới lấy tiền.', 'label': 1},
    {'comment': 'Quên mang áo mưa, giờ mưa to thế này sao về.', 'label': 0},
    {'comment': 'Có ai biết cách đăng ký hiến máu tình nguyện của trường không?', 'label': 0},
    {'comment': 'Bạn đang gặp vấn đề về TÀI CHÍNH? Vay tiền sinh viên, giải ngân 10 phút, không gọi người thân. Dr mình tư vấn.', 'label': 1},
    {'comment': 'Học xong 4 năm cảm thấy kiến thức như một trang giấy trắng.', 'label': 0},
    {'comment': 'Xem mà tức á. Diễn viên chính diễn như cc.', 'label': 0},
    {'comment': 'Tuyển người mẫu ảnh cho bộ sưu tập áo dài Tết. Yêu cầu tóc dài, mặt ưa nhìn. Cast cuối tuần này.', 'label': 1},
    {'comment': 'Có ai biết cách xin visa du lịch Hàn Quốc cho sinh viên không ạ?', 'label': 0},
    {'comment': 'Thằng nyc vừa add friend lại. Nó muốn cc gì vậy chúng m?', 'label': 0},
    {'comment': 'Chuyên cung cấp sỉ lẻ yến sào, saffron chính hãng. Bồi bổ sức khỏe, làm đẹp da. Ib để được tư vấn.', 'label': 1},
    {'comment': 'Môn thể dục nên đăng ký chạy hay bóng rổ dễ qua hơn ạ?', 'label': 0},
    {'comment': 'Cmt để đánh dấu, tối về đọc.', 'label': 0},
    {'comment': 'Pass lại vé concert Westlife do trùng lịch thi. Giá gốc không phụ phí. Ai cần dr gấp.', 'label': 1},
    {'comment': 'Phòng trọ có cho nuôi chó không nhỉ mng?', 'label': 0},
    {'comment': 'Cãi nhau với đứa bạn thân chỉ vì mấy chuyện vớ vẩn. Mệt.', 'label': 0},
    {'comment': 'Nhận order sạc, cáp, tai nghe ZIN bóc máy. Cam kết chính hãng Apple, bao test trọn đời.', 'label': 1},
    {'comment': 'Học ngành này có cần laptop cấu hình cao không ạ?', 'label': 0},
    {'comment': 'Ngu thì dựa cột mà nghe, đừng có phát biểu.', 'label': 0},
    {'comment': 'Tham gia nhóm trading coin, phím lệnh hàng ngày, tỉ lệ thắng 90%. Về bờ an toàn. Link Zalo trong ảnh.', 'label': 1},
    {'comment': 'Có ai biết quán net nào có máy cấu hình cao để chơi game AAA không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm người với ạ. Em muốn tìm bạn nam mặc áo xanh tình nguyện giúp em dắt xe hôm qua ở nhà G.', 'label': 0},
    {'comment': 'Bán bộ dụng cụ làm nail tại nhà cho người mới bắt đầu. Full đồ chỉ 250k.', 'label': 1},
    {'comment': 'Lại sắp phải chuyển trọ rồi. Ngán ngẩm.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán đồ ăn vặt ship đêm không?', 'label': 0},
    {'comment': 'Bạn muốn nói giọng miền Nam chuẩn? Đăng ký ngay khóa học luyện giọng online, giáo viên 1 kèm 1.', 'label': 1},
    {'comment': 'Điểm rèn luyện để làm gì vậy mng?', 'label': 0},
    {'comment': 'Bài viết này cop của người khác mà không ghi nguồn à?', 'label': 0},
    {'comment': 'Nhận làm bằng tốt nghiệp, chứng chỉ tiếng Anh, tin học. Phôi thật, tem thật, bao soi rọi.', 'label': 1},
    {'comment': 'Có ai có đề cương môn Triết học Mác-Lênin không cho mình xin với.', 'label': 0},
    {'comment': 'Trời nồm ẩm khó chịu vcl.', 'label': 0},
    {'comment': 'Pass lại bàn phím cơ do không quen dùng. Mới mua 1 tuần, full box. Giá 500k.', 'label': 1},
    {'comment': 'Sắp đến sinh nhật mình mà chả có ai nhớ.', 'label': 0},
    {'comment': 'Mày với nó đẹp đôi đấy. Rác với rác.', 'label': 0},
    {'comment': 'Bạn nào cần ship hỏa tốc tài liệu, đồ đạc trong nội thành Hà Nội thì alo mình nhé. Giá sinh viên.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán cây cảnh để bàn mini không?', 'label': 0},
    {'comment': 'Tự nhiên thèm nem chua rán quá.', 'label': 0},
    {'comment': 'Dịch vụ vệ sinh, dọn dẹp phòng trọ cuối tuần. Sạch bong kin kít, giá chỉ từ 100k/phòng.', 'label': 1},
    {'comment': 'Mọi người có ai apply học bổng của trường năm nay không?', 'label': 0},
    {'comment': 'Đm cái đứa nào đi xe đạp điện trong sân trường mà phóng nhanh vượt ẩu.', 'label': 0},
    {'comment': 'Bạn nào bị mồ hôi tay, mồ hôi chân thì về đội của em. Serum thảo dược cam kết trị dứt điểm.', 'label': 1},
    {'comment': 'Thầy X dạy môn này có hay cho thêm điểm chuyên cần không ạ?', 'label': 0},
    {'comment': 'Cần tìm người quay tiktok cùng. Mình có ý tưởng nhưng không biết quay.', 'label': 0},
    {'comment': 'Bán tài khoản ChatGPT Plus giá rẻ, dùng chung ổn định, không bị kick. Ib để chốt đơn.', 'label': 1},
    {'comment': 'Học xong mệt quá, đi lượn hồ Tây không chúng mày?', 'label': 0},
    {'comment': 'Nói chuyện không đầu không đuôi, đọc chả hiểu gì.', 'label': 0},
    {'comment': 'Nhận đánh bóng, làm mới giày sneaker. Tẩy ố, repaint chuyên nghiệp. Lấy lại vẻ đẹp cho đôi giày của bạn.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký thẻ thư viện liên kết không?', 'label': 0},
    {'comment': 'Crush không rep tin nhắn. Chắc tạch rồi.', 'label': 0},
    {'comment': 'Bán tài khoản xem phim trên các nền tảng HÀNG ĐẦU. Giá chỉ bằng 1/10 giá gốc. Ib để nhận danh sách.', 'label': 1},
    {'comment': 'Có ai biết cách ẩn bạn bè trên facebook không?', 'label': 0},
    {'comment': 'Thằng này lại bắt đầu sủa rồi.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page fan club idol Hàn. Yêu cầu online thường xuyên, am hiểu về nhóm. Lương thỏa thuận.', 'label': 1},
    {'comment': 'Nên đi thực tập từ năm mấy thì hợp lý ạ?', 'label': 0},
    {'comment': 'Có ai có link full trận bóng tối qua không?', 'label': 0},
    {'comment': 'Cần pass gấp cái máy sấy tóc do chuyển trọ có sẵn rồi. Hàng Philip còn tốt, giá 100k.', 'label': 1},
    {'comment': 'Học online chán thật sự, chả có động lực gì.', 'label': 0},
    {'comment': 'Mọi người ơi, thi lại thì lấy điểm lần 1 hay lần 2 ạ?', 'label': 0},
    {'comment': 'TRÀ SỮA ĐỒNG GIÁ 19K - DUY NHẤT HÔM NAY. Áp dụng cho toàn menu. Tag bạn bè vào rủ đi uống ngay nào.', 'label': 1},
    {'comment': 'Cần tìm người làm chung bài tập lớn môn Mạng máy tính. Yêu cầu biết code C++, không báo team.', 'label': 0},
    {'comment': 'Nhận thiết kế logo, banner, CV, portfolio. Chuyên nghiệp, sáng tạo, giá sinh viên. Ib để xem sản phẩm.', 'label': 1},
    {'comment': 'Có đứa nào đi học muộn bị ghi sổ đầu bài chưa?', 'label': 0},
    {'comment': 'Phòng trọ em còn 1 chỗ trống cho nữ. Phòng 2tr5/tháng, đầy đủ đồ, an ninh tốt. Ai cần dr mình nhé.', 'label': 0},
    {'comment': 'BỎ VÀI ĐỒNG BẠC, NHẬN LẠI VÀI CHỤC TRIỆU. Cơ hội đầu tư sinh lời X10 tài khoản chỉ trong 1 đêm. Ai muốn biết cách thì chấm.', 'label': 1},
    {'comment': 'Review công ty thực tập FPT software đi mng ơi.', 'label': 0},
    {'comment': 'Mình có nhận làm mô hình kiến trúc cho sinh viên. Đảm bảo đẹp, đúng tiến độ. Liên hệ để được báo giá.', 'label': 1},
    {'comment': 'Mày học trường nào mà phát biểu nghe thiểu năng vậy?', 'label': 0},
    {'comment': 'Flex nhẹ chiếc mail báo trúng tuyển intern.', 'label': 0},
    {'comment': 'Bán giày Jordan 1 rep 1:1, đi 2-3 lần còn như mới. Size 42, giá 450k. Có fix cho ai nhiệt tình.', 'label': 1},
    {'comment': 'Có ai biết cách xin xác nhận là sinh viên để vay vốn ngân hàng không ạ?', 'label': 0},
    {'comment': 'Cần tìm người make up chụp kỷ yếu. Yêu cầu tone trong trẻo, tự nhiên.', 'label': 0},
    {'comment': 'Đã xấu còn hay ra vẻ ta đây.', 'label': 0},
    {'comment': 'Tuyển 5 bạn nữ PG đứng booth game tại hội chợ. Cao trên 1m6, mặt xinh, lương 600k/ngày + thưởng.', 'label': 1},
    {'comment': 'Môn Xác suất thống kê có được mang bảng công thức vào phòng thi không mng?', 'label': 0},
    {'comment': 'Góc tìm người: Tìm bạn nam mặc áo sơ mi trắng ngồi cạnh mình ở thư viện sáng nay. Bạn làm rơi cái bút, mình muốn trả lại.', 'label': 0},
    {'comment': 'Bạn nào đang bị tình trạng "RAU MÙI" cơ thể thì đừng bỏ qua em lăn khử mùi này nhé. Khô thoáng 48h.', 'label': 1},
    {'comment': 'Có ai đăng ký cuộc thi khởi nghiệp của trường không?', 'label': 0},
    {'comment': 'Cần tìm một bạn biết chơi guitar đệm hát cho tiết mục văn nghệ.', 'label': 0},
    {'comment': 'Nhận chạy deadline, code dạo, fix bug giá sinh viên. Nhận mọi kèo từ dễ đến khó.', 'label': 1},
    {'comment': 'Trời Hà Nội dạo này ẩm ương khó chịu thật sự.', 'label': 0},
    {'comment': 'Thằng bạn mượn con lap của mình về làm đổ nước vào. Giờ bắt đền ntn đây chúng m?', 'label': 0},
    {'comment': 'Dịch vụ chép phạt, viết tay thuê. Nét chữ đẹp, sạch sẽ, trả bài đúng hẹn. Giá chỉ 5k/trang.', 'label': 1},
    {'comment': 'Môn Chủ nghĩa xã hội khoa học có điểm danh không ạ?', 'label': 0},
    {'comment': 'Nói chuyện kiểu này chắc học hết lớp 12 chưa?', 'label': 0},
    {'comment': 'Bán bộ bài ma sói ultimate deluxe full box, chơi được hơn 20 người. Giá 150k.', 'label': 1},
    {'comment': 'Lịch bảo vệ đồ án tốt nghiệp có chưa vậy mng?', 'label': 0},
    {'comment': 'Có ai biết cách dùng Canva Pro miễn phí không?', 'label': 0},
    {'comment': 'Tuyển mẫu tóc miễn phí cho salon. Uốn/nhuộm/cắt theo yêu cầu, chỉ cần chụp ảnh feedback.', 'label': 1},
    {'comment': 'Lại sắp hết một năm học mà cảm thấy chưa làm được gì.', 'label': 0},
    {'comment': 'Bài viết này xàm l quá ad ơi.', 'label': 0},
    {'comment': 'GOM ORDER NEM NƯỚNG NHA TRANG. 35K/suất đầy đủ topping. Chốt đơn trước 10h sáng mai.', 'label': 1},
    {'comment': 'Có ai thi TOEIC ở IIG về cho em xin ít review với ạ.', 'label': 0},
    {'comment': 'Mày không nói thì không ai bảo mày câm đâu.', 'label': 0},
    {'comment': 'Nhận thay màn hình, ép kính điện thoại iPhone, Samsung. Lấy ngay sau 30 phút, bảo hành 6 tháng.', 'label': 1},
    {'comment': 'Ở KTX có được nuôi hamster không mng?', 'label': 0},
    {'comment': 'Cần tìm người thông não hộ bài tập Lập trình hướng đối tượng.', 'label': 0},
    {'comment': 'Bạn muốn có một CV ấn tượng trong mắt nhà tuyển dụng? Gửi CV của bạn cho chúng tôi để được review và chỉnh sửa miễn phí.', 'label': 1},
    {'comment': 'Sắp Tết rồi mà chưa có tiền về quê.', 'label': 0},
    {'comment': 'Xem phim mà gặp mấy đứa vô ý thức nói chuyện, spoil phim tức vcl.', 'label': 0},
    {'comment': 'Tuyển nhân viên pha chế cho quán cafe sách. Ưu tiên các bạn có kinh nghiệm, có thể làm full-time.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký học cùng lúc hai trường không?', 'label': 0},
    {'comment': 'Crush vừa like ảnh mình. Liệu có hy vọng không chúng m?', 'label': 0},
    {'comment': 'Chuyên cung cấp các loại lens, kính áp tròng cận, loạn. Hàng nội địa Hàn, đeo êm, không cộm. Dr để được tư vấn độ cận.', 'label': 1},
    {'comment': 'Môn thể chất có bắt buộc phải đi học đầy đủ không ạ?', 'label': 0},
    {'comment': 'Cmt để hóng biến.', 'label': 0},
    {'comment': 'Pass lại máy tính Casio 580VNX do đã ra trường không dùng đến. Máy còn mới, giá 300k.', 'label': 1},
    {'comment': 'Phòng bên cạnh ồn ào quá không học được.', 'label': 0},
    {'comment': 'Bị người yêu cắm sừng. Cảm giác thật cc.', 'label': 0},
    {'comment': 'Nhận ship đồ ăn đêm khu vực ký túc xá. Mỳ tôm trứng, chân gà, xúc xích... Alo là có.', 'label': 1},
    {'comment': 'Học ngành logistics ra trường có dễ xin việc không ạ?', 'label': 0},
    {'comment': 'Não không dùng đến thì hiến tặng đi bạn ơi.', 'label': 0},
    {'comment': 'Tham gia group kèo thơm bóng đá, tip free mỗi ngày, tỉ lệ thắng cao. Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán bún riêu nào ngon gần trường không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài pass đồ với ạ. Em có nhiều quần áo mặc không vừa muốn pass lại giá rẻ cho các bạn sv.', 'label': 0},
    {'comment': 'Bán tài khoản Adobe full app (Photoshop, AI, Premiere...) bản quyền. Giá chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Sắp đến hạn báo cáo thực tập rồi mà chưa xin được dấu.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào dán skin laptop đẹp không?', 'label': 0},
    {'comment': 'Bạn muốn kiếm 5-7 triệu/tháng chỉ với việc xem video và like dạo? Ib mình chỉ cách, không mất phí.', 'label': 1},
    {'comment': 'Đăng ký tín chỉ còn khó hơn cả săn sale.', 'label': 0},
    {'comment': 'Bài này copy mà không ghi nguồn, ý thức như cc.', 'label': 0},
    {'comment': 'Nhận làm slide Powerpoint, Prezi, Canva. Sáng tạo, chuyên nghiệp, giá chỉ từ 10k/slide.', 'label': 1},
    {'comment': 'Có ai biết cách chuyển file pdf sang word không bị lỗi font không?', 'label': 0},
    {'comment': 'Đang phân vân giữa việc đi làm thêm và tập trung học.', 'label': 0},
    {'comment': 'Pass lại thẻ bơi 10 buổi ở bể bơi trường. Giá 200k. Ai lấy dr mình.', 'label': 1},
    {'comment': 'Trượt phỏng vấn. Buồn.', 'label': 0},
    {'comment': 'Mày là cái thá gì mà lên mặt dạy đời?', 'label': 0},
    {'comment': 'Tuyển cộng tác viên chốt đơn hàng thời trang online. Chỉ cần có điện thoại, làm việc tại nhà, hoa hồng cao.', 'label': 1},
    {'comment': 'Thủ tục xin cấp lại thẻ sinh viên bị mất như thế nào ạ?', 'label': 0},
    {'comment': 'Cần tìm người dạy kèm guitar tại nhà.', 'label': 0},
    {'comment': 'Bán voucher Buffet Sen Tây Hồ giá gốc. Hạn sử dụng còn dài. Ai cần liên hệ mình.', 'label': 1},
    {'comment': 'Học online thế này thì kiến thức vào đầu được bao nhiêu.', 'label': 0},
    {'comment': 'Đm cái đứa nào lấy xe của tao mà không đổ xăng.', 'label': 0},
    {'comment': 'Nhận mix nhạc, làm beat theo yêu cầu. Giá cả phải chăng cho các bạn yêu rap, ca hát.', 'label': 1},
    {'comment': 'Có ai biết cách liên hệ với phòng công tác sinh viên không?', 'label': 0},
    {'comment': 'Đứa nào có link xem The Voice tối qua không?', 'label': 0},
    {'comment': 'Pass lại bộ sách IELTS Cambridge từ 10-17. Sách photo nhưng còn mới, giá 150k/bộ.', 'label': 1},
    {'comment': 'Lại sắp đến mùa chạy deadline sml rồi anh em ạ.', 'label': 0},
    {'comment': 'Nói chuyện ngu như m thì tốt nhất nên im lặng.', 'label': 0},
    {'comment': 'Bạn muốn x2, x3 tài sản của mình? Tham gia ngay nhóm tín hiệu Forex của chúng tôi. Hoàn toàn miễn phí.', 'label': 1},
    {'comment': 'Có ai đi xe bus về Hà Đông không cho mình đi ké với.', 'label': 0},
    {'comment': 'Tự nhiên thèm ốc luộc chấm mắm gừng quá.', 'label': 0},
    {'comment': 'Nhận đặt vé máy bay, vé tàu xe dịp Tết. Giữ chỗ giá tốt, không lo hết vé.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào làm thêm theo giờ không ạ?', 'label': 0},
    {'comment': 'Bài viết này như đấm vào mặt mình.', 'label': 0},
    {'comment': 'Bán acc game Valorant rank kim cương, nhiều skin súng. Giao dịch trung gian cho an toàn.', 'label': 1},
    {'comment': 'Đi học cả ngày về mệt rã rời.', 'label': 0},
    {'comment': 'Đã dốt còn hay tỏ ra nguy hiểm.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực quán net ca đêm. Lương 5tr/tháng + bao ăn ở.', 'label': 1},
    {'comment': 'Môn Triết có dễ qua không mng?', 'label': 0},
    {'comment': 'Mọi người ơi, CLB tình nguyện của trường mình có đợt tuyển thành viên mới chưa ạ?', 'label': 0},
    {'comment': '✨ SERUM TRỊ THÂM MỤN THẦN THÁNH ✨ Cam kết mờ thâm, sáng da chỉ sau 7 ngày. Mua 1 tặng 1 duy nhất hôm nay. 👉 Dr ngay!', 'label': 1},
    {'comment': 'Cần tìm người share acc Netflix, 40k/tháng xem 4K tẹt ga.', 'label': 0},
    {'comment': '🚀 BẠN MUỐN TỰ DO TÀI CHÍNH? 🚀 Tham gia nhóm tín hiệu trade coin của mình. Tỉ lệ thắng 95%. Hoàn toàn miễn phí cho người mới. Link ở bio!', 'label': 1},
    {'comment': 'Đứa nào đi học mà để xe chắn hết cả lối ra vậy. Ý thức như cc.', 'label': 0},
    {'comment': 'Có ai biết chỗ in tài liệu nào mở cửa chủ nhật không ạ?', 'label': 0},
    {'comment': '💻 VIỆC LÀM THÊM CHO SINH VIÊN 💻 Tuyển 20 bạn CTV đăng bài, lương 3-5tr/tháng. Không cần kinh nghiệm. Chấm (.) mình gửi JD.', 'label': 1},
    {'comment': 'Review giáo trình mới của môn Triết đi mọi người. Có dễ hiểu hơn sách cũ không?', 'label': 0},
    {'comment': 'Mình có nhận làm bánh sinh nhật, bánh kem theo yêu cầu. 🎂 Freeship khu vực Cầu Giấy. Ib để xem mẫu nhé.', 'label': 1},
    {'comment': 'Não mày chứa gì ngoài bã đậu à?', 'label': 0},
    {'comment': 'Flex nhẹ con ảnh kỷ yếu vừa chụp chiều nay.', 'label': 0},
    {'comment': '🎧 TAI NGHE BLUETOOTH HỔ VẰN 1562AE CHỈ 399K 🎧 Âm thanh 9/10 so với auth, full chức năng. Bảo hành 6 tháng 1 đổi 1.', 'label': 1},
    {'comment': 'Có ai biết cách xin phúc khảo bài thi cuối kỳ không ạ?', 'label': 0},
    {'comment': 'Cần tìm bạn nữ chụp ảnh cho shop thời trang. Không cần kinh nghiệm, chỉ cần dáng người cân đối, ăn ảnh.', 'label': 0},
    {'comment': 'Đã dốt lại còn hay đi cà khịa.', 'label': 0},
    {'comment': '🆘 CẦN TIỀN GẤP TRONG NGÀY? 🆘 Vay online chỉ cần CMND, giải ngân sau 5 phút. Hỗ trợ nợ xấu. Tải app tại link bio.', 'label': 1},
    {'comment': 'Mọi người ơi, ngày hội việc làm của trường mình tổ chức ở đâu vậy ạ?', 'label': 0},
    {'comment': 'Góc tìm đồ: Sáng nay mình có để quên 1 chiếc áo khoác màu đen ở giảng đường G201. Ai nhặt được cho mình xin lại với ạ.', 'label': 0},
    {'comment': '👗 XẢ KHO TOÀN BỘ VÁY THIẾT KẾ - ĐỒNG GIÁ 99K 👗 Nhanh tay kẻo hết size các nàng ơi. Ship COD toàn quốc.', 'label': 1},
    {'comment': 'Có ai đi xem concert cuối tuần này không, lập team đi chung cho vui.', 'label': 0},
    {'comment': 'Cần tìm 1 bạn nam ở ghép, phòng rộng rãi, có ban công, 1tr2/người. Yêu cầu sạch sẽ, không hút thuốc.', 'label': 0},
    {'comment': '🔮 Bói bài Tarot online - Thấu hiểu bản thân, định hướng tương lai 🔮 Xem chi tiết tình duyên, công việc. Phí tùy tâm. Dr để đặt lịch.', 'label': 1},
    {'comment': 'Web trường lại sập rồi, không xem được điểm.', 'label': 0},
    {'comment': 'Thằng bạn thân vừa tỏ tình. Giờ phải làm sao đây chúng mày?', 'label': 0},
    {'comment': '💸 KIẾM TIỀN CHƯA BAO GIỜ DỄ ĐẾN THẾ 💸 Chỉ cần treo app, mỗi ngày kiếm 50-100k. Không cần làm gì cả. Link tải ở comment.', 'label': 1},
    {'comment': 'Môn Cơ sở dữ liệu có khó không các anh chị đi trước?', 'label': 0},
    {'comment': 'Nói chuyện với mày tốn calo thật sự.', 'label': 0},
    {'comment': 'Bán tài khoản Spotify Premium, nghe nhạc không quảng cáo. Giá chỉ 20k/tháng. 🔥', 'label': 1},
    {'comment': 'Lịch nghỉ Tết âm của trường mình được bao nhiêu ngày vậy mng?', 'label': 0},
    {'comment': 'Có ai biết cách viết mail xin thực tập cho chuyên nghiệp không?', 'label': 0},
    {'comment': 'Bạn nào bị các vấn đề về dạ dày, đại tràng thì ib mình nhé. 🌿 Thuốc nam gia truyền hiệu quả lắm ạ.', 'label': 1},
    {'comment': 'Chạy deadline xong chắc gầy đi mấy cân.', 'label': 0},
    {'comment': 'Post bài câu like rẻ tiền.', 'label': 0},
    {'comment': 'GOM ORDER ĐỒ ĂN VẶT CỔNG TRƯỜNG 🍡 Nem chua rán, khoai tây lốc xoáy... 5h chiều mình trả đơn ở KTX nhé.', 'label': 1},
    {'comment': 'Có ai thi VNU-EPT rồi cho em xin ít kinh nghiệm với ạ.', 'label': 0},
    {'comment': 'Mày là chuyên gia bẻ lái à?', 'label': 0},
    {'comment': 'Nhận order giày MLB, Vans, Converse chính hãng. 🔥 Sale 30-50% so với store. Hàng về sau 2 tuần.', 'label': 1},
    {'comment': 'Ở thư viện có ổ cắm điện không mng?', 'label': 0},
    {'comment': 'Tìm đồng đội cày rank Liên Quân tối nay. Mình chơi SP.', 'label': 0},
    {'comment': '💪 Cần tìm PT hướng dẫn tập gym. Em là nữ, muốn tăng vòng 3. Ai nhận ib em báo giá với ạ.', 'label': 0},
    {'comment': 'Pass lại voucher buffet lẩu nướng 200k. Hạn dùng đến cuối tháng. Ai lấy mình để lại 120k.', 'label': 1},
    {'comment': 'Chán, lại một mùa noel một mình.', 'label': 0},
    {'comment': 'Xem phim mà gặp đứa ngồi sau gác chân lên ghế. Vô ý thức vcl.', 'label': 0},
    {'comment': 'Bạn nào cần mua acc Netflix, HBO Go, Disney+ thì hú mình nhé. 🎬 Giá sinh viên, bảo hành uy tín.', 'label': 1},
    {'comment': 'Học ngành Quản trị kinh doanh sau này có dễ thất nghiệp không ạ?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán xôi khúc ngon ở gần đây không?', 'label': 0},
    {'comment': 'Tuyển 10 bạn làm khảo sát thị trường online. Mỗi khảo sát 10-15 phút, nhận 20-30k. 💰 Ib để nhận link.', 'label': 1},
    {'comment': 'Sắp phải chọn chuyên ngành rồi mà vẫn mông lung quá.', 'label': 0},
    {'comment': 'Mày tưởng mày là trung tâm của vũ trụ à?', 'label': 0},
    {'comment': 'Nhận làm video intro, outro cho các bạn làm Youtube, Tiktok. Chuyên nghiệp, giá chỉ từ 50k. 🎥', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký tham gia chương trình trao đổi sinh viên không?', 'label': 0},
    {'comment': 'Thằng bạn cùng phòng đêm nào cũng gọi điện cho người yêu nói chuyện đến 2h sáng. Bực mình.', 'label': 0},
    {'comment': 'SALE SỐC ‼️ Nước hoa chiết vial chính hãng chỉ từ 79k. Thơm sang, giữ mùi lâu. Check highlight để xem list mùi nhé.', 'label': 1},
    {'comment': 'Điểm giữa kỳ môn này thấp quá, có cách nào gỡ lại không mng?', 'label': 0},
    {'comment': 'Đã xấu còn tỏ ra thượng đẳng.', 'label': 0},
    {'comment': 'Bán data khách hàng tiềm năng, phù hợp cho các bạn sale BĐS, bảo hiểm. ✅ Data chất, tỉ lệ chuyển đổi cao.', 'label': 1},
    {'comment': 'Thủ tục làm lại thẻ BHYT cho sinh viên như thế nào vậy ạ?', 'label': 0},
    {'comment': 'Có ai có link download bộ Office full crack không cho mình xin với.', 'label': 0},
    {'comment': 'Bạn nào cần người đi chợ, nấu cơm hộ thì liên hệ mình nhé. 👩‍🍳 Đảm bảo ngon, sạch, giá cả hợp lý.', 'label': 1},
    {'comment': 'Căng tin trường mình có thanh toán bằng thẻ không mng?', 'label': 0},
    {'comment': 'Bị bồ đá chỉ vì không có xe xịn. Cuộc sống.', 'label': 0},
    {'comment': 'Pass lại cặp vé xem kịch cuối tuần này ở nhà hát Tuổi Trẻ. Mua 500k pass 350k. 🎭', 'label': 1},
    {'comment': 'Sắp tốt nghiệp rồi, cảm thấy hơi lo lắng.', 'label': 0},
    {'comment': 'Bài viết nhảm nhí, câu view.', 'label': 0},
    {'comment': 'Nhận thông tắc cống, bồn cầu. 🚽 Có mặt sau 15 phút. Xử lý triệt để, không đục phá.', 'label': 1},
    {'comment': 'Môn Kỹ năng mềm có phải làm việc nhóm nhiều không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, môn Tin học đại cương thi cuối kỳ có được dùng tài liệu không ạ?', 'label': 0},
    {'comment': '🎉 VOUCHER VINPEARL NHA TRANG GIẢM SỐC 🎉 Trọn gói vé vào cửa + buffet chỉ 999k. Số lượng có hạn. Ib em để giữ chỗ ngay!', 'label': 1},
    {'comment': 'Cần tìm người đi cafe chụp ảnh sống ảo cuối tuần này. Mình chụp cho bạn, bạn chụp cho mình.', 'label': 0},
    {'comment': 'Nhận gõ văn bản, nhập liệu, xử lý số liệu Excel. ✍️ Nhanh, chính xác, giá chỉ từ 2k/trang. Dr mình để trao đổi.', 'label': 1},
    {'comment': 'Đứa nào học cùng lớp thể chất ca 3 chiều nay, thấy cái bình nước Lock&Lock màu xanh của tao ở sân không?', 'label': 0},
    {'comment': 'Có ai biết cách xin miễn giảm học phí cho hộ nghèo không ạ?', 'label': 0},
    {'comment': '🐶 NHẬN TRÔNG THÚ CƯNG DỊP LỄ 🐱 Nhà rộng rãi, có sân chơi, yêu thương động vật. Giá sinh viên. Ba mẹ các bé yên tâm đi du lịch nhé.', 'label': 1},
    {'comment': 'Review CLB Tiếng Anh của trường mình đi mng. Sinh hoạt có hiệu quả không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang phim (Netflix, WeTV, iQiYi). 🎬 Xem phim thả ga không lo quảng cáo. Giá chỉ từ 25k/tháng.', 'label': 1},
    {'comment': 'Mày bị ngáo à? Đọc kỹ đề bài đi rồi hẵng comment.', 'label': 0},
    {'comment': 'Flex nhẹ chiếc vé concert idol vừa săn được.', 'label': 0},
    {'comment': 'BÁN ACC LIÊN QUÂN FULL TƯỚNG, FULL NGỌC, 200 SKIN. 🔥 Rank Tinh Anh, thông tin trắng. Giao dịch trung gian cho ae yên tâm.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán đồ secondhand đẹp mà rẻ không?', 'label': 0},
    {'comment': 'Cần tìm một bạn nữ hát hay để song ca bài cho đêm nhạc sắp tới.', 'label': 0},
    {'comment': 'Nhìn mặt mày hãm tài thật sự.', 'label': 0},
    {'comment': 'Tuyển 5 bạn test game mobile mới ra mắt. 🎮 Chơi game nhận lương. Không yêu cầu kinh nghiệm. Link đăng ký ở bio.', 'label': 1},
    {'comment': 'Mọi người ơi, lịch thi lại đã có chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái máy ép chậm mua về lười dùng. Mới dùng 2-3 lần, còn full box. Mua 1tr2 pass 700k.', 'label': 0},
    {'comment': 'Bạn nào bị RỤNG TÓC, HÓI ĐẦU thì đừng bỏ lỡ siêu phẩm này. 🌿 Tinh chất bưởi cô đặc, kích thích mọc tóc con chỉ sau 2 tuần.', 'label': 1},
    {'comment': 'Có ai đăng ký cuộc thi hùng biện của khoa mình không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua sách chuyên ngành, photo ra cho rẻ.', 'label': 0},
    {'comment': 'Nhận làm gia sư online các môn Toán, Lý, Hóa cấp 2, 3. 👨‍🏫 Kinh nghiệm 2 năm, sv ĐH Sư Phạm. Ib để học thử miễn phí.', 'label': 1},
    {'comment': 'Trời mưa ntn chỉ muốn trốn học ở nhà ngủ.', 'label': 0},
    {'comment': 'Thằng bạn mượn xe mình đi làm xước. Giờ nó im luôn mới cay.', 'label': 0},
    {'comment': 'Dịch vụ book vé xem phim CGV, Lotte. 🍿 Luôn rẻ hơn mua trực tiếp 20-30%. Không cần xếp hàng. Dr để check lịch chiếu.', 'label': 1},
    {'comment': 'Môn Kinh tế lượng có phải dùng phần mềm gì không ạ?', 'label': 0},
    {'comment': 'Nói chuyện mà không có tí muối nào.', 'label': 0},
    {'comment': 'Bán 1000 PRESET LIGHTROOM tone Hàn, Nhật, Trung. 📸 Chỉnh ảnh nghìn like chỉ với 1 click. Trọn bộ chỉ 49k.', 'label': 1},
    {'comment': 'Lịch phát bằng tốt nghiệp đợt này có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách fix lỗi "This site can\'t be reached" không?', 'label': 0},
    {'comment': 'Bạn nào bị MỒ HÔI TRỘM tay chân thì đừng lo. 🖐️ Bột thảo dược nhà mình cam kết trị dứt điểm. Freeship hôm nay.', 'label': 1},
    {'comment': 'Lại sắp đến hạn nộp bài tập lớn rồi. Chưa làm chữ nào.', 'label': 0},
    {'comment': 'Bài đăng như này mà cũng nhiều like được. Khó hiểu.', 'label': 0},
    {'comment': 'GOM ORDER BÚN ĐẬU MẮM TÔM. 🤤 Suất đầy đủ 35k. 12h trưa mình ship, ai ăn hú mình nhé.', 'label': 1},
    {'comment': 'Có ai đi hội sách ở công viên Thống Nhất không?', 'label': 0},
    {'comment': 'Cái loại như mày không đáng để tao nói chuyện.', 'label': 0},
    {'comment': 'Nhận vẽ, thiết kế CAD, 3D MAX, SKETCHUP. 📐 Sinh viên Kiến trúc nhận job thêm. Giá cả thương lượng.', 'label': 1},
    {'comment': 'Ở KTX có cho dùng ấm siêu tốc không mng?', 'label': 0},
    {'comment': 'Tìm đồng đội chơi Ma sói tối nay ở Circle K.', 'label': 0},
    {'comment': 'Bạn muốn cải thiện điểm GPA? 📈 Nhận học hộ, thi hộ, điểm danh hộ tất cả các môn. Uy tín, bảo mật tuyệt đối.', 'label': 1},
    {'comment': 'Thèm trà chanh giã tay quá chúng mày ạ.', 'label': 0},
    {'comment': 'Xem story của crush mà lỡ tay thả tim. Giờ làm sao đây?', 'label': 0},
    {'comment': 'Tuyển nhân viên dọn dẹp theo giờ cho các căn hộ chung cư. 🧹 Lương 40-60k/h, thời gian linh hoạt.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký thi chứng chỉ tin học quốc tế MOS không?', 'label': 0},
    {'comment': 'Crush vừa rep story của mình. Vui cả ngày.', 'label': 0},
    {'comment': 'Chuyên cung cấp các loại hạt dinh dưỡng, ngũ cốc giảm cân. 🥜 Ăn vặt healthy, không lo béo. Ship COD toàn quốc.', 'label': 1},
    {'comment': 'Môn Lịch sử Đảng có được dùng tài liệu không ạ?', 'label': 0},
    {'comment': 'Cmt để xem có bao nhiêu người cùng quan điểm.', 'label': 0},
    {'comment': 'Pass lại áo khoác hoodie size L do mặc không vừa. Mới 99%, giá 150k. Ai lấy dr mình gửi ảnh thật. 🧥', 'label': 1},
    {'comment': 'Phòng trọ đối diện suốt ngày hát karaoke. Muốn trầm cảm.', 'label': 0},
    {'comment': 'Bị bạn thân nói xấu sau lưng. Cảm thấy thất vọng vcl.', 'label': 0},
    {'comment': 'Nhận ship gà rán, pizza, đồ ăn nhanh. 🍕 Freeship bán kính 3km. Alo là có mặt sau 20 phút.', 'label': 1},
    {'comment': 'Học ngành Công nghệ thông tin có cần giỏi Toán không ạ?', 'label': 0},
    {'comment': 'Não mày bị úng nước à?', 'label': 0},
    {'comment': 'Tham gia group share voucher, mã giảm giá Shopee, Lazada, Tiki. 🛍️ Săn sale không lo về giá. Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào có phòng riêng cho nhóm học không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm chó lạc với ạ. Em làm mất 1 bé Poodle màu nâu ở khu vực cổng trường.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Grammarly, Turnitin. ✍️ Check đạo văn, sửa lỗi ngữ pháp. Rất cần cho các bạn làm luận.', 'label': 1},
    {'comment': 'Sắp hết đời sinh viên rồi, nhanh thật.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bánh mì que ngon không?', 'label': 0},
    {'comment': 'Bạn muốn kiếm thêm thu nhập từ việc viết lách? 📝 Tuyển CTV viết content cho page, website. Nhuận bút hấp dẫn.', 'label': 1},
    {'comment': 'Đăng ký học phần xong tài khoản còn 0 đồng.', 'label': 0},
    {'comment': 'Bài viết xàm, thiếu kiến thức.', 'label': 0},
    {'comment': 'Nhận làm nail, gội đầu dưỡng sinh tại nhà. 💅 Thư giãn cuối tuần không cần đi đâu xa. Book lịch ngay nhé các chị em.', 'label': 1},
    {'comment': 'Có ai biết cách bảo quản slide thuyết trình không bị mốc không?', 'label': 0},
    {'comment': 'Mọi người ơi, thủ tục làm thẻ thư viện có lâu không ạ?', 'label': 0},
    {'comment': 'Tuyển nhân viên phục vụ cho quán cafe mới khai trương. Lương 22k/h + thưởng. Ưu tiên các bạn có thể làm Tết.', 'label': 1},
    {'comment': 'Cần tìm người đi tập gym chung cho có động lực 💪. Mình tập ở California Fitness.', 'label': 0},
    {'comment': 'Bán tài khoản Canva Pro, Capcut Pro, Quillbot Premium. Đồng giá 30k/tháng. Ib để được tư vấn thêm.', 'label': 1},
    {'comment': 'Đứa nào học lớp Triết sáng nay có nhặt được cái thẻ ngân hàng tên NGUYEN VAN A không cho mình xin lại với 😭.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán cơm cháy kho quẹt ngon không, cho mình xin địa chỉ với.', 'label': 0},
    {'comment': '✨ Miếng dán trắng răng an toàn từ Mỹ ✨ Răng trắng bật 2-3 tông chỉ sau 1 liệu trình. Đang sale 50%, dr mình ngay nhé.', 'label': 1},
    {'comment': 'Review công ty Shopee vị trí intern marketing đi mng. Môi trường có ổn không ạ?', 'label': 0},
    {'comment': 'Nhận order sỉ lẻ khô gà lá chanh, da heo cháy tỏi. 🌶️ Đồ nhà làm, đảm bảo an toàn vệ sinh. Freeship từ 3 hộp.', 'label': 1},
    {'comment': 'Mày bị ảo tưởng sức mạnh à? Bớt bớt lại đi.', 'label': 0},
    {'comment': 'Đậu môn tư tưởng rồi chúng mày ơi, mừng quá 😂', 'label': 0},
    {'comment': 'Bán voucher nghỉ dưỡng FLC 2N1Đ giá rẻ. Hạn sử dụng còn dài, áp dụng cuối tuần. Ai cần ib mình.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký học vượt không, chỉ mình với.', 'label': 0},
    {'comment': 'Cần tìm một bạn biết chụp ảnh để đi lang thang Hà Nội cuối tuần.', 'label': 0},
    {'comment': 'Nói chuyện mà cứ như bố đời người ta.', 'label': 0},
    {'comment': '💰 Việc làm thêm cho các bạn sinh viên rảnh rỗi. Chỉ cần online chốt đơn, hoa hồng 10-20% trên mỗi đơn hàng. Không cọc, không vốn.', 'label': 1},
    {'comment': 'Ngày hội hiến máu của trường mình tổ chức khi nào vậy mng?', 'label': 0},
    {'comment': 'Góc cảnh báo lừa đảo: Mọi người cẩn thận với tài khoản X chuyên lừa cọc tiền phòng trọ nhé.', 'label': 0},
    {'comment': 'Bên mình có nhận làm dấu thực tập cho các bạn sinh viên. Nhanh chóng, tiện lợi, giá hạt dẻ.', 'label': 1},
    {'comment': 'Có ai tham gia cuộc thi "Sinh viên thanh lịch" của trường không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua tài khoản Coursera Plus để học.', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Facebook, Tiktok ra đơn cho các shop online. Cam kết hiệu quả, giá cạnh tranh.', 'label': 1},
    {'comment': 'Mai thi rồi mà giờ vẫn chưa học chữ nào. Toang thật rồi 🥲.', 'label': 0},
    {'comment': 'Thằng bạn thân vừa bị người yêu cắm sừng. Giờ đang ngồi an ủi nó.', 'label': 0},
    {'comment': 'Pass lại vé xe giường nằm về Nghệ An ngày 28 Tết. Do mình đổi lịch về sớm hơn. Giá gốc.', 'label': 1},
    {'comment': 'Môn Vi điều khiển có khó không các anh chị K65?', 'label': 0},
    {'comment': 'Logic của mày có vấn đề à? Nói trước quên sau.', 'label': 0},
    {'comment': 'Bán tài khoản Zoom Pro không giới hạn thời gian. Phù hợp cho các bạn học nhóm, họp online. Giá 50k/tháng.', 'label': 1},
    {'comment': 'Lịch nghỉ đông của trường có chưa ạ?', 'label': 0},
    {'comment': 'Có ai biết app nào dịch tiếng Anh chuyên ngành tốt không?', 'label': 0},
    {'comment': 'Tuyển CTV viết bài chuẩn SEO cho website. Nhuận bút 50-100k/bài tùy độ khó. Yêu cầu có laptop và chăm chỉ.', 'label': 1},
    {'comment': 'Lại sắp đến hạn đóng tiền nhà rồi, áp lực quá.', 'label': 0},
    {'comment': 'Cái kết phim xàm ghê, phí cả thời gian xem.', 'label': 0},
    {'comment': 'GOM ORDER CHÂN GÀ SỐT THÁI 🌶️ Chân to, giòn sần sật. 50k/hộp 500gr. Chiều mai mình ship nhé.', 'label': 1},
    {'comment': 'Có ai đi xem triển lãm ở trung tâm hội nghị quốc gia không?', 'label': 0},
    {'comment': 'Mày là thánh phán à? Chuyện gì cũng biết.', 'label': 0},
    {'comment': 'Nhận in ốp lưng điện thoại theo yêu cầu. In mọi hình ảnh, logo. Bền đẹp, không phai màu. Chỉ từ 49k.', 'label': 1},
    {'comment': 'Ở cổng sau trường có quán bún chả nào ngon không mng?', 'label': 0},
    {'comment': 'Tìm đồng đội lập team tham gia cuộc thi Robocon.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 200-500k mỗi ngày? Việc nhẹ nhàng, làm tại nhà. Ib mình để được hướng dẫn chi tiết, không lừa đảo.', 'label': 1},
    {'comment': 'Thèm tokbokki phô mai quá chúng mày ơi.', 'label': 0},
    {'comment': 'Hôm nay đi học quên không mang tiền mặt, may mà có đứa bạn cho vay. Quý hóa quá 🙏.', 'label': 0},
    {'comment': 'Pass lại đôi giày Dr. Martens auth do đi không hợp. Size 38, mới 98%, giá 1tr5 có fix.', 'label': 1},
    {'comment': 'Học ngành Marketing có cần ngoại hình không ạ?', 'label': 0},
    {'comment': 'Crush vừa xem story của mình. Tim đập thình thịch luôn 💕.', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản game steam, origin giá rẻ hơn store. Full game bản quyền.', 'label': 1},
    {'comment': 'Môn Tâm lý học có dễ qua không mng?', 'label': 0},
    {'comment': 'Cmt để lưu lại bài viết hay.', 'label': 0},
    {'comment': 'Bán bộ dụng cụ học tập (thước kẻ, compa, eke) kỹ thuật. Mới 100%. Mua 200k pass 120k.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay dẫn người yêu về ngủ lại. Khó xử vcl.', 'label': 0},
    {'comment': 'Bị rớt visa du học. Buồn quá không biết làm gì.', 'label': 0},
    {'comment': 'Nhận làm video highlight bóng đá, video tiktok. 🔥 Chuyên nghiệp, nhanh chóng. Ib để xem sản phẩm.', 'label': 1},
    {'comment': 'Học tín chỉ GDQP ở đâu vậy mng?', 'label': 0},
    {'comment': 'Mày bớt sân si lại đi thì cuộc sống sẽ tốt đẹp hơn đấy.', 'label': 0},
    {'comment': 'Tham gia nhóm đầu tư chứng khoán cho sinh viên. 📈 Được chuyên gia hỗ trợ 1-1. Link tham gia ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào có nhiều mèo không ạ?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm người thân với ạ. Em tìm bạn học cùng cấp 3, mất liên lạc lâu rồi.', 'label': 0},
    {'comment': 'Bán tài khoản Elsa Speak Pro học tiếng Anh trọn đời. Giá chỉ bằng 1/5 giá gốc.', 'label': 1},
    {'comment': 'Sắp ra trường rồi, có chút bồi hồi.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bánh gối ngon không?', 'label': 0},
    {'comment': 'Tuyển mẫu make up cho lớp học trang điểm cá nhân. ✨ Hoàn toàn miễn phí, được chụp ảnh xinh mang về.', 'label': 1},
    {'comment': 'Lại phải học lại rồi, tốn tiền quá 😭.', 'label': 0},
    {'comment': 'Bài viết này phân tích phiến diện, thiếu căn cứ.', 'label': 0},
    {'comment': 'Nhận sửa máy tính, cài win tại nhà. 💻 Nhanh chóng, uy tín, giá sinh viên. Có mặt sau 30 phút.', 'label': 1},
    {'comment': 'Có ai biết cách xin gia hạn học phí không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, môn Cấu trúc dữ liệu và giải thuật có nặng lắm không ạ? Em hơi yếu code.', 'label': 0},
    {'comment': '🔥 Bán bộ 2000+ font chữ việt hóa cho dân thiết kế. Trọn bộ chỉ 39k. Chuyển khoản nhận link google drive ngay.', 'label': 1},
    {'comment': 'Đứa nào ở phòng 402 KTX B5 làm ơn vứt rác đúng chỗ dùm cái. Bừa bộn không thể chịu nổi. 😠', 'label': 0},
    {'comment': 'Nhận làm báo cáo thực tập, chuyên đề tốt nghiệp các ngành kinh tế. Cam kết đúng deadline, chuẩn format, không đạo văn.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào gội đầu massage ổn áp gần trường không ạ?', 'label': 0},
    {'comment': 'Điểm danh cái nhẹ rồi trốn về thôi anh em ơi 😂.', 'label': 0},
    {'comment': 'Tuyển nhân viên bán hàng cho shop giày sneaker. 👟 Lương 25k/h + thưởng doanh số. Yêu cầu nhanh nhẹn, trung thực.', 'label': 1},
    {'comment': 'Review giảng viên hướng dẫn khóa luận khoa Công nghệ thông tin đi mng.', 'label': 0},
    {'comment': 'Mình có nhận order bánh tráng cuốn, bánh tráng trộn. Đầy đủ topping, 25k/suất. Ib mình để đặt nhé.', 'label': 1},
    {'comment': 'Mày không có não hay sao mà comment câu đó?', 'label': 0},
    {'comment': 'Cuối cùng cũng bảo vệ thành công đồ án. Nhẹ cả người 🙏.', 'label': 0},
    {'comment': 'SALE SỐC‼️ Áo phông local brand đồng giá 150k. Mua 2 freeship. Check ngay gian hàng của mình nhé.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký thi chứng chỉ tiếng Nhật JLPT online không?', 'label': 0},
    {'comment': 'Cần tìm một bạn nữ hiền lành, sạch sẽ ở ghép quận Đống Đa. Phòng 3tr/tháng chia đôi.', 'label': 0},
    {'comment': 'Nhìn cái mặt đã thấy không có thiện cảm rồi.', 'label': 0},
    {'comment': 'Bạn đang gặp khó khăn tài chính? 💸 Hỗ trợ vay sinh viên lãi suất thấp, không cần thẩm định. Giải ngân trong 1 nốt nhạc.', 'label': 1},
    {'comment': 'Mọi người ơi, lịch thi đấu giải bóng đá sinh viên có chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái bàn học gấp thông minh còn mới 99%. Mua 300k pass 180k.', 'label': 0},
    {'comment': 'Tinh chất mọc tóc, trị hói gia truyền. Hiệu quả thấy rõ sau 1 tháng. Ib để được tư vấn miễn phí.', 'label': 1},
    {'comment': 'Có ai tham gia CLB nhiếp ảnh của trường không cho em xin ít review với ạ.', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua vé máy bay đi Đà Nẵng tháng tới để săn được vé rẻ.', 'label': 0},
    {'comment': 'Nhận xem chỉ tay, tướng số qua ảnh. ✋ Xem vận mệnh, tình duyên, sự nghiệp. Phí tùy tâm sau khi xem.', 'label': 1},
    {'comment': 'Wifi trên thư viện yếu vcl, load mãi không xong cái video bài giảng.', 'label': 0},
    {'comment': 'Thằng bạn thân có bồ xong bơ mình luôn. Chơi với nó từ hồi cởi truồng tắm mưa... 🥲', 'label': 0},
    {'comment': 'Pass lại 2 vé xem phim "Mai" suất chiếu tối nay. Do mình có việc bận. Giá rẻ hơn ở rạp.', 'label': 1},
    {'comment': 'Môn Triết học có được mang Atlat vào phòng thi không ạ? 🤔', 'label': 0},
    {'comment': 'Nói chuyện mà cứ cà khịa là giỏi.', 'label': 0},
    {'comment': 'Bán tài khoản Youtube Premium, xem video không quảng cáo, nghe nhạc tắt màn hình. 40k/tháng.', 'label': 1},
    {'comment': 'Lịch học quân sự đợt này ở đâu vậy mng?', 'label': 0},
    {'comment': 'Có ai biết cách fix lỗi máy tính bị màn hình xanh không?', 'label': 0},
    {'comment': 'Tuyển 15 bạn làm cộng tác viên cho các dự án NFT. 🚀 Không cần vốn, chỉ cần chăm chỉ học hỏi. Cơ hội X100 tài sản.', 'label': 1},
    {'comment': 'Lại sắp đến hạn nộp học phí rồi, chưa dám xin tiền bố mẹ.', 'label': 0},
    {'comment': 'Cái giọng của bà này nghe khó chịu thật sự.', 'label': 0},
    {'comment': 'GOM ORDER SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN 흑당 버블티. 🥤 Chuẩn vị Đài Loan, 30k/cốc. 3h chiều mình ship nhé.', 'label': 1},
    {'comment': 'Có ai đi xem kịch ở Nhà hát Lớn bao giờ chưa?', 'label': 0},
    {'comment': 'Mày bớt thể hiện đi được không?', 'label': 0},
    {'comment': 'Nhận rửa xe, thay dầu, bảo dưỡng xe máy tại nhà. 🛵 Tiện lợi, uy tín, giá cả phải chăng.', 'label': 1},
    {'comment': 'Ở KTX có chỗ nào cho nấu ăn riêng không ạ?', 'label': 0},
    {'comment': 'Tìm đồng đội tối nay đi ăn phở cuốn Ngũ Xã.', 'label': 0},
    {'comment': 'Bạn muốn nói tiếng Trung như gió? Đăng ký ngay khóa học giao tiếp với người bản xứ. Giảm 50% cho SV.', 'label': 1},
    {'comment': 'Thèm nem lụi Huế quá chúng mày ơi.', 'label': 0},
    {'comment': 'Đi học về thấy mẹ gửi đồ ăn lên. Hạnh phúc giản đơn. ❤️', 'label': 0},
    {'comment': 'Pass lại cuốn sách "Nhà giả kim" còn mới tinh. Mua 80k pass 50k.', 'label': 1},
    {'comment': 'Học ngành Kế toán có cần giỏi excel không ạ?', 'label': 0},
    {'comment': 'Crush vừa trả lời tin nhắn của mình. Chắc đêm nay mất ngủ. 🥰', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản game bản quyền trên Steam với giá rẻ hơn 70%. Ib để check game bạn cần.', 'label': 1},
    {'comment': 'Môn Quản trị học có dễ lấy điểm A không mng?', 'label': 0},
    {'comment': 'Hóng link full không che.', 'label': 0},
    {'comment': 'Bán balo chống trộm, chống nước, có cổng sạc USB. ✨ Siêu tiện lợi cho các bạn hay di chuyển. Giá chỉ 299k.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay nghe nhạc bằng loa ngoài. Nói mấy lần không được.', 'label': 0},
    {'comment': 'Chia tay rồi, làm sao để quên người yêu cũ đây mọi người?', 'label': 0},
    {'comment': 'Nhận làm tour du lịch, teambuilding cho các lớp, CLB. 🌴 Lịch trình hấp dẫn, giá cả sinh viên.', 'label': 1},
    {'comment': 'Chỗ gửi xe của trường có nhận trông qua đêm không ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ ra mình là nạn nhân đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ tài liệu học tập, slide bài giảng free. Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe sách nào yên tĩnh ở khu vực Ba Đình không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài pass lại mấy bộ đồ mặc không vừa với ạ.', 'label': 0},
    {'comment': 'Bán tài khoản Duolingo Plus học ngoại ngữ không giới hạn. Trọn đời chỉ 150k.', 'label': 1},
    {'comment': 'Sắp 4 năm đại học rồi mà vẫn chưa biết mình thích gì.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bún đậu mắm tôm ngon mà nhiều topping không?', 'label': 0},
    {'comment': 'Tuyển nhân viên thu ngân cho siêu thị mini. Có thể xoay ca linh hoạt. Lương 20k/h.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải học online rồi, chán thật.', 'label': 0},
    {'comment': 'Bài viết này như kiểu google dịch ấy nhỉ?', 'label': 0},
    {'comment': 'Nhận khắc laser lên bút, ví da, móc khóa làm quà tặng. 🎁 Lấy ngay sau 5 phút. Giá chỉ từ 30k.', 'label': 1},
    {'comment': 'Có ai biết cách xin bảng điểm online không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, phòng y tế của trường mình có khám BHYT không ạ?', 'label': 0},
    {'comment': '🔥 XẢ KHO GIÀY DÉP MÙA HÈ - ĐỒNG GIÁ 79K 🔥 Sandal, dép lê, sục đủ mẫu. Nhanh tay vào link bio săn sale nhé!', 'label': 1},
    {'comment': 'Cần tìm 2 bạn nam ở ghép khu vực Mỹ Đình. Phòng 2tr8/tháng, có nóng lạnh điều hòa, không chung chủ. Yêu cầu gọn gàng.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 1-2 triệu/tháng chỉ bằng việc lướt Tiktok? 📱 Ib mình để nhận link app, không đa cấp, không lừa đảo.', 'label': 1},
    {'comment': 'Đứa nào học lớp Marketing căn bản chiều nay có thấy cái tai nghe Airpods pro ở bàn 3 không? Cho mình xin lại với 😭.', 'label': 0},
    {'comment': 'Có ai biết cách đăng ký tình nguyện viên cho Sea Games không ạ?', 'label': 0},
    {'comment': 'Nhận order trà mãng cầu, trà dâu hot hit. 🍓 Quả tươi mỗi ngày, đảm bảo ngon. 35k/ly, ship từ 2 ly.', 'label': 1},
    {'comment': 'Review trung tâm tiếng Anh Ila đi mng. Học phí có đắt không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản ChatGPT 4.0 Plus giá rẻ. Hỗ trợ làm bài tập, viết code, content. 🤖 Dr mình để được tư vấn.', 'label': 1},
    {'comment': 'Mày đọc có hiểu tiếng người không vậy?', 'label': 0},
    {'comment': 'Bảo vệ thành công khóa luận tốt nghiệp rồi. Cảm ơn thầy cô và bạn bè rất nhiều ❤️.', 'label': 0},
    {'comment': 'Tuyển nhân viên pha chế, thu ngân cho quán trà sữa. Lương 23k/h, có thể xoay ca theo lịch học. 🥤', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy xác nhận sinh viên để giảm giá vé tàu không?', 'label': 0},
    {'comment': 'Cần tìm một bạn biết chơi piano để đệm cho mình hát trong đêm nhạc hội.', 'label': 0},
    {'comment': 'Nhìn cái cách nói chuyện đã thấy không ưa nổi rồi.', 'label': 0},
    {'comment': 'Dịch vụ tăng follow, like, comment cho Instagram, Facebook, Tiktok. 🚀 Giúp bạn trở thành hot face. Uy tín, giá rẻ.', 'label': 1},
    {'comment': 'Mọi người ơi, website của phòng đào tạo lại không vào được rồi à?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái máy ảnh film còn dùng tốt. Tặng kèm 1 cuộn film. Giá 500k.', 'label': 0},
    {'comment': 'Bạn nào bị thâm mắt, bọng mắt, nếp nhăn thì về đội em. Kem mắt AHC cam kết hiệu quả sau 14 ngày. ✨', 'label': 1},
    {'comment': 'Có ai tham gia CLB Sách và Hành động không cho em xin ít review với.', 'label': 0},
    {'comment': 'Cần tìm người share tiền phòng khách sạn đi Vũng Tàu cuối tháng này.', 'label': 0},
    {'comment': 'Nhận làm powerpoint, slide thuyết trình. 💻 Đẹp, chuyên nghiệp, sáng tạo. Giá chỉ từ 8k/slide.', 'label': 1},
    {'comment': 'Trời nồm ẩm quần áo phơi mãi không khô. Khó chịu vcl. 😠', 'label': 0},
    {'comment': 'Thằng bạn mượn tiền mình từ đầu kỳ mà giờ cuối kỳ rồi chưa thấy trả.', 'label': 0},
    {'comment': 'Pass lại vé concert BLACKPINK tại Thái Lan. Do mình không xin được visa. Giá gốc + phí ship.', 'label': 1},
    {'comment': 'Môn Hệ quản trị cơ sở dữ liệu có cần laptop cấu hình cao không ạ?', 'label': 0},
    {'comment': 'Mày bớt dạy đời người khác đi.', 'label': 0},
    {'comment': 'Bán 10.000+ khóa học online đủ mọi lĩnh vực. 📚 Marketing, IT, Design... Trọn bộ chỉ 199k. Ib để nhận list khóa học.', 'label': 1},
    {'comment': 'Lịch thi kết thúc học phần có trên trang tín chỉ chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách dùng hàm Vlookup trong excel không, chỉ mình với huhu.', 'label': 0},
    {'comment': 'Bạn muốn trở thành affiliate marketer? 🔥 Tuyển CTV bán các khóa học online, hoa hồng lên đến 60%. Không cần vốn.', 'label': 1},
    {'comment': 'Lại sắp đến hạn nộp báo cáo thực tập rồi, vẫn chưa xin được chữ ký.', 'label': 0},
    {'comment': 'Cái giọng lồng tiếng phim này nghe chán thật sự.', 'label': 0},
    {'comment': 'GOM ORDER BÁNH MÌ CHẢO. 🍳 Suất đầy đủ 30k (pate, trứng, xúc xích). 7h sáng mai mình ship.', 'label': 1},
    {'comment': 'Có ai đi xem giải Vovinam của trường mình không?', 'label': 0},
    {'comment': 'Mày tưởng mày là ai mà có quyền phán xét người khác?', 'label': 0},
    {'comment': 'Nhận sửa chữa, vệ sinh máy lạnh, quạt hơi nước tại nhà. ❄️ Nhanh chóng, sạch sẽ, giá sinh viên.', 'label': 1},
    {'comment': 'Ở KTX có được phép dùng bếp từ mini không ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi lượn phố đi bộ không?', 'label': 0},
    {'comment': 'Bạn muốn cải thiện kỹ năng giao tiếp? Đăng ký ngay khóa học MC, kỹ năng mềm. Giảm 40% cho nhóm 3 người.', 'label': 1},
    {'comment': 'Thèm bánh tráng nướng Đà Lạt quá chúng mày ơi. 😭', 'label': 0},
    {'comment': 'Đi học về mệt mà thấy bạn cùng phòng nấu sẵn cơm cho. Cảm động thực sự. 🥰', 'label': 0},
    {'comment': 'Pass lại con máy đọc sách Kindle còn mới. Mua về mà lười đọc. Giá 1tr8.', 'label': 1},
    {'comment': 'Học ngành Ngôn ngữ Nhật ra trường có cần bằng N mấy ạ?', 'label': 0},
    {'comment': 'Crush vừa follow instagram của mình. Có nên chủ động nhắn tin không nhỉ? 🤔', 'label': 0},
    {'comment': 'Chuyên cung cấp key Windows, Office, phần mềm diệt virus bản quyền. Rẻ hơn mua trực tiếp 80%.', 'label': 1},
    {'comment': 'Môn Logic học có khó không mng?', 'label': 0},
    {'comment': 'Bài viết này chất lượng quá, cảm ơn ad.', 'label': 0},
    {'comment': 'Bán tài khoản VIP FPT Play xem bóng đá C1. ⚽️ Rẻ hơn đăng ký trực tiếp. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay lấy đồ của mình dùng mà không hỏi. Khó chịu nhưng ngại nói.', 'label': 0},
    {'comment': 'Bị từ chối học bổng du học. Cảm thấy cả thế giới sụp đổ.', 'label': 0},
    {'comment': 'Nhận làm phụ đề (subtitle) cho video, phim ảnh. 🎞️ Dịch chuẩn, khớp time. Giá cả thương lượng.', 'label': 1},
    {'comment': 'Chỗ photo ở gần cổng trường có in màu không ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ vẻ hiểu biết đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ kinh nghiệm apply học bổng, du học. Link ở bio nhé. ✈️', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào có board game ở khu vực Cầu Giấy không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm mèo lạc với ạ. Em làm mất 1 bé mèo ta màu vàng ở KTX.', 'label': 0},
    {'comment': 'Bán tài khoản Grammarly Premium check ngữ pháp, đạo văn. Siêu cần thiết cho các bạn học tiếng Anh. ✍️', 'label': 1},
    {'comment': 'Sắp hết 4 năm mà vẫn chưa có người yêu.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bún bò Huế chuẩn vị không?', 'label': 0},
    {'comment': 'Tuyển cộng tác viên review sản phẩm, mỹ phẩm. 💄 Được dùng sản phẩm miễn phí + nhận hoa hồng.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải đóng tiền học lại. Buồn của tôi. 🥲', 'label': 0},
    {'comment': 'Bài viết này phân tích sơ sài quá, chưa tới.', 'label': 0},
    {'comment': 'Nhận bọc sách vở, dán bìa plastic cho học sinh, sinh viên. 📚 Cẩn thận, đẹp, lấy ngay.', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy chứng nhận tham gia hoạt động ngoại khóa không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, môn học nào tự chọn của khối C là dễ qua nhất ạ? Em cần kéo GPA.', 'label': 0},
    {'comment': '🔥 VÉ MÁY BAY 0Đ SĂN SALE NỬA ĐÊM 🔥 Book vé du lịch hè ngay thôi. Mình có nhận săn vé giá rẻ, phí chỉ 50k/vé.', 'label': 1},
    {'comment': 'Tìm 1 bạn nam ở ghép khu vực Thanh Xuân. Phòng 2tr4, điện nước giá dân, giờ giấc tự do. Yêu cầu không hút thuốc, sạch sẽ.', 'label': 0},
    {'comment': 'Chỉ cần online 1-2h mỗi ngày, kiếm 3-4 triệu/tháng. 💸 Công việc đăng bài, tư vấn sản phẩm. Không vốn, không cọc. Ib mình để được hướng dẫn.', 'label': 1},
    {'comment': 'Đứa nào để cái xe đạp không khóa ở nhà xe G. Cẩn thận mất đấy.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán đồ ăn vặt Hàn Quốc chuẩn vị không ạ?', 'label': 0},
    {'comment': 'Son Black Rouge, 3CE đồng giá 129k. 💄 Hàng auth check code thoải mái. Mua 2 thỏi freeship. Dr mình để xem bảng màu nhé.', 'label': 1},
    {'comment': 'Review CLB Kỹ năng mềm của trường mình đi ạ. Em là tân sinh viên muốn tham gia.', 'label': 0},
    {'comment': 'Bán tài khoản VIP VieON, K+ xem phim, bóng đá. ⚽️ Chất lượng full HD, không giật lag. Giá chỉ từ 30k/tháng.', 'label': 1},
    {'comment': 'Mày học lại môn văn đi rồi hẵng comment nhé.', 'label': 0},
    {'comment': 'Trượt môn chuyên ngành. Cảm giác thật tồi tệ. 😞', 'label': 0},
    {'comment': 'Tuyển nhân viên soát vé cho rạp chiếu phim. Lương 25k/h, được xem phim miễn phí. Yêu cầu trung thực, nhanh nhẹn.', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy chứng nhận là sinh viên để mua vé xe bus tháng không ạ?', 'label': 0},
    {'comment': 'Cần tìm một bạn biết edit video bằng Premiere để làm bài tập nhóm.', 'label': 0},
    {'comment': 'Nhìn cái avatar đã thấy không muốn tranh luận rồi.', 'label': 0},
    {'comment': 'Dịch vụ seeding, tăng tương tác cho bài viết, fanpage. 🚀 Giúp bài viết của bạn lên top. Giá cả phải chăng, ib để được báo giá.', 'label': 1},
    {'comment': 'Mọi người ơi, điểm thi giữa kỳ có cập nhật trên app của trường chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái giá sách 5 tầng còn mới. Mua 400k pass 250k. Mình ở gần trường.', 'label': 0},
    {'comment': 'Bạn nào bị viêm da cơ địa, á sừng, vảy nến thì đừng bỏ qua bài thuốc nam gia truyền này nhé. Cam kết hiệu quả.', 'label': 1},
    {'comment': 'Có ai tham gia cuộc thi "Olympic Kinh tế học" của trường không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua account Figma Pro cho nhóm làm UI/UX.', 'label': 0},
    {'comment': 'Nhận thiết kế CV, portfolio, profile cá nhân. ✨ Chuyên nghiệp, ấn tượng, giúp bạn ghi điểm với nhà tuyển dụng.', 'label': 1},
    {'comment': 'Lại sắp đến deadline đóng tiền điện nước rồi. Ví lại mỏng đi. 🥲', 'label': 0},
    {'comment': 'Bạn thân nhất của mình sắp đi du học. Cảm thấy buồn và hụt hẫng.', 'label': 0},
    {'comment': 'Pass lại voucher ăn buffet Dookki. Hạn sử dụng còn 1 tháng. Mua 139k pass 100k.', 'label': 1},
    {'comment': 'Môn Kỹ thuật phần mềm có nặng về lý thuyết không ạ?', 'label': 0},
    {'comment': 'Mày bớt sân si chuyện của người khác đi được không?', 'label': 0},
    {'comment': 'Bán 1000+ template Powerpoint, Keynote, Google Slides. 💻 Đầy đủ các chủ đề. Trọn bộ chỉ 99k.', 'label': 1},
    {'comment': 'Lịch tốt nghiệp đợt 2 có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách khôi phục file word chưa kịp lưu mà máy bị sập nguồn không? Cứu em với. 😭', 'label': 0},
    {'comment': 'Tuyển cộng tác viên dịch truyện, phim (Anh - Việt). ✍️ Yêu cầu tiếng Anh tốt, chăm chỉ. Nhuận bút tính theo sản phẩm.', 'label': 1},
    {'comment': 'Lại một kỳ nữa qua đi mà vẫn chưa có mảnh tình vắt vai. FA muôn năm.', 'label': 0},
    {'comment': 'Cái kết của bộ phim này vô lý thật sự.', 'label': 0},
    {'comment': 'GOM ORDER CƠM CHÁY CHIÊN MẮM HÀNH. 🍚 Giòn rụm, đậm vị. 25k/túi 200gr. Chiều mai mình ship nhé.', 'label': 1},
    {'comment': 'Có ai đi xem show của Hà Anh Tuấn tối nay không?', 'label': 0},
    {'comment': 'Mày tưởng mày là ai mà lên mặt dạy bảo tao?', 'label': 0},
    {'comment': 'Nhận lắp đặt, sửa chữa wifi, mạng LAN tại nhà. 🌐 Nhanh chóng, chuyên nghiệp, giá sinh viên.', 'label': 1},
    {'comment': 'Ở KTX có được mang nồi lẩu mini vào dùng không ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi dạo hồ Gươm không?', 'label': 0},
    {'comment': 'Bạn muốn tự tin thuyết trình trước đám đông? Đăng ký ngay khóa học kỹ năng mềm. Giảm 30% khi đăng ký trước ngày X.', 'label': 1},
    {'comment': 'Thèm chè khúc bạch quá chúng mày ơi.', 'label': 0},
    {'comment': 'Hôm nay lên thư viện gặp crush. Cả ngày vui vẻ. 😊', 'label': 0},
    {'comment': 'Pass lại con tai nghe Sony WH-1000XM4 full box, còn bảo hành. Ít dùng nên còn mới. Giá 4tr5.', 'label': 1},
    {'comment': 'Học ngành Quan hệ công chúng có cần phải năng động lắm không ạ?', 'label': 0},
    {'comment': 'Crush vừa rep story mình bằng sticker. Có ý gì không nhỉ? 🤔', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản game pass, PC game pass giá rẻ. Chơi hàng trăm game bản quyền chỉ với 50k/tháng.', 'label': 1},
    {'comment': 'Môn Kinh tế chính trị Mác Lênin có dễ học không mng?', 'label': 0},
    {'comment': 'Bài này phân tích sâu sắc quá.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Elsa Speak, Memrise, Busuu học ngoại ngữ. 🌍 Giá sinh viên. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay đi chơi về khuya làm mình mất ngủ.', 'label': 0},
    {'comment': 'Thi trượt bằng lái xe máy lần thứ 2. Chán không buồn nói.', 'label': 0},
    {'comment': 'Nhận viết content, quản lý fanpage cho các shop. ✍️ Nội dung sáng tạo, thu hút khách hàng. Giá cả thương lượng.', 'label': 1},
    {'comment': 'Chỗ giữ xe ở nhà D có làm việc cuối tuần không ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ ra mình là người bị hại đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ kinh nghiệm săn học bổng chính phủ. Toàn các pro thôi. Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào có view sân thượng đẹp ở khu vực Hà Đông không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm người ở ghép với ạ. Em là nữ, sv năm 2.', 'label': 0},
    {'comment': 'Bán tài khoản Turnitin check đạo văn. 📝 Cần thiết cho các bạn làm bài tập lớn, khóa luận. 20k/lượt check.', 'label': 1},
    {'comment': 'Sắp 20/10 rồi, nên tặng quà gì cho mẹ và cô giáo đây nhỉ?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bún riêu cua ngon ở gần đây không?', 'label': 0},
    {'comment': 'Tuyển cộng tác viên bán mỹ phẩm xách tay. 💄 Chiết khấu cao, không cần bỏ vốn. Được training về sản phẩm.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải học online. Tương tác với thầy cô bạn bè chán ghê. 😩', 'label': 0},
    {'comment': 'Bài viết này dịch word-by-word, đọc ngang phè.', 'label': 0},
    {'comment': 'Nhận order quần áo Taobao, 1688, Pinduoduo. 👕 Công rẻ, hàng về nhanh. Chốt đơn mỗi ngày.', 'label': 1},
    {'comment': 'Có ai biết cách xin cấp lại mật khẩu tài khoản sinh viên không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, lên phòng công tác sinh viên xin dấu có cần mang theo thẻ sinh viên không ạ?', 'label': 0},
    {'comment': '⚽️ NHẬN IN ÁO LỚP, ÁO NHÓM, ÁO CLB ⚽️ Vải cotton co giãn, in sắc nét không bong tróc. Thiết kế miễn phí. Ib để nhận báo giá.', 'label': 1},
    {'comment': 'Cần tìm người đi ăn sập phố Tạ Hiện tối nay. Mình bao kèo đầu. 😂', 'label': 0},
    {'comment': 'Bạn đang ĐAU ĐẦU vì deadline? 🤯 Nhận làm assignment, research paper, thesis... mọi chuyên ngành. Uy tín, bảo mật, check turnitin <10%.', 'label': 1},
    {'comment': 'Đứa nào ở ktx mà đêm hôm còn bấm game kêu lạch cạch vậy. Vô ý thức vcl. 😠', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán xôi chiên ngon ở gần đây không?', 'label': 0},
    {'comment': 'Bán tài khoản VIP TestKing, Acloud Guru cho dân IT. 👨‍💻 Full khóa học, lab thực hành. Giá chỉ bằng 1/10 giá gốc.', 'label': 1},
    {'comment': 'Review CLB Guitar của trường mình đi ạ. Có cần biết chơi đàn trước khi vào không?', 'label': 0},
    {'comment': 'Nhận order bánh crépe sầu riêng, bánh mousse. 🍰 Bánh tươi mỗi ngày. Ship hỏa tốc nội thành.', 'label': 1},
    {'comment': 'Nhìn cái mặt đã thấy gian rồi.', 'label': 0},
    {'comment': 'Cuối cùng cũng qua môn xác suất thống kê. Hú hồn. 🙏', 'label': 0},
    {'comment': 'Tuyển cộng tác viên review, unbox sản phẩm công nghệ. 📱 Được trải nghiệm sản phẩm mới miễn phí + có nhuận bút.', 'label': 1},
    {'comment': 'Có ai biết cách xin học bổng khuyến khích học tập của khoa không?', 'label': 0},
    {'comment': 'Cần tìm một bạn nữ làm mẫu ảnh cho BST áo dài Tết. Yêu cầu cao trên 1m6, tóc dài.', 'label': 0},
    {'comment': 'Nói chuyện mà cứ như mình biết tuốt ấy nhỉ.', 'label': 0},
    {'comment': 'Dịch vụ phá mật khẩu điện thoại, máy tính. 🔐 Lấy lại dữ liệu an toàn, nhanh chóng. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Mọi người ơi, lịch nghỉ giữa kỳ đã có chính thức chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái máy lọc không khí mini để bàn. Mua về mà phòng chật quá. Mới 99%, giá 300k.', 'label': 0},
    {'comment': 'Bạn nào bị nhiệt miệng, lở miệng kinh niên thì ib mình nhé. ✨ Thuốc cam gia truyền bôi vào là khỏi ngay.', 'label': 1},
    {'comment': 'Có ai tham gia cuộc thi "Ý tưởng sáng tạo sinh viên" không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua tên miền và hosting cho đồ án web.', 'label': 0},
    {'comment': 'Nhận thiết kế, in ấn name card, poster, standee. 🎨 Chuyên nghiệp, giá rẻ. Có ship tận nơi.', 'label': 1},
    {'comment': 'Sắp phải thuyết trình môn đầu tiên. Run quá chúng mày ơi. 😥', 'label': 0},
    {'comment': 'Bạn thân nhất của mình có người yêu. Cảm giác như bị ra rìa vậy.', 'label': 0},
    {'comment': 'Pass lại voucher buffet King BBQ. Hạn sử dụng còn 2 tháng. Mua 250k pass 180k.', 'label': 1},
    {'comment': 'Môn Trí tuệ nhân tạo có phải code nhiều không ạ?', 'label': 0},
    {'comment': 'Mày bớt phán xét người khác khi chưa biết gì về họ đi.', 'label': 0},
    {'comment': 'Bán 500+ template CV xin việc chuyên nghiệp. 📄 File word, dễ dàng chỉnh sửa. Trọn bộ chỉ 49k.', 'label': 1},
    {'comment': 'Lịch thi chứng chỉ ngoại ngữ đợt này có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách chèn video vào powerpoint mà không bị lỗi không ạ?', 'label': 0},
    {'comment': 'Tuyển 20 bạn điền form khảo sát online. ✍️ Mỗi form 5 phút, nhận 10k. Tích tiểu thành đại. Ib để nhận link.', 'label': 1},
    {'comment': 'Lại một kỳ nữa sắp hết mà vẫn chưa có cái bằng lái xe.', 'label': 0},
    {'comment': 'Nhân vật này xây dựng tính cách khó chịu thật sự.', 'label': 0},
    {'comment': 'GOM ORDER XÔI KHÚC NÓNG HỔI. 🍙 15k/cái. 7h tối mình trả đơn ở sảnh KTX nhé.', 'label': 1},
    {'comment': 'Có ai đi xem triển lãm tranh ở bảo tàng mỹ thuật không?', 'label': 0},
    {'comment': 'Mày nghĩ mày là ai mà nói chuyện giọng đó với tao?', 'label': 0},
    {'comment': 'Nhận vệ sinh, tra keo tản nhiệt cho laptop, PC. 💻 Giúp máy mát hơn, chạy nhanh hơn. Giá chỉ từ 80k.', 'label': 1},
    {'comment': 'Ở KTX có được treo tranh ảnh lên tường không ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi xem phim suất chiếu sớm không?', 'label': 0},
    {'comment': 'Bạn muốn có một body săn chắc? 💪 Tuyển thành viên cho CLB thể hình. Cùng nhau tập luyện, chia sẻ kinh nghiệm.', 'label': 0},
    {'comment': 'Thèm bánh ép Huế quá chúng mày ơi. 😭', 'label': 0},
    {'comment': 'Thi giữa kỳ được điểm cao. Phải khao cả phòng một chầu trà sữa mới được. 🎉', 'label': 0},
    {'comment': 'Pass lại con loa bluetooth JBL Go 3. Nghe ít, pin còn trâu. Giá 600k.', 'label': 1},
    {'comment': 'Học ngành Truyền thông đa phương tiện có cần máy tính cấu hình mạnh không?', 'label': 0},
    {'comment': 'Crush vừa xem hết story của mình. Có nên tự tin lên không nhỉ? 🤔', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản Notion Pro, Evernote Premium. 📝 Giúp bạn quản lý công việc, học tập hiệu quả. Giá sinh viên.', 'label': 1},
    {'comment': 'Môn Xã hội học đại cương có phải học thuộc nhiều không mng?', 'label': 0},
    {'comment': 'Bài này phân tích hay, bookmark lại để đọc kỹ.', 'label': 0},
    {'comment': 'Bán tài khoản VIP VOV, Spotify, Apple Music. 🎧 Nghe nhạc chất lượng cao không quảng cáo. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay mở nhạc to lúc mình đang học bài. Bực mình ghê.', 'label': 0},
    {'comment': 'Apply intern mấy chỗ đều bị từ chối. Cảm thấy bất lực.', 'label': 0},
    {'comment': 'Nhận làm MC, hoạt náo viên cho các sự kiện, party. 🎤 Khuấy động không khí, cháy hết mình. Ib để trao đổi.', 'label': 1},
    {'comment': 'Chỗ nào photo gần trường có đóng bìa mạ vàng không ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ ra mình là chuyên gia đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ tip học Ielts 8.0+. ✍️ Toàn tài liệu xịn. Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán bún cá nào ngon ở khu vực Hoàng Mai không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm bạn cùng đi học tiếng Hàn với ạ.', 'label': 0},
    {'comment': 'Bán tài khoản Midjourney, Stable Diffusion cho dân design. 🎨 Tạo ảnh AI không giới hạn. Giá rẻ hơn đăng ký gốc.', 'label': 1},
    {'comment': 'Sắp 20/11 rồi, nên tặng quà gì cho thầy cô đây mng?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán cao sao vàng không? Tự nhiên thèm mùi này.', 'label': 0},
    {'comment': 'Tuyển cộng tác viên bán sim số đẹp. 📱 Hoa hồng cao, không cần bỏ vốn. Được đào tạo kỹ năng bán hàng.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải học lại môn thể dục. Buồn của Linh. 🥲', 'label': 0},
    {'comment': 'Bài viết này nông cạn, nhìn từ một phía.', 'label': 0},
    {'comment': 'Nhận thêu tay tên, logo lên áo, khăn, mũ. 🧵 Tỉ mỉ, cẩn thận. Món quà ý nghĩa cho người thân.', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy tạm vắng ở công an phường không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, thư viện trường mình có cho mượn sách chuyên ngành về mang về không ạ?', 'label': 0},
    {'comment': '🔥 ÁO KHOÁC DÙ 2 LỚP CHỐNG NƯỚC, CHỐNG NẮNG 🔥 Sale 50% chỉ còn 149k. Đủ size nam nữ. Ib shop để được tư vấn nhé.', 'label': 1},
    {'comment': 'Tìm 1 bạn nữ ở ghép khu vực Cầu Giấy. Phòng 3tr/tháng, đầy đủ đồ, gần điểm bus. Yêu cầu là sinh viên, sạch sẽ, hiền lành.', 'label': 0},
    {'comment': 'Nhận thiết kế CV, Cover Letter song ngữ Anh-Việt. ✨ Gây ấn tượng mạnh với nhà tuyển dụng. File PDF + Word chỉ 80k.', 'label': 1},
    {'comment': 'Đứa nào đi học lớp Kỹ năng giao tiếp sáng nay có nhặt được cái sạc dự phòng màu trắng không cho mình xin lại với. 😭', 'label': 0},
    {'comment': 'Có ai biết cách đăng ký tham gia CLB hùng biện không ạ?', 'label': 0},
    {'comment': 'Tuyển CTV bán nước hoa chiết chính hãng. 💯 Không cần vốn, không ôm hàng. Chiết khấu cao. Được training kiến thức mùi hương.', 'label': 1},
    {'comment': 'Review trung tâm tin học MOS Master đi mọi người. Dạy có ổn không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản VIP Skillshare, Coursera, Udemy. 📚 Học mọi kỹ năng chỉ với 50k/tháng. Ib mình để chốt đơn.', 'label': 1},
    {'comment': 'Mày bị thiếu i-ốt à mà phát biểu ngu thế?', 'label': 0},
    {'comment': 'Cuối cùng cũng lấy được cái bằng lái xe máy. Mừng rơi nước mắt. 😂', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page, chốt đơn hàng thời trang. Làm việc online tại nhà, lương 4-6tr/tháng + thưởng.', 'label': 1},
    {'comment': 'Có ai biết cách xin giấy chứng nhận hộ nghèo để miễn giảm học phí không?', 'label': 0},
    {'comment': 'Cần tìm một bạn biết quay phim bằng điện thoại để thực hiện một dự án video ngắn.', 'label': 0},
    {'comment': 'Nhìn cái cách trả lời cmt đã thấy tính cách có vấn đề rồi.', 'label': 0},
    {'comment': 'Dịch vụ get link Fshare, 4share max tốc độ. 🚀 Chỉ 5k/link. Nhanh chóng, tiện lợi. Dr link cho mình nhé.', 'label': 1},
    {'comment': 'Mọi người ơi, điểm tổng kết trên app đã chính xác chưa hay còn cập nhật nữa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái đèn học kẹp bàn còn dùng tốt. Mua 150k pass 70k.', 'label': 0},
    {'comment': 'Bạn nào bị các vấn đề về xương khớp, đau lưng mỏi gối thì ib mình nhé. Cao dán thảo dược gia truyền hiệu quả lắm.', 'label': 1},
    {'comment': 'Có ai tham gia cuộc thi "Thiết kế poster" của đoàn trường không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua gói Adobe full app dùng chung.', 'label': 0},
    {'comment': 'Nhận trang trí, setup tiệc sinh nhật, sự kiện. 🎈 Concept đa dạng, giá cả sinh viên. Ib để xem các set decor.', 'label': 1},
    {'comment': 'Sắp thi vấn đáp rồi mà chưa ôn được chữ nào. Lo quá. 😥', 'label': 0},
    {'comment': 'Bạn thân đi lấy chồng. Vừa mừng vừa có chút buồn. 🥲', 'label': 0},
    {'comment': 'Pass lại vé concert Maroon 5 tại Singapore. Do mình không sắp xếp được công việc. Giá gốc, có bill.', 'label': 1},
    {'comment': 'Môn An toàn thông tin có cần học tốt môn mạng trước không ạ?', 'label': 0},
    {'comment': 'Mày bớt dùng cái giọng bề trên đó đi.', 'label': 0},
    {'comment': 'Bán 2000+ Mockup cho dân thiết kế (áo, cốc, túi...). 👕 File PSD dễ dàng thay đổi. Trọn bộ chỉ 99k.', 'label': 1},
    {'comment': 'Lịch nhận đồ án tốt nghiệp đợt này có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách gộp nhiều file pdf thành một file không ạ? Chỉ em với.', 'label': 0},
    {'comment': 'Tuyển 30 bạn xem video kiếm tiền. 📱 Mỗi video 30s nhận 5k. Rút tiền về momo trong ngày. Link tải app ở bio.', 'label': 1},
    {'comment': 'Lại một kỳ nữa trôi qua mà GPA vẫn lẹt đẹt.', 'label': 0},
    {'comment': 'Cái plot twist này đỉnh thật sự. Xem mà há hốc mồm.', 'label': 0},
    {'comment': 'GOM ORDER BÁNH GIÒ NÓNG. 🥟 10k/cái. 4h chiều mình trả đơn ở sảnh KTX nhé.', 'label': 1},
    {'comment': 'Có ai đi xem Lễ hội pháo hoa Đà Nẵng năm nay không?', 'label': 0},
    {'comment': 'Mày nghĩ mày là rốn của vũ trụ à?', 'label': 0},
    {'comment': 'Nhận unlock, bẻ khóa, cài tiếng Việt cho điện thoại xách tay. 📱 Lấy ngay, giá rẻ. Hỗ trợ mọi dòng máy.', 'label': 1},
    {'comment': 'Ở KTX có cho dùng máy sấy tóc công suất lớn không ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi nghe nhạc acoustic ở Tạ Hiện không?', 'label': 0},
    {'comment': 'Bạn muốn cải thiện vóc dáng? CLB yoga của trường đang tuyển thành viên mới. Cùng nhau tập luyện cho khỏe đẹp nhé. 🧘‍♀️', 'label': 0},
    {'comment': 'Thèm bún chả que tre quá chúng mày ơi. 😭', 'label': 0},
    {'comment': 'Hôm nay được thầy giáo khen bài thuyết trình. Vui cả ngày. 😊', 'label': 0},
    {'comment': 'Pass lại con bàn phím cơ Akko full box, còn bảo hành. Mới dùng 1 tháng. Giá 900k.', 'label': 1},
    {'comment': 'Học ngành Luật kinh tế có cần giỏi tranh luận không ạ?', 'label': 0},
    {'comment': 'Crush vừa thả tim ảnh của mình. Liệu đây có phải là một tín hiệu? 🤔', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản Github Student Pack. 🚀 Full quyền lợi cho dân IT. Giá chỉ 100k.', 'label': 1},
    {'comment': 'Môn Văn hóa Anh Mỹ có thi trắc nghiệm không mng?', 'label': 0},
    {'comment': 'Bài này viết sâu sắc, chạm đến trái tim mình.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các app học tập (Quizlet, Photomath...). 📚 Hỗ trợ học tập hiệu quả. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay vứt đồ lung tung. Mình nói mãi không được.', 'label': 0},
    {'comment': 'Bị từ chối visa đi Nhật. Buồn quá không biết phải làm sao.', 'label': 0},
    {'comment': 'Nhận quay dựng video, clip viral Tiktok. 🎬 Trend nào cũng bắt được. Giá cả sinh viên. Ib để xem sản phẩm.', 'label': 1},
    {'comment': 'Chỗ photo ở cổng sau có scan tài liệu không ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ ra mình là người tử tế đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ kinh nghiệm phỏng vấn, review công ty. Link ở bio. ✨', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào yên tĩnh, có ổ cắm để ngồi làm việc ở khu vực Tây Hồ không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm bạn cùng đi học tiếng Trung với ạ. Em đang tự học.', 'label': 0},
    {'comment': 'Bán tài khoản VIP TradingView Pro+. 📈 Công cụ không thể thiếu cho các trader. Giá rẻ hơn 80%.', 'label': 1},
    {'comment': 'Sắp đến ngày Nhà giáo Việt Nam rồi, nên tặng quà gì cho thầy cô chủ nhiệm đây?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán tào phớ ngon ở gần đây không?', 'label': 0},
    {'comment': 'Tuyển cộng tác viên bán tour du lịch trong nước. ✈️ Chiết khấu cao, không cần bỏ vốn. Được đi du lịch miễn phí.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải học lại môn Triết. Nản không tả nổi. 😩', 'label': 0},
    {'comment': 'Bài viết này hời hợt, không đi vào trọng tâm.', 'label': 0},
    {'comment': 'Nhận đan len, móc khóa theo yêu cầu. 🧶 Món quà handmade ý nghĩa. Ib để đặt mẫu nhé.', 'label': 1},
    {'comment': 'Có ai biết cách xin xác nhận thực tập tại công ty không ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, môn này điểm chuyên cần chiếm bao nhiêu % thế ạ?', 'label': 0},
    {'comment': '🔥 VÉ XEM PHIM CGV CHỈ 69K/VÉ 🔥 Áp dụng tất cả các rạp, tất cả các ngày. Mua combo bắp nước cũng được giảm giá. Ib mình chốt đơn liền.', 'label':1},
    {'comment': 'Đứa nào đi con wave alpha màu xanh bks 29-G1 123.45 dựng xe như cc, chắn hết lối đi.', 'label': 0},
    {'comment': 'Nhận chạy deadline sml cho anh em. Code, assignment, báo cáo... cân hết. Đừng để nước đến chân mới nhảy nhé. 🚀', 'label': 1},
    {'comment': 'Vl quả đề thi giữa kỳ hôm nay. Khó như chó, làm chả được câu nào. 😭', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán bún ngan ngon ở khu Cầu Giấy không mng?', 'label': 0},
    {'comment': 'Bán tài khoản VIP Pornhub Premium. 🔞 Xem full HD không quảng cáo. Ib kín đáo, an toàn.', 'label': 1},
    {'comment': 'Review CLB Bóng rổ của trường đi ạ. Có yêu cầu chiều cao ko mng?', 'label': 0},
    {'comment': 'Mày là cái thá gì mà dám nói chuyện với tao kiểu đó? Đm thằng ranh con.', 'label': 0},
    {'comment': 'Đậu phỏng vấn intern rồi chúng mày ơi. Vui vl. 🎉', 'label': 0},
    {'comment': 'Bạn muốn kiếm 500k/ngày? Chỉ cần 1 chiếc điện thoại. KHÔNG lừa đảo, KHÔNG đa cấp. Chấm (.) mình dr cách làm.', 'label': 1},
    {'comment': 'Có ai biết cách xin bảo lưu kết quả học tập ko?', 'label': 0},
    {'comment': 'Tìm 1 bạn nữ ở ghép, phòng 1tr5/người ở gần ĐH Thương Mại. Yêu cầu sạch sẽ, ko dẫn bạn trai về.', 'label': 0},
    {'comment': 'Nhìn cái mặt đã thấy ngu rồi, nói chuyện thì như đấm vào tai.', 'label': 0},
    {'comment': 'Dịch vụ làm dấu công ty, dấu chức danh. 💯 Sắc nét, bền đẹp, lấy ngay sau 10 phút. Ship hỏa tốc.', 'label': 1},
    {'comment': 'Mọi người ơi, lịch nghỉ hè có chính thức chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Pass lại cái tai nghe bluetooth mua về mà ko hợp tai. Mới dùng 1 lần, giá 200k.', 'label': 0},
    {'comment': 'Mày bớt sủa lại đi được không? Nghe ngứa đít vcl.', 'label': 0},
    {'comment': 'Có ai tham gia cuộc thi "Nét bút tri ân" của trường không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua gói Envato Elements dùng chung.', 'label': 0},
    {'comment': 'Nhận làm tiểu luận, khóa luận tốt nghiệp trọn gói. ✅ Cam kết điểm 7+, check turnitin < 5%. Uy tín làm nên thương hiệu.', 'label': 1},
    {'comment': 'Sắp thi vấn đáp mà chữ thầy trả thầy hết rồi. Cứu. 😥', 'label': 0},
    {'comment': 'Dm, thằng bạn vay tiền mua iphone mới xong bùng luôn. Chó thật sự.', 'label': 0},
    {'comment': 'Pass lại voucher buffet lẩu Haidilao. Giảm 300k trên tổng bill. Hạn sử dụng còn 1 tháng.', 'label': 1},
    {'comment': 'Môn Học máy có cần kiến thức toán tốt ko ạ?', 'label': 0},
    {'comment': 'Mày là cái cc gì mà cứ thích thể hiện thế?', 'label': 0},
    {'comment': 'Bán 100GB tài liệu, khóa học về AI, Machine Learning. 🤖 Cho các bạn tự học. Trọn bộ chỉ 150k.', 'label': 1},
    {'comment': 'Lịch thi lại đợt này có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách chuyển ảnh từ iphone sang máy tính mà ko bị giảm chất lượng ko?', 'label': 0},
    {'comment': 'Tuyển 50 bạn đọc báo, xem video kiếm tiền. 💸 Rút tiền trực tiếp về Momo. App uy tín, đã có hàng nghìn người tham gia. Link tải ở bio.', 'label':1},
    {'comment': 'Lại một kỳ nữa GPA dưới 2.5. Nản vcl.', 'label': 0},
    {'comment': 'Cái plot twist này đúng là lừa người xem, xàm cc.', 'label': 0},
    {'comment': 'GOM ORDER BÁNH BAO CHIÊN. 🥟 5k/cái. 5h chiều mình trả đơn ở cổng KTX nhé.', 'label': 1},
    {'comment': 'Có ai đi xem Rap Việt All-star concert ko?', 'label': 0},
    {'comment': 'Mày nghĩ mày ngon à? Bố mày thách mày đấy.', 'label': 0},
    {'comment': 'Nhận cài đặt phần mềm diệt virus Kaspersky, Bitdefender bản quyền. 🛡️ Bảo vệ máy tính của bạn an toàn. Giá chỉ 99k/năm.', 'label': 1},
    {'comment': 'Ở KTX có cho dùng bàn là hơi nước ko ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi chơi phố cổ ko?', 'label': 0},
    {'comment': 'CLB võ thuật của trường đang tuyển thành viên mới nhé. Rèn luyện sức khỏe, tự vệ bản thân. 🥋', 'label': 0},
    {'comment': 'Thèm bánh đúc nóng vcl chúng mày ạ. 😭', 'label': 0},
    {'comment': 'Hôm nay được crush rủ đi ăn chè. Vui như bắt được vàng. 😊', 'label': 0},
    {'comment': 'Pass lại con chuột Logitech G102 còn mới, full box. Mua về chơi game mà ko hợp tay. Giá 300k.', 'label': 1},
    {'comment': 'Học ngành Quản trị khách sạn có yêu cầu ngoại hình ko ạ?', 'label': 0},
    {'comment': 'Crush vừa like cmt của mình. Có nên inbox làm quen ko nhỉ? 🤔', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản Office 365, Windows 10/11 bản quyền. 💻 Kích hoạt vĩnh viễn, giá sinh viên.', 'label': 1},
    {'comment': 'Môn Phân tích thiết kế hệ thống có khó ko mng?', 'label': 0},
    {'comment': 'Ngu như m thì im mẹ mồm đi.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang báo (VnExpress, CafeF...). 📰 Đọc tin tức không giới hạn. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay đi nhậu về khuya rồi ói ra phòng. Bẩn vcl.', 'label': 0},
    {'comment': 'Apply học bổng mấy lần đều trượt. Thấy mình vô dụng thật sự.', 'label': 0},
    {'comment': 'Nhận làm video recap, video kỷ yếu. 🎬 Lưu giữ những khoảnh khắc đẹp của thời sinh viên. Giá cả thương lượng.', 'label': 1},
    {'comment': 'Chỗ photo ở gần nhà sách có in decal ko ạ?', 'label': 0},
    {'comment': 'Mày bớt tỏ ra mình là người đạo đức đi.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ tài liệu tự học design (PTS, AI, Figma). Link ở bio. ✨', 'label': 1},
    {'comment': 'Có ai biết quán cafe trứng nào ngon ở khu Hoàn Kiếm ko?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm đồng đội đi Mộc Châu cuối tuần này với ạ.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các app hẹn hò (Tinder, Bumble...). 🔥 Tăng cơ hội match, tìm người yêu. Ib kín đáo.', 'label': 1},
    {'comment': 'Sắp đến sinh nhật ny rồi, nên tặng quà gì cho ý nghĩa đây mng?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán cháo sườn quẩy ngon ở gần đây ko?', 'label': 0},
    {'comment': 'Tuyển CTV bán đồ ăn vặt online. 🍡 Không cần vốn, chỉ cần đăng bài. Chiết khấu hấp dẫn. Được ăn thử sản phẩm.', 'label': 1},
    {'comment': 'Lại một kỳ nữa tạch môn thể dục. Đen vcl. 😩', 'label': 0},
    {'comment': 'Bài viết này phân tích như cc, đọc tốn thời gian.', 'label': 0},
    {'comment': 'Nhận order hàng Mỹ, Nhật, Hàn. ✈️ Hàng auth, bill đầy đủ. Công mua hộ chỉ 5%. Hàng về sau 2-3 tuần.', 'label': 1},
    {'comment': 'Có ai biết cách xin lại biên lai thu tiền học phí ko ạ?', 'label': 0},
    {'comment': 'Mọi người ơi, môn này thi cuối kỳ là tự luận hay trắc nghiệm thế?', 'label': 0},
    {'comment': '🔥 VOUCHER STARBUCKS MUA 1 TẶNG 1 🔥 Áp dụng toàn menu. Số lượng có hạn. Dr mình để lấy voucher free nhé.', 'label': 1},
    {'comment': 'Cần tìm 1 bạn nam ở ghép khu vực Hai Bà Trưng. Phòng 3tr2, rộng, có ban công, điều hòa. Yêu cầu gọn gàng, có ý thức.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 3-5 triệu/tháng lúc rảnh rỗi? 💸 Công việc seeding, review sản phẩm. Làm online 100%. Ib mình để nhận việc ngay.', 'label': 1},
    {'comment': 'Đm cái đứa nào đi ex 150 độ pô nẹt trong sân KTX. Mất ngủ vì mày.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán dimsum ngon mà giá sv không ạ?', 'label': 0},
    {'comment': 'Tuyển mẫu xăm mini, hình đơn giản. ✍️ Free 100% cho 10 bạn đăng ký đầu tiên. Giúp thợ mới lên tay nghề.', 'label': 1},
    {'comment': 'Review CLB Âm nhạc của trường đi ạ. Em biết chơi sáo thì có tham gia được không?', 'label': 0},
    {'comment': 'Nhận order đồ ăn vặt nội địa Trung. 🌶️ Chân vịt cay, que cay, miến chua cay... Ib để xem menu.', 'label': 1},
    {'comment': 'Mày nói chuyện nghe như thằng thiểu năng vậy. Cút mẹ mày đi.', 'label': 0},
    {'comment': 'Cuối cùng cũng hoàn thành xong cái báo cáo thực tập. Nhẹ cả người. 🙏', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang truyện tranh (nettruyen, webtoon...). 📖 Đọc truyện không quảng cáo. Chỉ 20k/tháng.', 'label': 1},
    {'comment': 'Có ai biết cách đăng ký học ghép với khóa dưới không?', 'label': 0},
    {'comment': 'Cần tìm một bạn nữ chụp ảnh lookbook cho brand thời trang local. Có trả phí.', 'label': 0},
    {'comment': 'Nhìn cái mặt mày đã thấy ghét rồi, đừng có nói nhiều.', 'label': 0},
    {'comment': 'Dịch vụ lấy lại tài khoản Facebook, Instagram bị hack, bị khóa. 🚀 Nhanh chóng, uy tín, bảo mật. Tỷ lệ thành công 99%.', 'label': 1},
    {'comment': 'Mọi người ơi, lịch học bù nghỉ lễ đã có chưa ạ?', 'label': 0},
    {'comment': 'Góc pass đồ: Mình pass lại cái máy tính bảng Samsung còn dùng tốt. Pin hơi chai. Giá 800k.', 'label': 0},
    {'comment': 'Bạn nào bị hôi miệng, sâu răng, nhiệt miệng thì ib mình nhé. Nước súc miệng thảo dược gia truyền cam kết hiệu quả.', 'label': 1},
    {'comment': 'Có ai tham gia cuộc thi "SV Startup" của trường không?', 'label': 0},
    {'comment': 'Cần tìm người share tiền mua data train AI cho đồ án.', 'label': 0},
    {'comment': 'Nhận làm video lyric, video karaoke. 🎤 Chữ chạy đẹp, khớp nhạc. Giá sinh viên. Ib để xem sản phẩm.', 'label': 1},
    {'comment': 'Sắp thi hết môn rồi mà vẫn còn một đống bài chưa làm. Stress vcl. 😩', 'label': 0},
    {'comment': 'Thằng bạn thân vay 500k từ tháng trước mà giờ đòi nó cứ lảng đi. Cay.', 'label': 0},
    {'comment': 'Pass lại vé xe Limousine về Hải Phòng. Sáng mai 8h xe chạy. Do mình ngủ quên. 😭 Giá rẻ.', 'label': 1},
    {'comment': 'Môn Thị giác máy tính có khó không các anh chị K64?', 'label': 0},
    {'comment': 'Mày bớt ra vẻ ta đây lại đi.', 'label': 0},
    {'comment': 'Bán 3000+ template, plugin, theme Wordpress bản quyền. 💻 Giúp bạn làm web chuyên nghiệp. Trọn bộ chỉ 299k.', 'label': 1},
    {'comment': 'Lịch bảo vệ khóa luận tốt nghiệp đợt này có chưa mng?', 'label': 0},
    {'comment': 'Có ai biết cách quay màn hình máy tính có cả âm thanh hệ thống không?', 'label': 0},
    {'comment': 'Tuyển 100 bạn chơi game kiếm tiền. 🎮 Cày level, bán vật phẩm. Thu nhập không giới hạn. Ib để vào group.', 'label': 1},
    {'comment': 'Lại một kỳ nữa không được học bổng. Buồn của tôi.', 'label': 0},
    {'comment': 'Cái kết phim như cc, phí 2 tiếng cuộc đời.', 'label': 0},
    {'comment': 'GOM ORDER BÁNH TRỨNG NON KEM CHEESE. 🧀 Thơm ngon béo ngậy. 15k/cái. 4h chiều mình ship nhé.', 'label': 1},
    {'comment': 'Có ai đi xem festival âm nhạc ở Ecopark không?', 'label': 0},
    {'comment': 'Mày nghĩ mày là ai mà chửi tao? Đm thằng vô học.', 'label': 0},
    {'comment': 'Nhận sửa chữa, thay pin, màn hình cho đồng hồ thông minh. ⌚️ Lấy ngay, giá rẻ. Hỗ trợ mọi loại smartwatch.', 'label': 1},
    {'comment': 'Ở KTX có cho nuôi cá cảnh không ạ?', 'label': 0},
    {'comment': 'Tối nay có ai đi bar không? Chán quá.', 'label': 0},
    {'comment': 'CLB Cờ vua của trường đang tuyển thành viên mới nhé. Giao lưu, học hỏi, nâng cao kỳ nghệ. ♟️', 'label': 0},
    {'comment': 'Thèm phở gà trộn vcl chúng mày ạ. 😭', 'label': 0},
    {'comment': 'Hôm nay đi thi quên mang máy tính. May mà có con bạn thân cho mượn. Quý nó vcl. ❤️', 'label': 0},
    {'comment': 'Pass lại con main B450 còn dùng tốt. Đã nâng cấp nên thừa ra. Giá 800k.', 'label': 1},
    {'comment': 'Học ngành Quản trị nhân lực ra trường làm ở đâu ạ?', 'label': 0},
    {'comment': 'Crush vừa rep tin nhắn của mình. Chắc tối nay khỏi ngủ. 🥰', 'label': 0},
    {'comment': 'Chuyên cung cấp tài khoản VIP cho các trang tải file (Fshare, rapidgator...). 🚀 Tải file không giới hạn tốc độ.', 'label': 1},
    {'comment': 'Môn Phương pháp nghiên cứu khoa học có phải làm bài tập lớn không mng?', 'label': 0},
    {'comment': 'Ngu thì im mẹ đi, nói ra người ta lại cười cho.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các app chỉnh ảnh (VSCO, Lightroom, Snapseed...). ✨ Full màu, full công cụ. Ib để chốt.', 'label': 1},
    {'comment': 'Bạn cùng phòng hay ăn vụng đồ của mình. Bực mình mà không có bằng chứng.', 'label': 0},
    {'comment': 'Đi làm thêm bị quỵt lương. Sinh viên khổ vl.', 'label': 0},
    {'comment': 'Nhận viết, đăng bài PR, quảng cáo trên các trang báo, diễn đàn lớn. 📰 Giúp thương hiệu của bạn được nhiều người biết đến.', 'label': 1},
    {'comment': 'Chỗ giữ xe ở nhà A có nhận trông xe đạp điện không ạ?', 'label': 0},
    {'comment': 'Mày bớt đóng vai nạn nhân đi, nhìn ngứa mắt lắm.', 'label': 0},
    {'comment': 'Tham gia nhóm chia sẻ mã giảm giá, deal hời ăn uống. 🍔 Link ở bio.', 'label': 1},
    {'comment': 'Có ai biết quán cafe nào có máy chiếu để xem bóng đá không?', 'label': 0},
    {'comment': 'Ad cho em đăng bài tìm bạn cùng đi học IELTS với ạ. Em đang ở level 5.0.', 'label': 0},
    {'comment': 'Bán tài khoản VIP LinkedIn Premium. 💼 Hỗ trợ tìm việc, kết nối network hiệu quả. Giá sinh viên.', 'label': 1},
    {'comment': 'Sắp đến sinh nhật mẹ rồi, nên tặng quà gì đây mọi người?', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán gà tần ngon ở gần đây không?', 'label': 0},
    {'comment': 'Tuyển cộng tác viên bán vé máy bay, tour du lịch. ✈️ Chiết khấu cao, thời gian linh hoạt. Không cần kinh nghiệm.', 'label': 1},
    {'comment': 'Lại một kỳ nữa phải học lại thể dục. Cuộc đời thật trớ trêu. 😩', 'label': 0},
    {'comment': 'Bài viết này phân tích như một thằng ngáo đá.', 'label': 0},
    {'comment': 'Nhận order mỹ phẩm chính hãng Hàn, Nhật, Âu Mỹ. 💄 Bill đầy đủ. Cam kết auth trọn đời. Dr để mình tư vấn.', 'label': 1},
    {'comment': 'Có ai biết cách xin miễn học môn Tiếng Anh vì đã có chứng chỉ IELTS không ạ?', 'label': 0},
    {'comment': 'Thằng chó nào lấy cái áo khoác của tao ở phòng tự học thì trả lại đây. Bố mày mà tìm ra thì mày chết con mẹ mày rồi.', 'label': 0},
    {'comment': 'Review môn Lập trình web của thầy Dũng đi anh em. Thầy có khó tính lắm không?', 'label': 0},
    {'comment': '🔥 VAY TIỀN SINH VIÊN - GIẢI NGÂN 5 PHÚT 🔥 Chỉ cần CMND/CCCD. Không gọi người thân. Bấm vào link bio để được tư vấn ngay.', 'label': 1},
    {'comment': 'Hôm nay đi thi gặp đúng câu mình bỏ. Đen như con chó. 😩', 'label': 0},
    {'comment': 'Mày nói chuyện nghe ngu lồn vcl. Câm mẹ cái mồm mày vào đi.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào sửa laptop uy tín mà giá sinh viên ở khu Bách Khoa không ạ?', 'label': 0},
    {'comment': 'Cần tìm 1 bạn nữ ở ghép, phòng rộng 25m2 ở Triều Khúc, 1tr5/người. Yêu cầu sạch sẽ, có ý thức.', 'label': 0},
    {'comment': 'Nhận làm trọn gói báo cáo thực tập, khoá luận. ✅ Đảm bảo điểm 7+, check đạo văn < 10%. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Cái con ml lớp trưởng làm việc như cc, thông báo thì chậm mà thái độ thì như bố đời.', 'label': 0},
    {'comment': 'Học xong 4 năm đại học mà vẫn thấy mông lung, đéo biết tương lai đi về đâu.', 'label': 0},
    {'comment': 'Có ai có link xem One Piece chap mới không cho mình xin với. Hóng quá.', 'label': 0},
    {'comment': 'Con phò đấy mà cũng có đứa yêu được à? Mắt mù hay sao.', 'label': 0},
    {'comment': 'Đm web trường sập đúng lúc đăng ký tín chỉ. Cay vcl.', 'label': 0},
    {'comment': 'Pass lại tài khoản Adobe full app bản quyền. Còn 6 tháng sử dụng. Giá chỉ 200k. ✍️', 'label': 1},
    {'comment': 'Điểm GPA kỳ này của chúng mày sao rồi? Flex nhẹ cái 3.7 xem nào. 😂', 'label': 0},
    {'comment': 'Mày là thằng nào mà sủa kinh thế? Bố mẹ đéo dạy mày à?', 'label': 0},
    {'comment': 'Có ai đi xem show của Đen Vâu cuối tháng này không?', 'label': 0},
    {'comment': 'Thèm bún đậu mắm tôm quá mà hết mẹ tiền rồi. 😭', 'label': 0},
    {'comment': 'Bán tài khoản ChatGPT Plus giá rẻ, dùng chung ổn định. Hỗ trợ anh em chạy deadline, code dạo. 🚀', 'label': 1},
    {'comment': 'CLB Marketing có đang tuyển cộng tác viên không ạ?', 'label': 0},
    {'comment': 'Thằng lồn nào đi ỉa không dội nước trong toilet tầng 3. Vô ý thức như một con súc vật.', 'label': 0},
    {'comment': 'Lịch nghỉ lễ 2/9 trường mình được nghỉ mấy ngày vậy mọi người?', 'label': 0},
    {'comment': 'Cần tìm người làm chung project môn Trí tuệ nhân tạo. Yêu cầu code được Python.', 'label': 0},
    {'comment': 'Nhìn cái mặt mày đã thấy hãm lồn rồi, đừng có cố tỏ ra tri thức.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page fanclub, lương 2-3tr/tháng. Yêu cầu online thường xuyên, biết des ảnh cơ bản.', 'label': 1},
    {'comment': 'Học xong môn này tao thấy mình ngu đi chúng mày ạ.', 'label': 0},
    {'comment': 'Có ai biết quán net nào ở gần trường có máy ngon để chơi Elden Ring không?', 'label': 0},
    {'comment': 'Con đĩ đấy cặp với cả thầy giáo mà vẫn ra vẻ thanh cao được. Tởm vcl.', 'label': 0},
    {'comment': 'Deadline dí đến đít rồi mà vẫn chưa có hứng làm. 🥲', 'label': 0},
    {'comment': 'Có ai biết cách xin miễn giảm môn GDQP không ạ? Em có chứng chỉ rồi.', 'label': 0},
    {'comment': 'Mày với nó đúng là một cặp trời sinh. Ngu và nguy hiểm.', 'label': 0},
    {'comment': 'Nhận order giày sneaker rep 1:1, full box + bill. 👟 Hàng về sau 7-10 ngày. Check dr để xem feedback.', 'label': 1},
    {'comment': 'Đứa nào có link tải full bộ giáo trình của khoa mình không cho tao xin với.', 'label': 0},
    {'comment': 'Cái thằng báo thủ, đi học thì không đi, đến lúc thi thì chép bài tao. Đm.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán chân gà nướng ngon ở khu Hồ Tùng Mậu không?', 'label': 0},
    {'comment': 'Cứu, mai thi vấn đáp mà giờ chữ đéo nào cũng không vào đầu.', 'label': 0},
    {'comment': 'Pass lại vé xem bóng đá trận Việt Nam - Thái Lan. Khán đài B, giá gốc.', 'label': 1},
    {'comment': 'Mày là cái đéo gì mà dám chửi bạn tao? Thích solo không?', 'label': 0},
    {'comment': 'Học lại tốn tiền vcl. Xót cả ruột.', 'label': 0},
    {'comment': 'Mọi người ơi, quán cơm gần cổng trường có tăng giá không ạ?', 'label': 0},
    {'comment': 'Cái loại rẻ rách như mày thì chỉ có đi lừa người khác là giỏi.', 'label': 0},
    {'comment': 'Có ai biết cách xin giấy xác nhận sinh viên online không ạ?', 'label': 0},
    {'comment': 'Bạn nào cần người đi chợ, nấu cơm hộ thì ib mình nhé. 👩‍🍳 Giá sinh viên, đảm bảo ngon sạch.', 'label': 1},
    {'comment': 'Dm thằng nào hút thuốc trong thang máy KTX, ám hết cả mùi vào người tao.', 'label': 0},
    {'comment': 'Có ai biết thủ tục xin ra khỏi đoàn trường không?', 'label': 0},
    {'comment': 'Mày bớt sủa bậy đi được không? Đéo ai quan tâm đâu.', 'label': 0},
    {'comment': 'Chạy xong deadline chắc tao đột quỵ. Mệt vcl.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 100-200k/ngày? 💸 Công việc xem video, thả tim. Rút tiền về Momo. Ib mình để được hướng dẫn.', 'label': 1},
    {'comment': 'Thi lại môn Triết. Cuộc đời thật trớ trêu. 😩', 'label': 0},
    {'comment': 'Thằng đấy gay lọ mà cứ thích đi gạ con gái. Bệnh hoạn vcl.', 'label': 0},
    {'comment': 'Cái thằng óc chó nào chấm bài tao môn Triết mà cho 5 điểm vậy? Bài làm dài vl mà điểm như cc.', 'label': 0},
    {'comment': '🔥 NHẬN LÀM BẰNG LÁI XE A1, B2 BAO ĐẬU 🔥 Không cần thi, có hồ sơ gốc. Uy tín, kín đáo. Ib Zalo: 09xx.xxx.xxx', 'label': 1},
    {'comment': 'Đm con ml lớp phó học tập, hỏi nó cái gì cũng vênh cái mặt lên như bố đời. Ghét vcl.', 'label': 0},
    {'comment': 'Bán full bộ tài liệu ôn thi TOEIC 900+. 📚 File PDF + Audio + App. Trọn bộ chỉ 99k. Cam kết không hiệu quả refund.', 'label': 1},
    {'comment': 'Tối nay có kèo nhậu không chúng mày? Chán quá.', 'label': 0},
    {'comment': 'Tuyển 20 bạn nữ PG cho sự kiện game. Yêu cầu ngoại hình xinh, cao >1m65. Lương 1 củ/ngày. Dr gửi info + ảnh. 💃', 'label': 1},
    {'comment': 'Mày là cái thá gì mà dám lên mặt dạy khôn tao? Ngu thì câm mẹ mồm vào.', 'label': 0},
    {'comment': 'Link xem full clip em gái sinh viên lộ hàng cực nét. 🔞 Link ở bio của mình nhé ae.', 'label': 1},
    {'comment': 'Học xong môn này tao cảm thấy đầu óc mình bị ngu đi hẳn.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán mì cay ngon ở khu vực Hà Đông không ạ?', 'label': 0},
    {'comment': 'NHẬN HACK LIKE, FOLLOW FACEBOOK, TIKTOK. 🚀 Giá rẻ nhất thị trường. Giúp bạn sống ảo thành công. Ib để được báo giá.', 'label': 1},
    {'comment': 'Cái thằng lồn phòng bên cạnh đêm nào cũng dẫn gái về phòng làm ồn, đéo cho ai ngủ.', 'label': 0},
    {'comment': 'VAY TIỀN ONLINE LÃI SUẤT 0%. 💸 Chỉ cần thẻ SV hoặc CMND. Giải ngân sau 3 phút. Bấm vào link để vay ngay.', 'label': 1},
    {'comment': 'Đm lại rớt môn. Tốn tiền học lại vcl.', 'label': 0},
    {'comment': 'Có ai có đề thi cuối kỳ môn CSDL của thầy Tuấn năm ngoái không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'Nhóm kéo Baccarat, Tài Xỉu. 💰 Vốn ít húp nhiều, cam kết về bờ an toàn. Ae xa bờ mau vào nhóm.', 'label': 1},
    {'comment': 'Con đĩ đấy thảo mai vcl, trước mặt thì chị em sau lưng thì đâm chọt.', 'label': 0},
    {'comment': 'Pass lại con acc Genshin Impact full waifu, nhiều vũ khí 5 sao. Tâm huyết nên giá hơi cao. Ib thương lượng.', 'label': 1},
    {'comment': 'Mày ăn nói như một thằng vô học, bố mẹ mày đéo dạy mày à?', 'label': 0},
    {'comment': 'Deadline ngập mặt mà vẫn phải đi làm thêm. Kiếp sv khổ như chó.', 'label': 0},
    {'comment': 'Cần tìm người share phòng trọ khu vực Chùa Láng, 2tr/người. Yêu cầu sạch sẽ.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Onlyfans, Fansly. 🥵 Full clip, ảnh HD. Giá chỉ 50k/tháng. Dr để chốt lẹ.', 'label': 1},
    {'comment': 'Đm thầy giáo dạy như cc, chả hiểu con mẹ gì.', 'label': 0},
    {'comment': 'Tuyển CTV bán mỹ phẩm, chiết khấu 30-40%. 💄 Không cần vốn, không ôm hàng. Được đào tạo bán hàng từ A-Z.', 'label': 1},
    {'comment': 'Mày là cái loại đéo có liêm sỉ à? Ăn cắp bài của người khác còn già mồm.', 'label': 0},
    {'comment': 'Có ai biết cách xin giấy tạm hoãn nghĩa vụ quân sự không?', 'label': 0},
    {'comment': 'Nhận thông tắc cống, bồn cầu. 🚽 Có mặt sau 15 phút. Xử lý triệt để, giá sinh viên.', 'label': 1},
    {'comment': 'Thằng chó đấy vay tiền tao mấy tháng rồi đéo trả, gọi thì đéo nghe. Cay vcl.', 'label': 0},
    {'comment': 'Mọi người ơi, quán net gần trường mình có chương trình khuyến mãi gì không ạ?', 'label': 0},
    {'comment': 'Cái loại rẻ rách, đéo có tiền còn bày đặt sĩ diện.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang cá độ bóng đá. ⚽️ Tip kèo miễn phí, tỉ lệ thắng >85%. Ae xa bờ về đây với đội của mình.', 'label': 1},
    {'comment': 'Đm cái thằng lồn nào đi SH để xe giữa sân KTX, bố mày đéo dắt xe ra được.', 'label': 0},
    {'comment': 'Nhận làm assignment, project, báo cáo thực tập. ✍️ Giá sinh viên, đảm bảo qua môn. Ib để được báo giá chi tiết.', 'label': 1},
    {'comment': 'Học xong môn này tao thấy mình ngu như một con chó.', 'label': 0},
    {'comment': 'Tuyển 10 bạn nữ livestream app X. Chỉ cần nói chuyện, ca hát. Thu nhập 10-20tr/tháng + quà tặng. Dr để phỏng vấn online.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán ốc luộc ngon ở khu vực Thanh Xuân không?', 'label': 0},
    {'comment': 'Mày là cái loại đéo có não à? Nói chuyện như thằng thiểu năng.', 'label': 0},
    {'comment': '🔥 KEY WINDOWS 10/11 PRO, OFFICE 365 BẢN QUYỀN 🔥 Giá chỉ 99k, kích hoạt vĩnh viễn. Bảo hành 1 đổi 1. Dr ngay!', 'label': 1},
    {'comment': 'Bạn muốn kiếm 200k/ngày chỉ bằng việc nghe nhạc? 🎧 App mới ra mắt, uy tín, rút tiền về Momo. Link tải ở bio.', 'label': 1},
    {'comment': 'Deadline dí sml mà thằng bạn cùng nhóm thì báo. Cay vcl.', 'label': 0},
    {'comment': 'Bán voucher nghỉ dưỡng Vinpearl 3N2Đ. 🏝️ Bao gồm vé máy bay + ăn sáng buffet. Giá rẻ hơn mua trực tiếp 50%.', 'label': 1},
    {'comment': 'Thằng chó đấy nợ tao 2 củ từ năm ngoái đến giờ chưa trả. Đm nó.', 'label': 0},
    {'comment': 'Mọi người ơi, CLB Bóng chuyền có tuyển thành viên mới không ạ?', 'label': 0},
    {'comment': 'Link group Zalo xem clip hot girl, sinh viên hằng ngày. Free cho 100 ae đầu tiên. 🥵', 'label': 1},
    {'comment': 'Mày là cái loại súc vật, đéo phải con người.', 'label': 0},
    {'comment': 'Nhận order Taobao, Pinduoduo không qua trung gian. 🚚 Hàng về nhanh, công mua hộ rẻ nhất thị trường.', 'label': 1},
    {'comment': 'Đm lại mất điện đúng lúc đang làm bài tập lớn. Đen như con chó.', 'label': 0},
    {'comment': 'Có ai có đề thi môn Mạng máy tính của K65 không cho em xin với ạ. 🙏', 'label': 0},
    {'comment': 'Bạn đang gặp khó khăn về tài chính? 💸 Vay nóng 1-10 triệu, giải ngân trong 10 phút. Không cần gặp mặt, chỉ cần CMND.', 'label': 1},
    {'comment': 'Cái loại bạn như mày, có cũng như không. Cút mẹ mày đi cho nước nó trong.', 'label': 0},
    {'comment': 'Bán tài khoản Grammarly Premium, Turnitin check đạo văn. ✍️ Cần thiết cho ae sinh viên làm luận. Giá chỉ 30k/tháng.', 'label': 1},
    {'comment': 'Trượt phỏng vấn intern. Buồn vcl chúng mày ạ.', 'label': 0},
    {'comment': 'Thằng đấy gay lọ mà cứ tỏ ra men lì. Nhìn tởm vcl.', 'label': 0},
    {'comment': 'Chỉ 299k sở hữu ngay bí kíp đầu tư Forex cho người mới. 📈 Cam kết x2 tài khoản sau 1 tháng. Ib để nhận tài liệu.', 'label': 1},
    {'comment': 'Học phí kỳ này tăng nhiều quá. Bố mẹ ở nhà còng lưng ra lo tiền.', 'label': 0},
    {'comment': 'Mày nói chuyện nghe ngứa cả lồn.', 'label': 0},
    {'comment': 'Pass lại con acc Tốc Chiến rank thách đấu, full tướng, 150 skin. Giá 1 củ khoai. Giao dịch trung gian.', 'label': 1},
    {'comment': 'Có ai đi xem Lễ hội âm nhạc Monsoon không? Lập team đi chung đi.', 'label': 0},
    {'comment': 'Cái con ml đấy giả tạo vcl, trước mặt thì thảo mai sau lưng thì nói xấu.', 'label': 0},
    {'comment': 'Cái thằng óc cặc nào chấm bài tao môn Triết mà cho 5 điểm vậy? Bài làm dài vl mà điểm như cc.', 'label': 0},
    {'comment': '🔥 NHẬN LÀM BẰNG LÁI XE A1, B2 BAO ĐẬU 🔥 Không cần thi, có hồ sơ gốc. Uy tín, kín đáo. Ib Zalo: 09xx.xxx.xxx', 'label': 1},
    {'comment': 'Đm con ml lớp phó học tập, hỏi nó cái gì cũng vênh cái mặt lên như bố đời. Ghét vcl.', 'label': 0},
    {'comment': 'Bán full bộ tài liệu ôn thi TOEIC 900+. 📚 File PDF + Audio + App. Trọn bộ chỉ 99k. Cam kết không hiệu quả refund.', 'label': 1},
    {'comment': 'Tối nay có kèo nhậu không chúng mày? Chán quá.', 'label': 0},
    {'comment': 'Tuyển 20 bạn nữ PG cho sự kiện game. Yêu cầu ngoại hình xinh, cao >1m65. Lương 1 củ/ngày. Dr gửi info + ảnh. 💃', 'label': 1},
    {'comment': 'Mày là cái thá gì mà dám lên mặt dạy khôn tao? Ngu thì câm mẹ mồm vào.', 'label': 0},
    {'comment': 'Link xem full clip em gái sinh viên lộ hàng cực nét. 🔞 Link ở bio của mình nhé ae.', 'label': 1},
    {'comment': 'Học xong môn này tao cảm thấy đầu óc mình bị ngu đi hẳn.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán mì cay ngon ở khu vực Hà Đông không ạ?', 'label': 0},
    {'comment': 'NHẬN HACK LIKE, FOLLOW FACEBOOK, TIKTOK. 🚀 Giá rẻ nhất thị trường. Giúp bạn sống ảo thành công. Ib để được báo giá.', 'label': 1},
    {'comment': 'Cái thằng lồn phòng bên cạnh đêm nào cũng dẫn gái về phòng làm ồn, đéo cho ai ngủ.', 'label': 0},
    {'comment': 'VAY TIỀN ONLINE LÃI SUẤT 0%. 💸 Chỉ cần thẻ SV hoặc CMND. Giải ngân sau 3 phút. Bấm vào link để vay ngay.', 'label': 1},
    {'comment': 'Đm lại rớt môn. Tốn tiền học lại vcl.', 'label': 0},
    {'comment': 'Có ai có đề thi cuối kỳ môn CSDL của thầy Tuấn năm ngoái không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'Nhóm kéo Baccarat, Tài Xỉu. 💰 Vốn ít húp nhiều, cam kết về bờ an toàn. Ae xa bờ mau vào nhóm.', 'label': 1},
    {'comment': 'Con đĩ đấy thảo mai vcl, trước mặt thì chị em sau lưng thì đâm chọt.', 'label': 0},
    {'comment': 'Pass lại con acc Genshin Impact full waifu, nhiều vũ khí 5 sao. Tâm huyết nên giá hơi cao. Ib thương lượng.', 'label': 1},
    {'comment': 'Mày ăn nói như một thằng vô học, bố mẹ mày đéo dạy mày à?', 'label': 0},
    {'comment': 'Deadline ngập mặt mà vẫn phải đi làm thêm. Kiếp sv khổ như chó.', 'label': 0},
    {'comment': 'Cần tìm người share phòng trọ khu vực Chùa Láng, 2tr/người. Yêu cầu sạch sẽ.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Onlyfans, Fansly. 🥵 Full clip, ảnh HD. Giá chỉ 50k/tháng. Dr để chốt lẹ.', 'label': 1},
    {'comment': 'Đm thầy giáo dạy như cc, chả hiểu con mẹ gì.', 'label': 0},
    {'comment': 'Tuyển CTV bán mỹ phẩm, chiết khấu 30-40%. 💄 Không cần vốn, không ôm hàng. Được đào tạo bán hàng từ A-Z.', 'label': 1},
    {'comment': 'Mày là cái loại đéo có liêm sỉ à? Ăn cắp bài của người khác còn già mồm.', 'label': 0},
    {'comment': 'Có ai biết cách xin giấy tạm hoãn nghĩa vụ quân sự không?', 'label': 0},
    {'comment': 'Nhận thông tắc cống, bồn cầu. 🚽 Có mặt sau 15 phút. Xử lý triệt để, giá sinh viên.', 'label': 1},
    {'comment': 'Thằng chó đấy vay tiền tao mấy tháng rồi đéo trả, gọi thì đéo nghe. Cay vcl.', 'label': 0},
    {'comment': 'Mọi người ơi, quán net gần trường mình có chương trình khuyến mãi gì không ạ?', 'label': 0},
    {'comment': 'Cái loại rẻ rách, đéo có tiền còn bày đặt sĩ diện.', 'label': 0},
    {'comment': 'Bán tool hack map, hack lag Liên Quân Mobile. 🚀 Anti ban, an toàn tuyệt đối. Mua 1 lần dùng vĩnh viễn. Ib để nhận link cài.', 'label': 1},
    {'comment': 'Đm cái con chó nào ăn cắp cái sạc dự phòng của tao trên thư viện. Bố mày mà biết thì mày no đòn.', 'label': 0},
    {'comment': 'VAY TÍN DỤNG ĐEN - GIẢI NGÂN SAU 5 PHÚT. 👹 Không cần thế chấp, không thẩm định. Chỉ cần CMND. Alo ngay 09xx.xxx.xxx', 'label': 1},
    {'comment': 'Có ai biết cách xin phúc khảo bài thi không? Tao chắc chắn con ml đấy chấm sai bài tao.', 'label': 0},
    {'comment': 'YẾU SINH LÝ, RA NHANH? 💊 Kẹo ngậm cường dương kéo dài thời gian quan hệ. Hiệu quả ngay lần đầu. Kín đáo, an toàn. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Học xong 4 năm chắc tao tổn thọ vì deadline mất. Mệt vcl.', 'label': 0},
    {'comment': 'Đứa nào có link tải Photoshop crack không cho tao xin với. Hết mẹ tiền rồi.', 'label': 0},
    {'comment': 'Tuyển nhân viên seeding cho các sàn TMĐT. 💬 Chỉ cần cmt dạo, đánh giá 5 sao. 5k/cmt. Ib để nhận job.', 'label': 1},
    {'comment': 'Cái thằng lồn đấy đá bóng như phá đám, cả đội gánh còng lưng.', 'label': 0},
    {'comment': 'Nhận làm bằng Đại học, Cao đẳng, chứng chỉ ngoại ngữ. Phôi thật, tem thật, bao soi. 📜 Ib để trao đổi.', 'label': 1},
    {'comment': 'Mày là cái loại súc vật đéo có nhân tính à? Chuyện đấy mà cũng làm được.', 'label': 0},
    {'comment': 'Đm con người yêu cũ, cắm sừng tao xong giờ lại đi kể xấu tao với bạn bè. Đúng là loại đĩ.', 'label': 0},
    {'comment': 'Bán đồ lót nữ sinh đã qua sử dụng. 👙 Mùi hương chân thật. Anh em có nhu cầu dr kín đáo nhé.', 'label': 1},
    {'comment': 'Có ai biết quán bún đậu nào nhiều thịt, nhiều topping không ạ?', 'label': 0},
    {'comment': 'Nhóm Zalo chia sẻ kèo bóng cỏ, tip nội gián. Tỉ lệ thắng 90%. ⚽️ Ae vào bờ cùng anh em mình.', 'label': 1},
    {'comment': 'Mày nghĩ mày là ai mà dám chửi nó? Nhìn lại mình đi, con chó.', 'label': 0},
    {'comment': 'Bán tài khoản Netflix, Spotify, Youtube Premium. 🎬 Đồng giá 30k/tháng. Bảo hành uy tín. Ib để chốt đơn.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn này vừa già vừa dê, toàn lựa mấy em xinh để hỏi bài.', 'label': 0},
    {'comment': 'Thi lại tốn tiền vcl. Đéo muốn xin tiền bố mẹ nữa.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 300k/ngày? Tải app ABC, nhập mã XYZ để nhận ngay 50k. 💸 Rút tiền về Momo sau 1 phút.', 'label': 1},
    {'comment': 'Cái loại bạn như mày, tao đéo thèm chấp.', 'label': 0},
    {'comment': 'Sim số đẹp, sim năm sinh. 📱 Giá sinh viên. Chọn số miễn phí. Ib mình để được tư vấn.', 'label': 1},
    {'comment': 'Có ai đăng ký thi lại môn GDTC không, lập team ôn đi.', 'label': 0},
    {'comment': 'Con đĩ đấy cặp với đại gia mà cứ lên mạng rao giảng đạo đức. Tởm vcl.', 'label': 0},
    {'comment': 'Dịch vụ viết thuê luận văn, đồ án tốt nghiệp. 🎓 Đội ngũ thạc sĩ, tiến sĩ nhiều kinh nghiệm. Cam kết điểm cao.', 'label': 1},
    {'comment': 'Đm thằng nào đi xe máy trong sân trường mà nẹt pô inh ỏi. Vô học.', 'label': 0},
    {'comment': 'Có ai biết cách fix lỗi font chữ trong slide không ạ?', 'label': 0},
    {'comment': 'Pass lại vé concert Taylor Swift tại Sing. 🎤 Do mình không sắp xếp được lịch bay. Giá gốc + phí đổi tên.', 'label': 1},
    {'comment': 'Mày là cái thằng đần độn, não toàn tinh trùng à?', 'label': 0},
    {'comment': 'Học xong đói quá. Có đứa nào đi ăn với tao không?', 'label': 0},
    {'comment': 'Cần tìm người share phòng trọ khu vực Mỹ Đình, 1tr8/người. Yêu cầu sạch sẽ, có ý thức.', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang web cá độ, casino online. 🎰 Nạp 1 được 2. Hoàn trả không giới hạn. Ib để nhận link.', 'label': 1},
    {'comment': 'Đm cái thằng lồn nào lấy trộm đôi dép của tao ở trước cửa phòng. Bố mày mà biết thì mày chết mẹ.', 'label': 0},
    {'comment': 'NHẬN LÀM GIẤY KHÁM SỨC KHỎE, GIẤY NGHỈ ỐM HƯỞNG BHXH. 📄 Dấu đỏ bệnh viện, ship hỏa tốc. Ib Zalo để được tư vấn.', 'label': 1},
    {'comment': 'Mày là cái loại súc vật, bạn bè khó khăn đéo giúp được đồng nào còn quay ra cắn. Cút mẹ mày đi.', 'label': 0},
    {'comment': 'Link group Telegram tổng hợp clip drama, phốt của các hot girl. 🔞 Full không che. Link ở bio, free 50 slot đầu.', 'label': 1},
    {'comment': 'Học xong môn này tao thấy não mình như bị teo lại. Đéo hiểu giảng cái lồn gì.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 500k-1tr/ngày? 💸 Tham gia nhóm đầu tư tài chính 4.0. Chuyên gia đọc lệnh, cam kết lợi nhuận. Ib mình để vào nhóm.', 'label': 1},
    {'comment': 'Có ai có đề cương ôn thi môn Lịch sử Đảng không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'Dịch vụ đòi nợ thuê cho sinh viên. 👹 Lấy lại tiền từ những thằng bạn chó má. Nhanh gọn, kín đáo. Liên hệ Zalo...', 'label': 1},
    {'comment': 'Con đĩ đấy thảo mai vcl, trước mặt thì ngon ngọt sau lưng thì nói xấu như chó.', 'label': 0},
    {'comment': 'Tuyển 15 bạn nữ livestream app hẹn hò. Chỉ cần xinh, nói chuyện có duyên. Lương 15-30tr/tháng. Dr để phỏng vấn.', 'label': 1},
    {'comment': 'Cái thằng óc cặc đấy đá bóng ngu như lợn mà cứ thích thể hiện.', 'label': 0},
    {'comment': 'Đm lại hết tiền ăn. Kiếp sinh viên khổ vcl.', 'label': 0},
    {'comment': 'TOOL BUFF MẮT LIVESTREAM, TĂNG TƯƠNG TÁC. 🚀 Giúp bạn bán hàng nghìn đơn. Giá chỉ 199k/tháng. Ib để được dùng thử.', 'label': 1},
    {'comment': 'Có ai biết cách fix lỗi máy tính tự động restart không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản Canva Pro, Grammarly, Quillbot. ✍️ Đồng giá 30k/tháng. Hỗ trợ học tập, làm luận hiệu quả.', 'label': 1},
    {'comment': 'Mày là cái loại đéo có não à? Đăng cái status ngu lồn.', 'label': 0},
    {'comment': 'Thi lại môn Triết học. Cay đéo tả được.', 'label': 0},
    {'comment': 'Pass lại vé xem show của Lệ Quyên. 🎤 Do mình có việc bận. Vị trí đẹp, giá gốc.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn này hãm lồn vcl, toàn trù sinh viên.', 'label': 0},
    {'comment': 'YẾU SINH LÝ, CHƯA ĐẾN CHỢ ĐÃ HẾT TIỀN? 💊 Kẹo ngậm thảo dược, kéo dài cuộc yêu. Kín đáo, hiệu quả. Dr ngay.', 'label': 1},
    {'comment': 'Có đứa nào biết info con bé xinh vãi lồn khoa Marketing mới vào không?', 'label': 0},
    {'comment': 'Bạn đang cần tiền gấp? 💰 VAY TIỀN ONLINE không cần gặp mặt. Chỉ cần ảnh CMND. Duyệt hồ sơ trong 5 phút.', 'label': 1},
    {'comment': 'Đứa nào có link torrent bộ phim The Boys mùa mới không cho tao xin.', 'label': 0},
    {'comment': 'Cái loại bạn như mày, sống chỉ thêm chật đất. Cút.', 'label': 0},
    {'comment': 'Nhận cày thuê rank LMHT, Valorant. 🎮 Kéo rank thần tốc, giá sinh viên. Uy tín, bảo mật acc.', 'label': 1},
    {'comment': 'Học xong đói vcl. Có đứa nào đi ăn đêm với tao không?', 'label': 0},
    {'comment': 'Con ml đấy vừa xấu vừa vô duyên mà cũng có người yêu. Đúng là nồi nào úp vung nấy.', 'label': 0},
    {'comment': 'Deadline dí đến đít rồi mà máy tính hỏng. Đen như con chó mực.', 'label': 0},
    {'comment': 'Đm cái thằng chó nào ném rác qua ban công phòng tao. Bố mày mà bắt được thì mày xác định.', 'label': 0},
    {'comment': '🔥 TOOL HACK TÀI XỈU, BACCARAT ONLINE 🔥 Tỷ lệ thắng 98%. Giúp anh em về bờ an toàn. Ib để được cài đặt và hướng dẫn.', 'label': 1},
    {'comment': 'Mày là cái loại bạn bè đéo gì thế? Thấy tao có người yêu là quay ra ghen ăn tức ở à, con đĩ.', 'label': 0},
    {'comment': 'Tuyển nhân viên nhập mã captcha tại nhà. 💻 Việc nhẹ lương cao, 200-300k/ngày. Rút tiền về tài khoản ngân hàng. Ib để nhận việc.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán bánh mì dân tổ ngon không? Sáng mai tao phải đi học sớm.', 'label': 0},
    {'comment': 'Nhận làm bằng tốt nghiệp, học bạ, chứng chỉ Tiếng Anh. 📜 Phôi thật 100%, bao công chứng. Kín đáo, uy tín. Dr Zalo...', 'label': 1},
    {'comment': 'Cái thằng lồn dạy môn thể dục, vừa dê vừa hãm, toàn lựa mấy em xinh để nắn bóp.', 'label': 0},
    {'comment': 'Bán tài khoản VIP các app hẹn hò Tinder, Bumble. 💘 Tăng match, auto swipe. Giúp ae FA tìm gấu. Giá chỉ 49k/tháng.', 'label': 1},
    {'comment': 'Mày nói chuyện nghe như một thằng bại não vậy. Cút mẹ mày đi cho trong lành.', 'label': 0},
    {'comment': 'Thi xong môn cuối rồi chúng mày ơi. Xõa thôi. 🎉', 'label': 0},
    {'comment': 'Bạn muốn kiếm tiền thụ động? 💸 Tham gia hệ thống affiliate marketing của chúng tôi. Hoa hồng lên đến 50%. Link đăng ký ở bio.', 'label': 1},
    {'comment': 'Con ml đấy sống lỗi vcl, vay tiền cả phòng xong bùng mẹ luôn.', 'label': 0},
    {'comment': 'Học xong ngành này ra trường đéo biết làm cái lồn gì. Chán vcl.', 'label': 0},
    {'comment': 'Pass lại vé xe về quê ăn Tết. 🚌 Giường nằm Limousine. Do mình được nghỉ sớm hơn. Giá gốc.', 'label': 1},
    {'comment': 'Có ai có link tải game God of War crack không cho tao xin với. 🙏', 'label': 0},
    {'comment': 'Cái loại rẻ rách như mày thì chỉ xứng đáng dùng hàng fake thôi.', 'label': 0},
    {'comment': 'Tuyển sugar baby, sinh viên năm 1, 2. Ngoan, biết nghe lời. Daddy chu cấp 20 củ/tháng. Dr info + ảnh kín đáo.', 'label': 1},
    {'comment': 'Đm lại phải đóng tiền học phí rồi. Xót cả ruột.', 'label': 0},
    {'comment': 'Mày là cái thằng óc chó à? Bài dễ thế mà cũng đéo làm được.', 'label': 0},
    {'comment': 'Nhận thi hộ, điểm danh hộ tất cả các môn. ✍️ Đội ngũ SV giỏi, cam kết qua môn. Bảo mật thông tin tuyệt đối.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn C++ giảng như cái cặc. Đéo hiểu con mẹ gì cả.', 'label': 0},
    {'comment': 'Bán acc Tiktok 100k follow. 🚀 Tương tác thật, phù hợp cho ae bán hàng online. Giao dịch trung gian.', 'label': 1},
    {'comment': 'Deadline ngập đến cổ rồi. Cứu tao với chúng mày ơi.', 'label': 0},
    {'comment': 'Con đĩ đấy suốt ngày lên mạng sống ảo, chứ ngoài đời thì xấu như ma.', 'label': 0},
    {'comment': 'Bạn đang bị nám, tàn nhang? ✨ Kem trị nám gia truyền, hiệu quả sau 1 liệu trình. Cam kết hoàn tiền nếu không đỡ.', 'label': 1},
    {'comment': 'Có đứa nào đi xem giải bóng đá sinh viên chiều nay không?', 'label': 0},
    {'comment': 'Mày là cái loại súc vật, đéo có đạo đức à?', 'label': 0},
    {'comment': 'Nhận order quần áo, giày dép Quảng Châu. 👕 Hàng về sau 5-7 ngày. Công mua hộ rẻ nhất Vịnh Bắc Bộ.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng ở bẩn như một con lợn. Quần sịp vứt lung tung. Tởm vcl.', 'label': 0},
    {'comment': 'Mày bớt sủa đi được không? Nghe nhức cả đầu.', 'label': 0},
    {'comment': 'Bán tài khoản ChatGPT 4.0, Midjourney. 🤖 Hỗ trợ anh em sinh viên chạy deadline, làm đồ án. Giá chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Đm thằng nào đi xe bus không có ý thức, chen lấn xô đẩy như một con súc vật.', 'label': 0},
    {'comment': 'Cần tìm người share tiền thuê sân bóng chiều mai. Sân 7, 150k/giờ. Thiếu 3 người.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 1-2 củ/tuần? 💰 Tuyển CTV chốt đơn hàng thời trang. Không cần vốn, làm tại nhà. Ib để được training.', 'label': 1},
    {'comment': 'Con ml đấy được học bổng chắc lại đi ngủ với thầy giáo chứ học hành cc gì.', 'label': 0},
    {'comment': 'VAY TIỀN CHỈ CẦN THẺ SINH VIÊN. 🔥 Lãi suất thấp, giải ngân trong 5 phút. Không gọi điện cho người nhà. Bấm vào link ở bio.', 'label': 1},
    {'comment': 'Thằng lồn nào lấy trộm cái mũ bảo hiểm của tao ở nhà xe. Bố mày mà thấy thì mày ăn cặc.', 'label': 0},
    {'comment': 'Nhận làm chứng chỉ Tin học, Tiếng Anh quốc gia. 📜 Bao soi, bao công chứng. Uy tín, kín đáo. Dr Zalo để trao đổi.', 'label': 1},
    {'comment': 'Mày là cái loại bạn bè đéo gì mà thấy tao bị chửi đéo bênh được một câu? Hèn như chó.', 'label': 0},
    {'comment': 'Tuyển sugar baby ngoan, kín đáo. Daddy lo chu cấp full từ A-Z, 25 củ/tháng. Yêu cầu ngoại hình ưa nhìn. 🌹', 'label': 1},
    {'comment': 'Học xong môn này tao thấy mình đần độn hẳn ra. Đéo hiểu thầy giảng cái lồn gì.', 'label': 0},
    {'comment': 'Link group Zalo xem clip xxx của các em gái sinh viên. 🥵 Free slot cho 200 ae đầu tiên. Link ở bio.', 'label': 1},
    {'comment': 'Đm lại phải học lại. Tốn tiền của bố mẹ vcl.', 'label': 0},
    {'comment': 'Cái con đĩ đấy xấu như ma chê quỷ hờn mà suốt ngày đăng ảnh câu like.', 'label': 0},
    {'comment': 'Bán tài khoản Netflix Premium 4K. 🎬 Xem phim không giới hạn. Giá chỉ 40k/tháng. Bảo hành full time.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán phở cuốn ngon ở khu vực Cầu Giấy không?', 'label': 0},
    {'comment': 'Mày là cái thằng óc lợn à? Nói mãi mà đéo hiểu à?', 'label': 0},
    {'comment': 'TOOL AUTO LIKE, CMT, SHARE FACEBOOK. 🚀 Giúp bạn kéo tương tác, bán hàng hiệu quả. Giá sinh viên. Ib để dùng thử.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn này hãm vcl, suốt ngày đạo lý.', 'label': 0},
    {'comment': 'Nhận cày thuê Elo, rank game Tốc Chiến, Liên Quân. 🎮 Kéo rank thần tốc, giá rẻ. Uy tín, không phá acc.', 'label': 1},
    {'comment': 'Deadline dí đến đít rồi mà mất mẹ nó mạng. Cay vcl.', 'label': 0},
    {'comment': 'Mày câm mẹ cái mồm thối của mày vào đi.', 'label': 0},
    {'comment': 'Bạn đang bí ý tưởng làm bài tập lớn? 🔥 Bán 1000+ báo cáo, đồ án các ngành kinh tế, kỹ thuật. Giá chỉ từ 50k/bài.', 'label': 1},
    {'comment': 'Có đứa nào đi xem Ravolution Music Festival không? Lập team đi cho vui.', 'label': 0},
    {'comment': 'Cái loại rẻ rách như mày thì đéo có tư cách nói chuyện với tao.', 'label': 0},
    {'comment': 'Nhận order hàng US, UK. ✈️ Cam kết auth, full bill. Công mua hộ rẻ. Hàng về sau 2-3 tuần.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng ở bẩn như lợn, bát đũa ăn xong đéo rửa. Tởm vcl.', 'label': 0},
    {'comment': 'Có ai biết cách xin gia hạn học phí không ạ? Nhà em đang có việc.', 'label': 0},
    {'comment': 'Pass lại con acc Valorant nhiều skin súng. Rank Vàng. Do không có thời gian chơi. Giá 5 lít.', 'label': 1},
    {'comment': 'Học xong đói quá, có đứa nào đi ăn đêm không?', 'label': 0},
    {'comment': 'Ae cho hỏi build PC tầm 20 củ thì nên đi chip Intel hay AMD nhỉ? Nhu cầu vừa game vừa code.', 'label': 0},
    {'comment': '🔥 BÁN DATA 5000 SINH VIÊN CÁC TRƯỜNG TPHCM 🔥 Full info (SĐT, email, trường, ngành). Phù hợp cho ae chạy sale, marketing. Ib kín.', 'label': 1},
    {'comment': 'Tối nay MU đá, thằng nào fan MC thì câm mẹ mồm vào nhé.', 'label': 0},
    {'comment': 'Nhận unlock iCloud, bypass iPhone, iPad. 📱 Nhanh chóng, giá rẻ. Hỗ trợ mọi đời máy. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Con laptop của tao vừa hết bảo hành thì lăn ra hỏng main. Đm đúng là số chó.', 'label': 0},
    {'comment': 'Tuyển 30 bạn làm việc tại nhà. Chỉ cần copy-paste, thu nhập 2-4tr/tháng. 💻 Không yêu cầu kinh nghiệm. Ib mình để nhận JD.', 'label': 1},
    {'comment': 'Mọi người ơi, thủ tục xin cấp lại thẻ sinh viên bị mất có phức tạp không ạ?', 'label': 0},
    {'comment': 'Bàn phím cơ switch brown gõ sướng thật sự. Êm hơn hẳn con blue cũ của tao.', 'label': 0},
    {'comment': 'Link group kín Zalo chia sẻ clip 2k4, 2k5. 🥵 Full HD không che. Vào nhanh trước khi full slot. Link ở bio.', 'label': 1},
    {'comment': 'Mày là cái loại bạn bè đéo gì mà thấy tao bị bắt nạt thì trốn như chó?', 'label': 0},
    {'comment': 'Học ngành IT năm nhất thì nên mua laptop nào tầm 15 củ vậy mng?', 'label': 0},
    {'comment': 'Bạn muốn kiếm 500k/ngày? 💸 Tham gia hệ thống TÀI XỈU của chúng tôi. Kéo 1-1, cam kết thắng lớn. Add Zalo để vào nhóm.', 'label': 1},
    {'comment': 'Thằng lồn đấy đá bóng ngu như lợn mà cứ thích cầm bóng đột phá.', 'label': 0},
    {'comment': 'Pass lại con chuột Logitech G Pro X Superlight. Hơi cũ, bị double click. Giá 3 lít cho ae về sửa.', 'label': 0},
    {'comment': 'Dịch vụ viết thuê, chạy deadline các môn đại cương. ✍️ Nhanh, rẻ, uy tín. Cứu ae qua mùa thi cử.', 'label': 1},
    {'comment': 'Idol của chúng mày đấy à? Nhìn như thằng nghiện. 😂', 'label': 0},
    {'comment': 'Có ai biết chỗ nào vệ sinh laptop, tra keo tản nhiệt uy tín ở Hà Nội không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang phim người lớn. 🔞 Full thể loại, không quảng cáo. Kín đáo, an toàn.', 'label': 1},
    {'comment': 'Đm cái thằng bạn cùng phòng, đêm thì ngáy như sấm, sáng thì đéo dậy tắt báo thức.', 'label': 0},
    {'comment': 'Có ai biết cách cài Win 11 không bị lỗi không?', 'label': 0},
    {'comment': 'Nhận làm bằng tốt nghiệp, chứng chỉ IELTS 7.0+. 📜 Phôi thật, bao soi. Giúp bạn có CV đẹp. Ib để trao đổi.', 'label': 1},
    {'comment': 'Mọi người ơi, học bổng khuyến khích học tập đã có kết quả chưa ạ?', 'label': 0},
    {'comment': 'Cái loại rẻ rách, mở mồm ra là điêu.', 'label': 0},
    {'comment': 'VAY TIỀN ONLINE - CHỈ CẦN CMND. 💰 Không thẩm định, không gọi người thân. Duyệt vay trong 3 phút. Tải app tại...', 'label': 1},
    {'comment': 'Em là nữ, đang phân vân giữa Dell XPS và Macbook Air M1. Nhu cầu văn phòng, giải trí nhẹ nhàng. Nhờ mn tư vấn.', 'label': 0},
    {'comment': 'Thằng chó đấy vay tiền tao mua bàn phím cơ xong bùng mẹ luôn. Cay.', 'label': 0},
    {'comment': 'Tuyển sugar baby, chu cấp 15 củ/tháng. Yêu cầu ngoan, biết chiều. Daddy ở Hà Nội. Dr info kín.', 'label': 1},
    {'comment': 'Tối nay có ai xem C1 không? Real Madrid chấp mấy quả nhỉ?', 'label': 0},
    {'comment': 'Bạn cùng nhóm làm ăn như cái lồn, deadline đến đít rồi mà slide còn chưa xong.', 'label': 0},
    {'comment': 'Bán acc Steam full game AAA. 🎮 GTA V, Cyberpunk, Elden Ring... Giá chỉ 500k. Giao dịch qua trung gian.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán ram laptop cũ giá rẻ không?', 'label': 0},
    {'comment': 'Con đĩ đấy cặp với cả khoa mà cứ tỏ ra mình là gái nhà lành. Tởm vcl.', 'label': 0},
    {'comment': 'Nhận order phụ kiện PC (VGA, RAM, SSD) từ Amazon, Newegg. ✈️ Hàng về nhanh, giá tốt hơn trong nước.', 'label': 1},
    {'comment': 'Đm mạng wifi ktx như cc, xem youtube 360p còn giật.', 'label': 0},
    {'comment': 'Mọi người cho em hỏi học ngành an toàn thông tin ra trường có dễ xin việc không ạ?', 'label': 0},
    {'comment': 'Nhà bạn chắc không có gì ngoài điều kiện nhỉ? Thấy đổi điện thoại liên tục.', 'label': 0},
    {'comment': 'Bán tài khoản Office 365, Google Drive không giới hạn dung lượng. 💾 Lưu trữ tài liệu an toàn. Giá 100k/năm.', 'label': 1},
    {'comment': 'Thua độ trận bóng tối qua, cay như dái.', 'label': 0},
    {'comment': 'Mày là cái thằng óc lợn à? Code sai logic mà cứ thích cãi.', 'label': 0},
    {'comment': 'Có ai biết cách cài dual boot Windows và Ubuntu không bị mất dữ liệu không?', 'label': 0},
    {'comment': 'Bạn muốn x5, x10 tài khoản? 🚀 Tham gia nhóm tín hiệu coin của chúng tôi. Hoàn toàn miễn phí. Link Zalo ở bio.', 'label': 1},
    {'comment': 'Chạy xong cái đồ án này chắc tao đi tu.', 'label': 0},
    {'comment': 'Cần tìm người share acc ChatGPT Plus dùng chung.', 'label': 0},
    {'comment': 'Tuyển CTV bán sim số đẹp, thẻ cào chiết khấu cao. 📱 Không cần vốn. Phù hợp cho SV kiếm thêm.', 'label': 1},
    {'comment': 'Có ai biết cách root điện thoại Android an toàn không ạ?', 'label': 0},
    {'comment': 'Mày bớt sủa lại đi, nghe nhức cả đầu.', 'label': 0},
    {'comment': 'Đội bóng trường mình đá giải sinh viên toàn quốc có vào sâu không mng?', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Facebook, Google cho các shop. 🔥 Cam kết ra đơn, target đúng đối tượng. Ib để được tư vấn miễn phí.', 'label': 1},
    {'comment': 'Cái loại bạn như mày, sống chỉ tổ chật đất.', 'label': 0},
    {'comment': 'Em là sinh viên năm nhất, có nên đi làm thêm ngay không hay nên tập trung học ạ?', 'label': 0},
    {'comment': 'Bán tài khoản Adobe full app bản quyền. 🎨 Dùng cho cả Windows và Mac. Giá chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Thằng lồn đấy hack game mà cứ làm như mình pro lắm.', 'label': 0},
    {'comment': 'Có ai biết cách kết nối tay cầm PS4 với PC không?', 'label': 0},
    {'comment': 'Pass lại con màn hình 27 inch 2k 144hz. Còn bảo hành. Mua về mà card không cân nổi. 😭', 'label': 0},
    {'comment': 'Nhận cài win, vệ sinh laptop tại nhà. 💻 Sạch sẽ, nhanh chóng, giá sinh viên. Có mặt sau 30 phút.', 'label': 1},
    {'comment': 'Chia tay người yêu xong cảm thấy trống rỗng vcl.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán card màn hình cũ uy tín không?', 'label': 0},
    {'comment': 'TOOL AUTO FARM GAME, TREO ACC 24/7. 🤖 Hỗ trợ mọi game mobile. An toàn, không khóa acc. Ib để nhận link tải.', 'label': 1},
    {'comment': 'Đm cô giáo dạy môn này nói giọng khó nghe vcl, chả hiểu cc gì.', 'label': 0},
    {'comment': 'Cần tìm người share acc Netflix Premium, 40k/slot xem 4K. Yêu cầu xem đúng profile.', 'label': 0},
    {'comment': '🔥 VAY TIỀN ONLINE - DUYỆT TỰ ĐỘNG SAU 2 PHÚT 🔥 Không cần gặp mặt, không gọi người thân. Chỉ cần CMND. Link vay ở bio.', 'label': 1},
    {'comment': 'Tối nay Liverpool đá với Chelsea, anh em bắt kèo nào?', 'label': 0},
    {'comment': 'Nhận cày thuê rank, kéo rank game Valorant, CSGO. 🎮 Kéo lên rank mong muốn chỉ trong 1 đêm. Uy tín, bảo mật.', 'label': 1},
    {'comment': 'Con laptop gaming của tao nóng như cái lò than. Chơi game tí mà sụt FPS vcl.', 'label': 0},
    {'comment': 'Bạn muốn kiếm thêm thu nhập? 💸 Tuyển CTV bán vé máy bay, tour du lịch. Chiết khấu cao. Làm việc online.', 'label': 1},
    {'comment': 'Mọi người ơi, môn Vật lý 2 có khó hơn Vật lý 1 nhiều không ạ?', 'label': 0},
    {'comment': 'Bàn phím cơ Quang học gõ có khác biệt nhiều so với phím cơ thường không mng?', 'label': 0},
    {'comment': 'Link group Zalo tổng hợp clip xxx của các hot tiktoker. 🥵 Full không che. Vào nhanh trước khi admin đóng link.', 'label': 1},
    {'comment': 'Thằng lồn đấy vay tiền tao mua gear xong đéo trả. Đúng là loại chó má.', 'label': 0},
    {'comment': 'Học ngành AI ra trường lương có cao không các anh chị đi trước?', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang bóng đá (Xoilac, Cakhia...). ⚽️ Xem full HD không quảng cáo, không giật lag. 20k/tháng.', 'label': 1},
    {'comment': 'Thằng trọng tài bắt trận hôm qua như cái lồn, thiên vị vcl.', 'label': 0},
    {'comment': 'Pass lại con main B550M còn bảo hành dài. Do nâng cấp lên X570. Giá 1 củ 2.', 'label': 0},
    {'comment': 'Nhận cài đặt phần mềm đồ họa (PTS, AI, Premiere) full crack. 🎨 Nhanh chóng, giá sinh viên. Ib mình nhé.', 'label': 1},
    {'comment': 'Flex nhẹ con PC mới build. Trắng tinh khôi. 🥰', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán lót chuột (mousepad) custom hình ảnh không ạ?', 'label': 0},
    {'comment': 'Tuyển CTV viết bài chuẩn SEO cho website công nghệ. Nhuận bút 60-80k/bài. Yêu cầu có kiến thức về PC, laptop.', 'label': 1},
    {'comment': 'Đm cái thằng bạn cùng phòng, dùng chung con lap của tao mà cài cắm ba cái phần mềm rác vào.', 'label': 0},
    {'comment': 'Có ai biết cách tối ưu Windows 10 cho máy cấu hình yếu không?', 'label': 0},
    {'comment': 'Bạn muốn trở nên giàu có? 📈 Tham gia khóa học đầu tư BĐS của chúng tôi. Bí quyết X10 tài sản sau 1 năm. Đăng ký ngay!', 'label': 1},
    {'comment': 'Mọi người ơi, lịch thi lại đã có trên trang tín chỉ chưa?', 'label': 0},
    {'comment': 'Mày là cái loại đéo có não à? Code bug lòi ra mà cứ thích cãi.', 'label': 0},
    {'comment': 'Dịch vụ làm giấy khám sức khỏe để xin việc, đi học. 📄 Dấu đỏ bệnh viện, lấy ngay sau 30 phút. Ship tận nơi.', 'label': 1},
    {'comment': 'Em là nữ, đang học marketing thì nên mua laptop nào mỏng nhẹ, pin trâu ạ?', 'label': 0},
    {'comment': 'Thằng chó đấy hack game rõ rành rành ra mà vẫn già mồm. Đúng là loại đéo có liêm sỉ.', 'label': 0},
    {'comment': 'Bán tài khoản Steam đã mua sẵn game Elden Ring. 🎮 Rẻ hơn mua trực tiếp. Giao dịch qua admin.', 'label': 1},
    {'comment': 'Tối nay có ai đi xem Man City đá không?', 'label': 0},
    {'comment': 'Bạn cùng nhóm thuyết trình như đọc rap, đéo nghe được cc gì.', 'label': 0},
    {'comment': 'NHẬN LÀM BẰNG ĐẠI HỌC, CAO ĐẲNG. 📜 Phôi thật, tem 7 màu, bao công chứng. Kín đáo, uy tín. Ib Zalo để được tư vấn.', 'label': 1},
    {'comment': 'Có ai biết chỗ nào bán SSD NVMe cũ giá tốt không?', 'label': 0},
    {'comment': 'Con đĩ đấy suốt ngày đi chê bai người khác trong khi bản thân thì như cái lồn.', 'label': 0},
    {'comment': 'Pass lại con VGA RX 580 8GB còn dùng tốt. Cân mượt các game e-sport. Giá 1 củ.', 'label': 1},
    {'comment': 'Đm mạng Viettel mấy hôm nay lag vcl, chơi game ping toàn 200.', 'label': 0},
    {'comment': 'Mọi người cho em hỏi học đồ họa thì nên dùng màn hình 2K hay 4K ạ?', 'label': 0},
    {'comment': 'Nhìn cái dàn PC của nó mà thèm vcl.', 'label': 0},
    {'comment': 'YẾU SINH LÝ, TRÊN BẢO DƯỚI KHÔNG NGHE? 💊 Kẹo ngậm thảo dược, hiệu quả sau 30 phút. Dr để được tư vấn kín đáo.', 'label': 1},
    {'comment': 'Thằng Ronaldo hết thời rồi, về dưỡng già thôi.', 'label': 0},
    {'comment': 'Mày là cái thằng óc chó, bảo vệ quan điểm ngu lồn của mày làm cc gì?', 'label': 0},
    {'comment': 'Có ai biết cách overclock CPU an toàn không?', 'label': 0},
    {'comment': 'Bạn muốn kiếm 2-3 củ/tháng? 💰 Tuyển CTV bán sim số đẹp, data 4G. Không vốn, không cọc. Làm online.', 'label': 1},
    {'comment': 'Làm xong cái đồ án này chắc tao phải đi đo lại mắt. Mỏi vcl.', 'label': 0},
    {'comment': 'Cần tìm người đi đá bóng phủi. Team mình đang thiếu thủ môn.', 'label': 0},
    {'comment': 'TOOL HACK GAME TÀI XỈU, XÓC ĐĨA. 🔥 Tỉ lệ thắng 95%. Hoàn vốn sau 1 ngày. Ib để được cài đặt.', 'label': 1},
    {'comment': 'Có ai biết cách build một con PC gaming giá rẻ tầm 10 triệu không?', 'label': 0},
    {'comment': 'Mày bớt sủa đi được không? Nghe điếc cả tai.', 'label': 0},
    {'comment': 'CLB Lập trình của trường có hay tổ chức workshop không mng?', 'label': 0},
    {'comment': 'Nhận order bàn phím cơ custom. ⌨️ Layout, switch, keycap theo yêu cầu. Giá cả phải chăng. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Cái loại bạn như mày, sống chỉ tổ làm khổ người khác.', 'label': 0},
    {'comment': 'Em là sinh viên năm nhất, có thắc mắc về các môn đại cương có thể hỏi ở đây không ạ?', 'label': 0},
    {'comment': 'Bán tài khoản Adobe, Office 365, Canva Pro. ✍️ Hỗ trợ học tập, làm việc hiệu quả. Giá chỉ từ 30k.', 'label': 1},
    {'comment': 'Thằng lồn đấy chơi game bẩn vcl, toàn đi camp.', 'label': 0},
    {'comment': 'Có ai biết cách undervolt GPU để giảm nhiệt độ không?', 'label': 0},
    {'comment': 'Pass lại con laptop gaming do cần tiền đóng học phí. 😭 Máy còn mới, full box. Cấu hình i5 11400H, RTX 3050.', 'label': 0},
    {'comment': 'Dịch vụ viết CV, Cover Letter chuyên nghiệp. ✨ Giúp bạn ghi điểm với nhà tuyển dụng. Chỉ 100k/bộ.', 'label': 1},
    {'comment': 'Thất tình đau vcl chúng mày ạ.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán tản nhiệt nước AIO uy tín không?', 'label': 0},
    {'comment': 'LINK GROUP TELEGRAM CHIA SẺ CLIP NÓNG CỦA IDOL. 🔞 Link ở bio. Vào nhanh không xóa.', 'label': 1},
    {'comment': 'Đm cô giáo dạy môn này nói nhanh như bắn rap, đéo chép kịp chữ nào.', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang stock (Freepik, Envato). 🎨 Hỗ trợ ae designer chạy deadline. Giá chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Đm cái thằng bạn cùng phòng, dùng máy tính của tao cày game làm nóng như cái lò sưởi. Sợ hỏng mẹ con card.', 'label': 0},
    {'comment': 'Mọi người cho em hỏi, main B460 có chạy được RAM bus 3200 không ạ, hay nó chỉ nhận 2933 thôi?', 'label': 0},
    {'comment': 'Tuyển 5 bạn nữ xinh, dáng đẹp làm mẫu ảnh cho local brand. 👗 Lương 500k-1tr/buổi. Dr để gửi profile.', 'label': 1},
    {'comment': 'Thằng lồn nào đá bóng mà sút vào người tao. Bố mày mà cáu thì mày ăn đấm.', 'label': 0},
    {'comment': 'LINK TELEGRAM TỔNG HỢP PHỐT, DRAMA SHOWBIZ. 🤫 Hóng biến cực căng. Vào nhanh trước khi bị xóa.', 'label': 1},
    {'comment': 'Mày là cái loại óc chó à? Code thì bug tùm lum mà cứ thích ra vẻ pro.', 'label': 0},
    {'comment': 'Bán tài khoản Github Student Pack full quyền lợi. 🚀 Dành cho ae IT. Giá chỉ 100k, dùng đến lúc ra trường.', 'label': 1},
    {'comment': 'Có ai biết cách undervolt CPU laptop để máy mát hơn không ạ?', 'label': 0},
    {'comment': 'Bàn phím cơ mới mua gõ sướng vcl chúng mày ạ. Lún lún phê phê.', 'label': 0},
    {'comment': 'YẾU SINH LÝ, VỢ CHÊ? 💊 Thuốc cường dương thảo dược, an toàn, hiệu quả. Giúp bạn lấy lại bản lĩnh. Kín đáo, dr ngay.', 'label': 1},
    {'comment': 'Con đĩ đấy suốt ngày đi nói xấu người yêu cũ. Đéo hiểu sao có người rước được nó.', 'label': 0},
    {'comment': 'Em là tân sinh viên, đang phân vân giữa việc ở KTX và thuê trọ. Mọi người cho em xin lời khuyên với.', 'label': 0},
    {'comment': 'Dịch vụ tăng điểm Google Maps, review 5 sao. ⭐ Giúp quán của bạn lên top tìm kiếm. Giá sinh viên.', 'label': 1},
    {'comment': 'Thằng Haaland trận hôm qua đá như thằng vô hồn. Chán vcl.', 'label': 0},
    {'comment': 'Cần tìm người ở ghép, phòng 2tr/người gần ĐH Công Nghiệp. Yêu cầu sạch sẽ, không ồn ào.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 3-5 củ/tháng? 💰 Tuyển CTV bán quần áo order. Không vốn, không ship hàng. Chỉ cần đăng bài. Ib mình training.', 'label': 1},
    {'comment': 'Đm lại học lại môn Cấu trúc dữ liệu. Ngu lồn thật.', 'label': 0},
    {'comment': 'Có ai biết chỗ nào bán keycap đẹp ở Hà Nội không?', 'label': 0},
    {'comment': 'TOOL HACK GAME BÀI ĐỔI THƯỞNG. 🔥 Biết trước kết quả, auto đặt cược. Giúp ae về bờ an toàn. Ib để cài đặt.', 'label': 1},
    {'comment': 'Mày là cái thằng đần, nói chuyện với mày tốn mẹ cả calo.', 'label': 0},
    {'comment': 'Nhận cài Win, Mac OS, vệ sinh laptop tại nhà. 💻 Lấy ngay, giá rẻ. Có mặt sau 30 phút khu vực Cầu Giấy.', 'label': 1},
    {'comment': 'Vừa chốt xong con màn 27 inch 2k 165hz. Phê vcl anh em ạ. 🎉', 'label': 0},
    {'comment': 'Con ml đấy suốt ngày khịa tao. Đéo hiểu tao làm cc gì nó.', 'label': 0},
    {'comment': 'Pass lại vé xem Westlife. 🎤 Do mình bận việc đột xuất. Vị trí đẹp, giá gốc.', 'label': 1},
    {'comment': 'Có ai biết cách build PC trong tầm giá 12 triệu để chơi game e-sport không ạ?', 'label': 0},
    {'comment': 'Mày bớt sủa lại đi, nghe chói tai vcl.', 'label': 0},
    {'comment': 'Nhận làm bằng tốt nghiệp, bằng lái xe, CCCD. 📜 Phôi thật, bao soi. Kín đáo, giao dịch trực tiếp.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng luộm thuộm vcl, quần áo vứt lung tung như cái ổ lợn.', 'label': 0},
    {'comment': 'Mọi người cho hỏi, nên mua SSD Sata 3 hay cố thêm tí lên NVME ạ? Tốc độ có khác biệt nhiều không?', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang báo nước ngoài (NYT, WSJ). 📰 Nâng cao trình độ tiếng Anh, kiến thức. Chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Tối nay Arsenal đá, fan Tottenham chắc lại trốn hết rồi. 😂', 'label': 0},
    {'comment': 'Đm thằng nào đỗ xe ô tô chắn hết mẹ cổng trường.', 'label': 0},
    {'comment': 'Có ai có link download bộ cài Davinci Resolve Studio không cho em xin với.', 'label': 0},
    {'comment': 'Dịch vụ đòi nợ thuê. 👹 Đòi lại tiền từ những thằng bạn chó má. Nhanh gọn, hiệu quả. Dr Zalo...', 'label': 1},
    {'comment': 'Mày là cái loại đéo có lòng tự trọng à?', 'label': 0},
    {'comment': 'Chia tay người yêu xong nên làm gì cho đỡ buồn đây chúng mày?', 'label': 0},
    {'comment': 'Tuyển 10 bạn làm Airdrop, Bounty Crypto. 🚀 Không cần vốn, cơ hội x100 tài sản. Ib để vào nhóm hướng dẫn.', 'label': 1},
    {'comment': 'Vừa xem xong trận bóng đá nữ, mấy chị đá hay vcl.', 'label': 0},
    {'comment': 'Cái loại rẻ rách, sống lỗi như mày thì đéo có ai chơi cùng đâu.', 'label': 0},
    {'comment': 'Có nên mua laptop gaming cũ không mọi người, hay cố thêm mua máy mới?', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang học code (Leetcode, Codecademy). 👨‍💻 Hỗ trợ ae IT nâng cao trình độ. Giá sinh viên.', 'label': 1},
    {'comment': 'Làm xong cái đồ án này chắc tao tổn thọ mất. Stress vcl.', 'label': 0},
    {'comment': 'Thằng Messi đá ở Inter Miami vẫn gánh team như thường. Đẳng cấp vcl.', 'label': 0},
    {'comment': 'Nhận order phụ kiện bàn phím cơ (keycap, switch, stab). ⌨️ Hàng nội địa Trung, giá tốt. Dr để xem mẫu.', 'label': 1},
    {'comment': 'Có ai biết cách fix lỗi "Your connection is not private" trên Chrome không?', 'label': 0},
    {'comment': 'Mày là thằng óc chó à? Đã ngu còn hay cãi lý.', 'label': 0},
    {'comment': 'Học xong ngành này thất nghiệp thì nhục. 🥲', 'label': 0},
    {'comment': 'Bán tài khoản Midjourney, Stable Diffusion. 🎨 Giúp bạn tạo ảnh AI không giới hạn. Chỉ 100k/tháng.', 'label': 1},
    {'comment': 'Cái loại bạn như mày, tao coi như con chó ven đường.', 'label': 0},
    {'comment': 'Mọi người ơi, môn Giải tích 2 có dùng lại kiến thức Giải tích 1 nhiều không?', 'label': 0},
    {'comment': 'VAY TIỀN NÓNG TRONG NGÀY. 💰 Chỉ cần CMND + Thẻ SV. Không gọi người thân. Ib mình để được tư vấn.', 'label': 1},
    {'comment': 'Con đĩ đấy suốt ngày lên mạng khoe tiền, đéo biết tiền của nó hay của sugar daddy nó.', 'label': 0},
    {'comment': 'Có ai biết quán cafe nào yên tĩnh, wifi mạnh để ngồi code không ạ?', 'label': 0},
    {'comment': 'Pass lại con VGA 1050ti huyền thoại. 5 lít cho anh em chống cháy.', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Google Ads, SEO website lên top. 🔥 Giúp bạn tiếp cận hàng nghìn khách hàng. Chi phí hợp lý.', 'label': 1},
    {'comment': 'Thằng lồn đấy hack game rõ rành rành mà admin đéo ban.', 'label': 0},
    {'comment': 'Có nên mua bàn phím không dây để chơi game không mng, sợ có độ trễ.', 'label': 0},
    {'comment': 'LINK GROUP TELEGRAM GIAO LƯU SUGAR BABY - DADDY. 🌹 Kín đáo, an toàn. Link ở bio.', 'label': 1},
    {'comment': 'Đm con người yêu cũ, đòi chia tay xong giờ lại quay lại làm phiền tao.', 'label': 0},
    {'comment': 'Tầm 15 củ build PC gaming thì nên đi R5 5600 + 2060 Super cũ không ae? Hay có combo nào ngon hơn?', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang web soi kèo, tip bóng đá. ⚽️ Tỉ lệ thắng > 80%. Giúp anh em về bờ. Ib để vào nhóm kín.', 'label': 1},
    {'comment': 'Đm cái thằng lồn nào lấy xe của tao mà đéo đổ xăng. Hết sạch mẹ xăng giữa đường.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page, trả lời tin nhắn cho shop thời trang. 💬 Làm xoay ca, 25k/h. Ib để ứng tuyển.', 'label': 1},
    {'comment': 'Mọi người ơi, môn Vi xử lý có khó không ạ? Em nghe nói môn này code chua lắm.', 'label': 0},
    {'comment': 'Dịch vụ làm bằng lái xe, cavet xe, CCCD. 📜 Phôi thật, bao soi. Giao dịch trực tiếp, an toàn. Dr Zalo...', 'label': 1},
    {'comment': 'Con đĩ đấy suốt ngày lên mạng dạy đời trong khi sống như cái lồn.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang xem phim người lớn (Javhd, Xvideos...). 🔞 Full HD, không quảng cáo. Ib kín đáo.', 'label': 1},
    {'comment': 'Mày là cái loại bạn chó, thấy người sang bắt quàng làm họ. Cút.', 'label': 0},
    {'comment': 'Thằng De Gea rời MU là đúng rồi, ở lại chỉ làm trò hề.', 'label': 0},
    {'comment': 'Bạn muốn kiếm 500k-1tr/ngày? 💸 Tham gia sàn giao dịch BO, chuyên gia đọc lệnh 1-1. Cam kết không cháy tài khoản. Link đăng ký ở bio.', 'label': 1},
    {'comment': 'Cái thằng óc chó đấy chơi game ngu mà cứ thích chỉ đạo. Mệt vcl.', 'label': 0},
    {'comment': 'Pass lại con màn hình LG 24 inch full HD 75hz. Còn bảo hành. Giá 1 củ 5.', 'label': 0},
    {'comment': 'Nhận cài lại Win, macOS, Linux. 💻 Cài full driver, phần mềm cơ bản. Giá chỉ 80k. Ib mình nhé.', 'label': 1},
    {'comment': 'Có ai biết cách fix lỗi màn hình xanh trên Windows 11 không?', 'label': 0},
    {'comment': 'Cái loại rẻ rách như mày, đéo ai thèm chơi cùng đâu.', 'label': 0},
    {'comment': 'Tuyển sugar baby, yêu cầu sv năm nhất, ngoan. Daddy chu cấp 20 củ/tháng, lo từ A-Z. Dr info + ảnh. 🌹', 'label': 1},
    {'comment': 'Đm thằng nào park con xe Vinfast trước cổng KTX, chắn hết mẹ đường đi.', 'label': 0},
    {'comment': 'Mày là cái thằng đần, não phẳng à? Nói thế mà cũng đéo hiểu.', 'label': 0},
    {'comment': 'Nhận order hàng Nhật (figure, manga, quần áo). 🇯🇵 Hàng về sau 2 tuần. Công mua hộ rẻ. Dr để được tư vấn.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn giải tích này giảng như ru ngủ. Chán đéo buồn học.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Tinder Gold, Bumble Premium. 🔥 Giúp bạn tìm gấu qua mùa đông. Giá sinh viên. Ib chốt đơn.', 'label': 1},
    {'comment': 'Deadline dí đến ass rồi mà tao vẫn ngồi đây lướt face. Đúng là vua lười.', 'label': 0},
    {'comment': 'Con ml đấy sống ảo vcl, ảnh trên mạng thì xinh như hot girl, ngoài đời nhìn như con nghiện.', 'label': 0},
    {'comment': 'Bạn đang gặp khó khăn tài chính? 💰 Vay nóng sinh viên, không cần gặp mặt. Lãi suất thấp. Giải ngân trong 5 phút.', 'label': 1},
    {'comment': 'Có đứa nào đi xem Blackpink ở Mỹ Đình không? Cho tao xin ít review.', 'label': 0},
    {'comment': 'Mày là cái loại súc vật, sống lỗi đéo chịu được.', 'label': 0},
    {'comment': 'Nhận cày thuê, làm nhiệm vụ game Genshin Impact, Honkai Star Rail. 🎮 Uy tín, nhanh gọn. Ib để báo giá.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng ở bẩn như một con heo, rác vứt đầy phòng. Tởm vcl.', 'label': 0},
    {'comment': 'Có ai biết cách build con PC mini ITX không? Em thích nhỏ gọn.', 'label': 0},
    {'comment': 'Bán acc Facebook cổ, acc via để chạy quảng cáo. 🚀 Bao trust, ít bị checkpoint. Ib để lấy info.', 'label': 1},
    {'comment': 'Tối nay Real đá với Barca, El Clasico đỉnh cao. Anh em dự đoán tỉ số đi.', 'label': 0},
    {'comment': 'Đm cái thằng lồn nào ăn trộm con chuột G102 của tao. Bố mày nguyền rủa mày.', 'label': 0},
    {'comment': 'Có ai có link download bộ Adobe 2023 full crack cho Mac không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'TOOL HACK GAME BÀI, SLOT, NỔ HŨ. 🔥 Giúp bạn đổi đời sau 1 đêm. An toàn, kín đáo. Dr Zalo để cài đặt.', 'label': 1},
    {'comment': 'Mày bớt thể hiện đi được không? Nhìn ngứa mắt vcl.', 'label': 0},
    {'comment': 'Em là sinh viên năm 2, muốn tìm một công việc làm thêm liên quan đến ngành marketing. Mọi người có gợi ý gì không ạ?', 'label': 0},
    {'comment': 'Tuyển 20 bạn xem video Tiktok kiếm tiền. 📱 1k/video. Ngày kiếm 100-200k là bình thường. Link tải app ở bio.', 'label': 1},
    {'comment': 'Vừa tạch môn. Buồn đéo tả được. 😩', 'label': 0},
    {'comment': 'Cái loại bạn như mày, chỉ biết lợi dụng người khác.', 'label': 0},
    {'comment': 'Có nên mua bàn phím cơ cũ không mọi người? Sợ bị liệt phím.', 'label': 0},
    {'comment': 'Nhận làm báo cáo thực tập, tiểu luận, chuyên đề. ✍️ Chuẩn format, không đạo văn. Giá rẻ. Cứu ae qua môn.', 'label': 1},
    {'comment': 'Làm xong cái project này chắc mắt tao tăng thêm mấy độ.', 'label': 0},
    {'comment': 'Thằng Messi qua Mỹ đá như đi dạo. Đẳng cấp vẫn là mãi mãi.', 'label': 0},
    {'comment': 'Dịch vụ đòi nợ thuê. 👹 Đòi lại tiền từ những thằng bạn chó má. Không lấy được không lấy tiền. Liên hệ...', 'label': 1},
    {'comment': 'Có ai biết cách vệ sinh tai nghe TWS đúng cách không?', 'label': 0},
    {'comment': 'Mày là thằng óc chó à? Đã ngu còn hay tỏ ra nguy hiểm.', 'label': 0},
    {'comment': 'Học xong ngành này ra trường lương tháng được 10 củ không chúng mày?', 'label': 0},
    {'comment': 'Bán tài khoản VIP các trang học tiếng Anh (Elsa, Cambly). 🇬🇧 Giúp bạn nói tiếng Anh như gió. Giá sinh viên.', 'label': 1},
    {'comment': 'Cái loại đĩ như mày, thằng nào yêu phải thì đúng là đời nó xuống dốc.', 'label': 0},
    {'comment': 'Mọi người ơi, môn Xác suất thống kê có được dùng máy tính 580VNX không ạ?', 'label': 0},
    {'comment': 'VAY TIỀN CHỈ CẦN CMND + BHYT. 💰 Không thẩm định, không giữ giấy tờ. Ib mình để được hỗ trợ.', 'label': 1},
    {'comment': 'Con ml đấy suốt ngày đi kể xấu người yêu cũ. Đéo hiểu cái nết gì.', 'label': 0},
    {'comment': 'Có ai biết quán cafe nào có mèo ở khu vực Đống Đa không?', 'label': 0},
    {'comment': 'Pass lại con laptop Dell do mới được tặng Macbook. Máy còn mới, cấu hình i5 gen 10. Giá 8 củ.', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Google, Facebook. 🔥 Target đúng đối tượng, tối ưu chi phí. Ib để được tư vấn miễn phí.', 'label': 1},
    {'comment': 'Thằng lồn đấy hack map mà cứ gáy to.', 'label': 0},
    {'comment': 'Có nên mua SSD của Kington không mọi người, thấy giá rẻ mà sợ không bền.', 'label': 0},
    {'comment': 'LINK GROUP TELEGRAM GIAO LƯU GÁI NGÀNH, SV. 🌹 Kín đáo, an toàn. Ae vào giao lưu văn minh. Link ở bio.', 'label': 1},
    {'comment': 'Đm con người yêu cũ, chia tay rồi còn quay lại mượn tiền tao.', 'label': 0},
    {'comment': 'Có ai hóng anime Jujutsu Kaisen season 2 không? Nghe bảo arc Shibuya đỉnh lắm.', 'label': 0},
    {'comment': 'Bán acc Genshin Impact AR 60, full top meta (Raiden C3, Nahida C2...). 💎 Do không có thời gian chơi. Giao dịch qua admin uy tín.', 'label': 1},
    {'comment': 'Đm cái thằng lồn nào pick Yasuo team tao feed 0/10. Óc chó thì cút mẹ mày đi.', 'label': 0},
    {'comment': 'Tuyển nhân viên trực page, chốt đơn hàng mô hình anime. 💥 Lương 3-5tr/tháng. Yêu cầu am hiểu về manga, anime. Ib ngay.', 'label': 1},
    {'comment': 'Mọi người ơi, có nên roll Eula rerun này không hay đợi lôi thần ạ? Em là newbie.', 'label': 0},
    {'comment': 'Dịch vụ cày thuê rank Valorant, LMHT, Tốc Chiến. 🎮 Kéo rank thần tốc, giá sinh viên. Uy tín, bảo mật acc.', 'label': 1},
    {'comment': 'Con đĩ đấy suốt ngày cosplay anime mà nhìn như con phò. Đéo có tí thần thái nào.', 'label': 0},
    {'comment': 'Bán tài khoản VIP Crunchyroll, Funimation. 🎌 Xem anime full HD không quảng cáo, có sub Việt. Chỉ 30k/tháng.', 'label': 1},
    {'comment': 'Mày là cái loại wibu rách, suốt ngày ba cái anime bệnh hoạn.', 'label': 0},
    {'comment': 'Chap mới One Piece có tình tiết hay vcl. Oda đúng là thánh. 🔥', 'label': 0},
    {'comment': 'Bạn muốn kiếm 500k/ngày? 💸 Chơi game bài đổi thưởng, nổ hũ. Nạp 1 được 2. Rút tiền về momo trong 1 nốt nhạc. Link tải ở bio.', 'label': 1},
    {'comment': 'Cái thằng óc lợn đấy chơi Elden Ring mà cứ đòi cầm song đao từ đầu. Ngu vcl.', 'label': 0},
    {'comment': 'Pass lại con figure Rem authentic do hết tiền đóng học phí. 😭 Mới unbox, full box. Giá 2 củ.', 'label': 0},
    {'comment': 'Nhận order manga, light novel, artbook từ Nhật Bản. 🇯🇵 Hàng về sau 2-3 tuần. Công mua hộ rẻ. Dr để check hàng.', 'label': 1},
    {'comment': 'Có ai biết cách cài mod cho game Stardew Valley không ạ?', 'label': 0},
    {'comment': 'Cái loại rẻ rách, xem phim lậu còn to mồm.', 'label': 0},
    {'comment': 'Tuyển sugar baby mê anime, cosplay. Daddy chu cấp 15 củ/tháng. Cùng daddy đi event, chụp ảnh. Dr info kín. 🌹', 'label': 1},
    {'comment': 'Đm thằng nào spoil phim trên confession. Bố mày mà biết thì mày ăn cặc.', 'label': 0},
    {'comment': 'Mày là thằng đần à? Gojo Satoru mạnh nhất truyện rồi, còn thắc mắc cc gì nữa.', 'label': 0},
    {'comment': 'Nhận làm mô hình, sa bàn cho các bạn học kiến trúc, mỹ thuật. ✍️ Tỉ mỉ, chi tiết. Giá cả thương lượng.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn này hãm vcl, suốt ngày lôi chuyện chính trị vào bài giảng.', 'label': 0},
    {'comment': 'Bán acc Steam full game JRPG. 🎮 Persona, Final Fantasy, Dragon Quest... Giá rẻ hơn mua lẻ rất nhiều. Ib để xem list game.', 'label': 1},
    {'comment': 'Lại sắp đến mùa deadline rồi. Chắc tao chết chìm trong đống game chưa chơi mất.', 'label': 0},
    {'comment': 'Con ml đấy suốt ngày đi xin tiền donate trên stream mà đéo có tài năng cc gì.', 'label': 0},
    {'comment': 'Bạn đang gặp khó khăn tài chính? 💰 Vay nóng sinh viên, không cần gặp mặt. Chỉ cần thẻ SV. Duyệt vay trong 5 phút.', 'label': 1},
    {'comment': 'Có đứa nào đi Comic Con cuối tuần này không? Lập team đi cho vui.', 'label': 0},
    {'comment': 'Mày là cái loại súc vật, xem anime loli còn thẩm du được.', 'label': 0},
    {'comment': 'Nhận cày sự kiện, la hoàn game Genshin Impact. 🔥 Nhanh gọn, uy tín. Ib để báo giá.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng cả ngày đéo làm gì ngoài cày game. Tương lai mờ mịt vcl.', 'label': 0},
    {'comment': 'Có ai biết cách build team cho Ayaka không ạ? Em mới roll được ẻm.', 'label': 0},
    {'comment': 'Bán tài khoản Netflix, Disney+ Premium 4K. 🎬 Xem phim Marvel, Star Wars thả ga. Giá chỉ 40k/tháng.', 'label': 1},
    {'comment': 'Tối nay MU đá, anh em fan Chelsea chuẩn bị tinh thần chưa? 😂', 'label': 0},
    {'comment': 'Đm cái thằng lồn nào chơi game mà bật mic xong chửi cả team. Vô văn hóa.', 'label': 0},
    {'comment': 'Có ai có link download game The Witcher 3 full DLC không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'TOOL HACK GAME BÀI, SLOT, NỔ HŨ. 🔥 Giúp bạn đổi đời sau 1 đêm. An toàn, kín đáo. Dr Zalo để cài đặt.', 'label': 1},
    {'comment': 'Mày bớt thể hiện đi được không? Fan anime nửa mùa.', 'label': 0},
    {'comment': 'Em là sinh viên năm nhất ngành CNTT, muốn tìm một CLB để tham gia. Mọi người có gợi ý gì không ạ?', 'label': 0},
    {'comment': 'Tuyển 20 bạn xem livestream kiếm tiền. 📱 1k/phút. Ngày kiếm 200-300k. Rút tiền về Momo. Link tải app ở bio.', 'label': 1},
    {'comment': 'Vừa tạch môn. Buồn đéo tả được. Lại tốn tiền học lại. 😩', 'label': 0},
    {'comment': 'Cái loại bạn như mày, chỉ biết lợi dụng người khác.', 'label': 0},
    {'comment': 'Có nên mua tay cầm game 8bitdo không mọi người? Thấy review khá tốt.', 'label': 0},
    {'comment': 'Nhận làm báo cáo thực tập, tiểu luận, chuyên đề. ✍️ Chuẩn format, không đạo văn. Giá rẻ. Cứu ae qua môn.', 'label': 1},
    {'comment': 'Làm xong cái project này chắc tao phải cày game bù mấy ngày.', 'label': 0},
    {'comment': 'Thằng main trong bộ anime này ngu lồn vcl, xem mà tức.', 'label': 0},
    {'comment': 'Dịch vụ đòi nợ thuê. 👹 Đòi lại tiền từ những thằng bạn chó má. Không lấy được không lấy tiền. Liên hệ...', 'label': 1},
    {'comment': 'Có ai biết cách stream game lên facebook không bị mờ không?', 'label': 0},
    {'comment': 'Mày là thằng óc chó à? Đã ngu còn hay tỏ ra nguy hiểm.', 'label': 0},
    {'comment': 'Học xong ngành này thất nghiệp thì nhục. 🥲', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang học tiếng Nhật (Duolingo, Memrise). 🇯🇵 Hỗ trợ ae wibu xem anime không cần sub. Giá sinh viên.', 'label': 1},
    {'comment': 'Cái loại đĩ như mày, thằng nào yêu phải thì đúng là đời nó xuống dốc.', 'label': 0},
    {'comment': 'Mọi người ơi, môn Phân tích thiết kế hệ thống có khó không ạ?', 'label': 0},
    {'comment': 'VAY TIỀN CHỈ CẦN CMND + BHYT. 💰 Không thẩm định, không giữ giấy tờ. Ib mình để được hỗ trợ.', 'label': 1},
    {'comment': 'Con ml đấy suốt ngày đi kể xấu người yêu cũ. Đéo hiểu cái nết gì.', 'label': 0},
    {'comment': 'Có ai biết quán cafe anime nào ở Hà Nội không?', 'label': 0},
    {'comment': 'Pass lại con laptop gaming do cần tiền mua figure. 😭 Máy còn mới, cấu hình i7 gen 11, RTX 3060.', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Google, Facebook. 🔥 Target đúng đối tượng, tối ưu chi phí. Ib để được tư vấn miễn phí.', 'label': 1},
    {'comment': 'Thằng lồn đấy hack game rõ rành rành mà admin đéo ban.', 'label': 0},
    {'comment': 'Có nên mua Nintendo Switch cũ không mng, sợ bị drift.', 'label': 0},
    {'comment': 'LINK GROUP TELEGRAM GIAO LƯU GÁI NGÀNH, SV. 🌹 Kín đáo, an toàn. Ae vào giao lưu văn minh. Link ở bio.', 'label': 1},
    {'comment': 'Đm con người yêu cũ, chia tay rồi còn quay lại mượn tiền tao.', 'label': 0},
    {'comment': 'Phim mới của Marvel dạo này xem chán vcl, kỹ xảo thì ổn mà kịch bản như cc.', 'label': 0},
    {'comment': 'Bán tài khoản Netflix Premium, HBO Go, Disney+. 🎬 Xem phim 4K không giới hạn. Giá chỉ từ 30k/tháng. Ib để chốt đơn.', 'label': 1},
    {'comment': 'Đứa nào có link xem Oppenheimer bản đẹp không cho tao xin với. Lười ra rạp quá.', 'label': 0},
    {'comment': 'Tuyển 20 bạn xem video, review phim kiếm tiền. 🍿 10k/review. Rút tiền về Momo. Ib để nhận job.', 'label': 1},
    {'comment': 'Mọi người ơi, có nên mua màn hình cong để chơi game và xem phim không? Trải nghiệm có sướng hơn nhiều không ạ?', 'label': 0},
    {'comment': 'Dịch vụ get link Fshare, 4share, rapidgator. 🚀 Tải phim, game, tài liệu max tốc độ. Chỉ 5k/link.', 'label': 1},
    {'comment': 'Cái kết phim The Glory xem đã cái nư vcl. Mong phần 3 quá.', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang phim lậu (phimmoi, bilutv...). 🎞️ Xem phim không quảng cáo, load nhanh. 20k/tháng.', 'label': 1},
    {'comment': 'Mày là cái loại fan phong trào, xem được mấy bộ phim đã ra vẻ ta đây am hiểu.', 'label': 0},
    {'comment': 'Anime Chainsaw Man art đẹp vcl, combat cũng mãn nhãn nữa.', 'label': 0},
    {'comment': 'Bạn muốn kiếm thêm thu nhập? 💸 Tuyển CTV bán vé xem phim CGV, Lotte. Chiết khấu cao. Làm online.', 'label': 1},
    {'comment': 'Cái thằng óc lợn đấy chơi game mà cứ AFK. Báo cả team.', 'label': 0},
    {'comment': 'Pass lại con laptop gaming do muốn lên đời PC. Cấu hình R5 5600H, RTX 3050Ti. Còn bảo hành. Giá 15 củ.', 'label': 0},
    {'comment': 'Nhận order đĩa Blu-ray, DVD phim, anime từ Amazon Nhật. 🇯🇵 Hàng về sau 2 tuần. Dr để check giá.', 'label': 1},
    {'comment': 'Có ai biết cách kết nối loa soundbar với laptop để xem phim cho hay không?', 'label': 0},
    {'comment': 'Cái loại rẻ rách, spoil phim làm mất hứng người khác.', 'label': 0},
    {'comment': 'Tuyển sugar baby mê phim ảnh, Netflix and chill. Daddy chu cấp 15 củ/tháng. Yêu cầu ưa nhìn, kín đáo. Dr info. 🌹', 'label': 1},
    {'comment': 'Đm cái rạp phim gần trường, màn hình thì mờ mà âm thanh như cái loa kẹo kéo.', 'label': 0},
    {'comment': 'Mày là thằng đần à? Tình tiết phim rõ rành rành ra thế mà cũng đéo hiểu.', 'label': 0},
    {'comment': 'Nhận làm phụ đề (subtitle) cho phim, video. ✍️ Dịch chuẩn, khớp time. Giá sinh viên. Ib để trao đổi.', 'label': 1},
    {'comment': 'Thằng thầy dạy môn này hãm vcl, suốt ngày spoil phim trong giờ học.', 'label': 0},
    {'comment': 'Bán tài khoản Adobe Premiere Pro, After Effects bản quyền. 💻 Hỗ trợ ae edit video, làm phim. Giá chỉ 50k/tháng.', 'label': 1},
    {'comment': 'Vừa xem xong John Wick 4. Đánh nhau từ đầu đến cuối phim, mãn nhãn vcl.', 'label': 0},
    {'comment': 'Con ml đấy suốt ngày đi xem phim một mình. Chắc đéo có ai yêu. 😂', 'label': 0},
    {'comment': 'Bạn đang gặp khó khăn tài chính? 💰 Vay nóng sinh viên, không cần gặp mặt. Chỉ cần thẻ SV. Duyệt vay trong 5 phút.', 'label': 1},
    {'comment': 'Có đứa nào đi xem Lật Mặt 6 của Lý Hải chưa? Nghe bảo hay lắm.', 'label': 0},
    {'comment': 'Mày là cái loại súc vật, xem phim quay lén còn đi share.', 'label': 0},
    {'comment': 'Nhận cày thuê, làm nhiệm vụ game Honkai Star Rail. 🔥 Nhanh gọn, uy tín. Ib để báo giá.', 'label': 1},
    {'comment': 'Thằng bạn cùng phòng cả đêm cày phim không ngủ, sáng ra ngáp như thằng nghiện.', 'label': 0},
    {'comment': 'Có ai biết cách build team cho Seele không ạ? Em mới quay được ẻm.', 'label': 0},
    {'comment': 'Bán tài khoản Steam đã mua sẵn game Cyberpunk 2077. 🎮 Rẻ hơn mua trực tiếp. Giao dịch qua admin.', 'label': 1},
    {'comment': 'Tối nay Chelsea đá, fan Arsenal đừng gáy sớm nhé.', 'label': 0},
    {'comment': 'Đm thằng nào chơi game mà để con nhỏ nhà nó khóc inh ỏi trong mic.', 'label': 0},
    {'comment': 'Có ai có link download phim Avatar 2 bản đẹp không cho mình xin với. 🙏', 'label': 0},
    {'comment': 'TOOL HACK GAME BÀI, SLOT, NỔ HŨ. 🔥 Giúp bạn đổi đời sau 1 đêm. An toàn, kín đáo. Dr Zalo để cài đặt.', 'label': 1},
    {'comment': 'Mày bớt thể hiện đi được không? Xem được mấy bộ phim Oscar đã nghĩ mình là nhà phê bình à?', 'label': 0},
    {'comment': 'Em là sinh viên năm nhất ngành Truyền thông, muốn tìm một CLB về làm phim. Mọi người có gợi ý gì không ạ?', 'label': 0},
    {'comment': 'Tuyển 20 bạn xem livestream kiếm tiền. 📱 1k/phút. Ngày kiếm 200-300k. Rút tiền về Momo. Link tải app ở bio.', 'label': 1},
    {'comment': 'Vừa tạch môn. Buồn đéo tả được. Lại tốn tiền học lại. 😩', 'label': 0},
    {'comment': 'Cái loại bạn như mày, chỉ biết lợi dụng người khác.', 'label': 0},
    {'comment': 'Có nên mua máy chiếu mini để trong phòng trọ xem phim không mọi người?', 'label': 0},
    {'comment': 'Nhận làm báo cáo thực tập, tiểu luận, chuyên đề. ✍️ Chuẩn format, không đạo văn. Giá rẻ. Cứu ae qua môn.', 'label': 1},
    {'comment': 'Làm xong cái project này chắc tao phải cày hết list phim đang dở.', 'label': 0},
    {'comment': 'Thằng main trong bộ phim này ngu lồn vcl, xem mà tức.', 'label': 0},
    {'comment': 'Dịch vụ đòi nợ thuê. 👹 Đòi lại tiền từ những thằng bạn chó má. Không lấy được không lấy tiền. Liên hệ...', 'label': 1},
    {'comment': 'Có ai biết cách stream phim từ laptop lên TV không?', 'label': 0},
    {'comment': 'Mày là thằng óc chó à? Đã ngu còn hay tỏ ra nguy hiểm.', 'label': 0},
    {'comment': 'Học xong ngành này thất nghiệp thì nhục. 🥲', 'label': 0},
    {'comment': 'Bán tài khoản VIP cho các trang học tiếng Anh qua phim (eJOY, Studyphim). 🇬🇧 Giúp bạn nói tiếng Anh như gió. Giá sinh viên.', 'label': 1},
    {'comment': 'Cái loại đĩ như mày, thằng nào yêu phải thì đúng là đời nó xuống dốc.', 'label': 0},
    {'comment': 'Mọi người ơi, môn Phân tích thiết kế hệ thống có khó không ạ?', 'label': 0},
    {'comment': 'VAY TIỀN CHỈ CẦN CMND + BHYT. 💰 Không thẩm định, không giữ giấy tờ. Ib mình để được hỗ trợ.', 'label': 1},
    {'comment': 'Con ml đấy suốt ngày đi kể xấu người yêu cũ. Đéo hiểu cái nết gì.', 'label': 0},
    {'comment': 'Có ai biết quán cafe phim nào ở Hà Nội không?', 'label': 0},
    {'comment': 'Pass lại con laptop gaming do cần tiền mua máy ảnh. 😭 Máy còn mới, cấu hình i7 gen 11, RTX 3060.', 'label': 0},
    {'comment': 'Nhận chạy quảng cáo Google, Facebook. 🔥 Target đúng đối tượng, tối ưu chi phí. Ib để được tư vấn miễn phí.', 'label': 1},
    {'comment': 'Thằng lồn đấy hack game rõ rành rành mà admin đéo ban.', 'label': 0},
    {'comment': 'Có nên mua Nintendo Switch cũ không mng, sợ bị drift.', 'label': 0},
    {'comment': 'LINK GROUP TELEGRAM GIAO LƯU GÁI NGÀNH, SV. 🌹 Kín đáo, an toàn. Ae vào giao lưu văn minh. Link ở bio.', 'label': 1},
    {'comment': 'Đm con người yêu cũ, chia tay rồi còn quay lại mượn tiền tao.', 'label': 0}
]

In [ ]:
import pandas as pd

In [ ]:
label_ds = pd.DataFrame(ds_dict)
label_ds.drop_duplicates()

In [ ]:
label_ds.drop_duplicates().to_csv("train_spam.csv", index=False, encoding='utf-8-sig')

In [ ]:
tt = pd.read_csv("train_spam.csv")
tt

In [ ]:
{
"cô hát thì cũng k gọi là hay đâu nhưng sao nh fan thế nhỉ :)))": 0,
"Thì thôi đừng nói 😂": 0,
"Vừa nhận hàng là mình test thử luôn. Xinh lắm, shop còn nối giúp nữa chỉ cần cắm vào là sử dụng thôi": 0,
"Sydney hay còn gọi là Auditorium nha bạn": 0,
"Tao sống thử ở đây 5 năm rồi, tao k hài lòng với thế giới tạo ra tao, tao muốn từ chối tiếp tục với thế giới này.": 0,
"Dương Chung muôn vc, t đợi hơn nửa tiếng": 0,
"Ghi  -4 bàn là được rồi làm 6 bàn thì hết đường về :))": 0,
"Hàng rất đẹp, xoay mượt, giá rẻ": 0,
"Đánh cho Mỹ cút Đánh cho Nguỵ nhào": 0,
"Đỗ Thanh Vinh hê lo nha": 0,
"Cmsn c nhá. Mỉ chs game quá h ms chúc dc🤣🤣": 0,
"có thể thg trộm đã nhìn thấy áo đấu MU nên đã thương tình dọn nhà": 0,
"Minh Ngọc. Chỉ anh với": 0,
"đã có Hành tinh ánh sáng trên youtube nhé": 0,
"E trúng xỉu số8": 0,
"📍 Chương trình MÙA TỰU TRƯỜNG áp dụng từ /9 trên toàn quốc ": 1,
"Cho mình  slot với <": 0,
"c My k thấy em ạ": 0,
"Bên em sẵn hàng chị nhé": 0,
"nhìn đểu thế :)) Rose Bùi": 0,
"Trần Thư quang trọng là người nghe có thấy hợp không nha bạn =))": 0,
"đột quỵ chết mẹ mày đi": 0,
"Khi  cánh cửa khép lại thì lại k có chìa khóa để mở các cánh cửa khác 😂": 0,
"người tốt luôn duoc may mắn ❤": 0,
"Thiếu tiền cj hiếu dell trả bớt xạo l đi m": 0,
"Cảm ơn chị ❤ em rất thích bài Sóng mà chị phân tích": 0,
"Ủa đâu ra xịn dữ z": 0,
"Chị hát dở mà đc cái seeding c mạnh": 0,
"Khóa kênh vào cho con cháu t yên :(((((( , nghe cái giọng đã chối rồi đừng nói gì tới nội dung 😒": 0,
"Đọc đề gần gũi mà dễ hiểu": 0,
"Lêu lêu ngta cho đã cái giờ bị trầm cảm? Bây thấy nó có vô lí vl k": 0,
"Con màu nâu cột nơ trên xào đi bạn": 0,
"Áo thun đẹp mà chỉ có 49k như thế liệu vải có tốt không? mua thử biết liền": 0,
"cán bộ mua xe cho em đi với": 0,
"Khúc cuối mv có  đoạn demo tầm s , có ai biết bản full là gì k": 0,
"Thuý Minh chẳng có lẽ của bà to thế ah. Chỉ có thể là bà. Haha": 0,
"Mới mua xong ly trà sữa thì thấy tin :))": 0,
"Packaging xink ạ, hàng đóng gói trong hộp giấy riêng của Cocoon cưng quá. Xài thử 2 lần thấy tẩy ở mức độ vừa phải, ko mạnh, bã cà phê ít. Có mùi cà phê kèm mùi hương liệu hơi nồng nhưng xả nước thì chỉ còn mùi cà phê thôi. Rate 5/5 khuyến khích Cocoon tăng % bã cà phê nhen.": 0,
"ông này tài quá": 0,
"Chị ơi chị ở đâu để khi nào em bảo với mẹ để đưa em đi nhà chị chơi , để em xem mẹ có bận ko đã": 0,
"Túi xịn xò. Giao hàng đầy đủ. Gói hàng cũng siêu xinh luôn. Nên mua nhé mọi người. Cảm ơn shop.": 0,
"May cho Đi súng không nổ . Chứ súng nổ vẫn kịp chạy trốn .": 0,
"Thiên Thanh thà phục vụ mấy ảnh🤣": 0,
"Thôi t nấu cgi là hư cái đó ": 0,
"Không hân hoan lắm vì còn học thể chất :((( ": 0,
"Toàn tin tào lao": 0,
"nhìn phần lưng con chim chỗ lông sáng nhất ngay sau cổ thì mới thấy nó nét hơn một tí": 0,
"anh mình cũng bắt trend nhé =)))": 0,
"You forgot to drag your eyes and say lêu lêu, not childish enough pal :))": 0,
"Austin Ames muốn ăn gì mà nói kiểu đó :)))": 0,
"May ep cham 8k 75585": 1,
"Sao add nói chuẩn thế😂😂": 0,
"cái đoạn nướng cháy pizza để phốt công nhận là chuyên nghiệp thật :)": 0,
"Đâu đây chị ơi": 0,
"Nguyễn Trần Thuỳ Trang Ấy cha chả:v": 0,
"Cực kì thuyết phục🙄": 0,
"Mới nhận hàng, đóng gói cẩn thận. Cần một thời gian trải nhiệm và dùng thử để cảm nhận sản phẩm.": 0,
"Mày cược đòi lấy Kim cương Thế mày thua thì mày trả cái gì Nói phải nghĩ đừng cho mình có quyền đòi hỏi như vậy Vẫn còn non và xanh lắm Đàm vĩnh Biệt ơi chuẩn bi đi là vừa ": 0,
"Nguyễn Trang  Giống quân với thư hey": 0,
"nghe thì có vẻ Lý Mạc Sầu nhưng thực tế có vẻ rất Mộ Dung Phục !!": 0,
"Mày có nín ngay k?": 0,
"giao hang hoi lau nhung ma shop cung co gui thu xin loi voi lai cung dang dich nen ko sao hehe. shop lam z cung chuyen nghiep phet. 5 sao": 0,
"vào sg 50k / không cũng mua . 100k / không cũng mua ! ha ha": 0,
"sp tốt, chắc chắn.": 0,
"Mất bao nhiêu xiền với bà nhàn mà đã đc trúng gì đâu😪😪😪😪": 0,
"Váy đỏ cuối sào đi chi oi ": 0,
"Mình đã nghiên cứu và kết luận là tại thuỷ triều đỏ nhed": 0,
"Hang Thu Ha làm vs nhọ đi": 0,
"Và đưa F lên top trending =)))))))": 0,
"Yêu nhau bác sĩ nói cưới thì cưới =))": 0,
"Thanh Tâm ê chị , y chang cái nón em🤣": 0,
"Sp xài cũng tốt lắm. Mình xài dd vệ sinh Intima Ziaja với gel Inner này thấy tình trạng khí trắng bớt hẳn.": 0,
"Sao ko chở em nó đi mua quà??? Chở vào bệnh viện tâm thần làm món quà bất ngờ cho em nó ngạc nhiên🤣": 0,
"Cô thật tình noi chuyện cưng quá": 0,
"Minh Chau trà đá hồ Giảng Võ thôi": 0,
"Nón địa nhieu chi d": 0,
"Quán lâu năm, nhưng không ngon.": 0,
"Để t đi đốt nhang á 😂😂😂😂": 0,
"Ủa sly có nắng:)) 😂😂😂": 0,
"Mất ngủ.. Vào xem ông mà tắt trym sớm thế hả.": 0,
"Trông bệnh đ chịu được ": 0,
"Mun Củ'Lìn má rảh hết sức😂": 0,
"Thanh Huyềnn thật xong lại chụp đc vài con ảnh xinh xắnn 🤭🤭": 0,
"Diep Nguyen lần sau mình thử cách này đi 🤣🤣🤣": 0,
"có qua có lại cđ gì, mình khéo phải xl ngta vì nói nhiều vl ;v": 0,
"r giờ nhà đó đi hết luôn r hả ??": 0,
"[https://www.facebook.com/people/Hoang-Anh/100036903588642?comment_id=Y29tbWVudDoyODk0ODEyNTUwNjM2NTIxXzE2OTU3NTgyMDk2OTY5Mg%3D%3D](https://www.facebook.com/people/Hoang-Anh/100036903588642?comment_id=Y29tbWVudDoyODk0ODEyNTUwNjM2NTIxXzE2OTU3NTgyMDk2OTY5Mg%3D%3D) Hoang Anh Chúc mừng năm mới 🙂": 1,
"Chi thac toc de thuong wa": 0,
"Đây là việc được voi đòi tiên thế là tèn ten": 0,
"Dương Hằng Bảo Châu hảo thầy =))": 0,
"nghe có vẻ vô lý nhỉ": 0,
"Nhân vật phụ mà lên hình to vậy abe 🙂": 0,
"Còn có tôm là may rồi :))": 0,
"Coi Coi coi chừng chị à nghe hứ": 0,
"mã số bhxh đúng theo thông tin tra rồi mà muốn khai báo cũng không  được  nữa. cứ bảo sai mã bhxh, trong khi đã tra cứu rõ ràng từ trang bhxh 😠": 0,
"Mai dò số, trúng ti cho luôn": 0
"Ru ngu tot that": 0,
"Năm nay mũ có lông ko e? Form to hay ôm": 0,
"Sau tiếng cầy game thua sml .-.": 0,
"Phan Hoàng Anh Thư tứk": 0,
"Cùng là du học sinh áo thun quần đùi balo vải nhìn phèn phèn y chang nhau. Ai ngờ đụng vô con trai thái tử bất động sản Hongkong": 0,
"Định bụng ko khẩu nghiệp rồi. Mà xuất hiện nhiều quá 😡": 0,
"Bên mình có bán quan tài nhé ai có ý định tự tử thì ib để dc phục vụ tận tình hợp lí nhé :))": 1,
"Xem de co vu Nhat Linh oi": 0,
"Người chết mà thấy thả haha không": 0,
"Mấy thằng chó ăn cứt lo chuyện bao đồng": 0,
"Đương nhiên k phai Đông Phương ca rôi 😂😂😂": 0,
"Hài lòng với sản phẩm này ổn thời gian sử dụng mới biết được độ bền": 0,
"Mũ xinh lắm ạ, có miếg giữ from nên rất ưng, cpi hàg giao hàg nhah 💓": 1,
"Đá Tảng như nàoooo": 0,
"Vẫn nhớ như in cái cảm giác trên bàn mổ, và khi e bé được ra ngoài. Cảm ơn các bác sĩ bệnh viện đã cho mẹ con nhà mình kỳ nghỉ dưỡng tại bệnh viện": 0,
"Dép như hình, hơi cứng và nặng mùi khá chim ưng, sẽ ủng hộ shop lần sau :v": 1,
"với tầm giá này thì ổn, ok nhe mn. mình thích phiên bản này hơn là phiên bản nội địa TQ. quá nhanh quá nguy hiểm :))))))": 0,
"C tu van cho e 9684798": 1,
"Các bạn ghé shop mình mua giày nha": 1,
"Nguyên hộp còn nhãn niêm phong, mở nguồn cài mới, hôm sau nhận tin nhắn bảo hành điện tử. rất tuyệt!": 1,
"Tôi rất thích máy sấy tóc này, nhỏ gọn, cầm chắc chắn, tôi đã mua 2 cái máy sấy tóc hiệu này trên tiki. Rất ưng": 1,
"Khóa học chăm sóc da mặt học phí .. giảm 5% chỉ còn 6.5., thời gian học là 6 ngày nhé": 1,
"4cm size bn ạ?": 0,
"Kem trị nám Bao nhiêu một hộp e ơi": 1,
"Chốt em 6887": 1,
"A oi em da 5tuoi roi khong biet con co the cao dk chut nao nua ko anh .": 0,
"C tu van cho e 9684798": 1,
"Cùng là du học sinh áo thun quần đùi balo vải nhìn phèn phèn y chang nhau. Ai ngờ đụng vô con trai thái tử bất động sản Hongkong": 0,
"M nói thiệt là m k biết m mua đồ cho 2 pé nhà ở bên shop k biết bao nhiêu lần rồi nữa, đồ chất vải bình thường thôi à nhưng đc giá rất là rẻ & kĩu mẫu cưng lắm. Các mẹ cứ yên tâm mua nha 😉": 1,
"Đừng cười chàng trai đấy. Chiếc xe là thành quả của hàng chục giờ lilo tụt cả lưỡi vào đấy. Các bạn nên coi trọng sức lao động của cậu ấy 😮💨": 0,
"Bí thư Nguyễn Văn Nên nhìn nhận đây là cơ hội để Cần giờ làm việc đó là: chia sẻ với cộng đồng và phát triển kinh tế du lịch dịch vụ.": 0,
"Thật tuyệt vời. Tôi đã sử dụng rất nhiều sản phẩm của Xiaomi và rất hài lòng về chất lượng, giá cả của Xiaomi.": 1,
"Khóa học giảm giá cực lớn chỉ hôm nay": 1,
"Code:Giảm trực tiếp % tối đa K cho đơn 99K": 1,
"Bộ má hồng Bn a": 1,
"Dẫn ít ví dụ .": 0,
"Ngộ không có chai tương ớt biết like": 0,
"Mơ mộng đi em": 0,
"em nghĩ thầy nên up slide trước mỗi buổi học để sinh viên có thể xem bài trước ở nhà thay vì up slide sau mỗi buổi học ạ": 0,
"A tôi đang bayyy. Rất xứng đáng 😍": 0,
"BB: cuộc sống vốn dĩ bất công mà em, page đang tập cho e trưởng thành hơn đó, get used to it :)": 0,
"Các bạn ghé shop mình mua giày nha": 1,
"Bên mình có bán quan tài nhé ai có ý định tự tử thì ib để dc phục vụ tận tình hợp lí nhé :))": 1,
"C tu van cho e 9684798": 1,
"Chúng tôi đang xem xét coi có phải sinh đôi kh": 0,
"Nên mua lắm chứ, sao còn chần chừ nữa ạ.": 0,
"Ngô Hiếu rp m h": 0,
"Bị đứng mẹ mít ơi ": 0,
"Loc Pham hôm trước đi ngồi bàn ăn tối chung với chú, chú có dạy như vậy đâu 😞": 0,
"Bao tên, bao check tích": 1,
"Chi my banh beo": 0,
"Nhược điểm là không có hộp thu bụi riêng nên hay nghẹt.": 0,
"Đã nhận dc hàng, giao hàng khá nhanh, được tặng hai miếng stickers xinh xinh": 0,
"Ngưỡng mộ thật nhưng mình k có ông, ông bỏ bà đi từ thời mẹ mình còn nhỏ, những câu chuyện đc nghe từ bà khi còn bé xíu chỉ vỏn vẹn là truyện cổ tích  👣": 0,
"Có 5 bình nước thuỷ tinh Home Credit bay vào balo  Hoa Hạt Mưa ;Hoàng Thị Mỹ Vân #HomeCreditVietnam": 1,
"Tưởng mất luôn rồi ": 0,
"Bao nhiêu cai chị": 0,
"cuốn này vừa hay vừa đẹp. trước toàn phải để nó với hoàng tử bé ngay đầu giường cho có cảm giác dễ ngủ. :": 0,
"Bao nhieu  chiec": 0,
"có mấy cái này thì đánh đẩm mẹ gì khi có giặc ngoại xâm": 0,
"Trần dần là cộngg sản nằm vùng bên cali😂": 0,
"Sản phẩm rất đẹp nha mọi người  shop giao hàng đúng sz   giao hàng cũng khá nhanh nha  mọi nên mua quá Tuyệt vời 🤨🤨🤨🤨🤨🤨❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤💏💏💏💏💏💏❤💏😔😔😔😊🤨💏💏": 0,
"Shop giao đầy đủ hàng, đóng gói cẩn thận; Chất lượng thì phải dùng một thời gian nữa mới biết, nếu có dịp sẽ tiếp tục ủng hộ shop ^^": 0,
"Ngốc Nghếck bắt đại t ik cho òi 😂 bắt t về m nui t nhoa 😁😂😅": 0,
"Kiểu táo sắp biểu diễn ở hà nội ý": 0,
"Yeah. Rất yomost. Thà môt phút huy hoàng rồi vụt tắt": 0,
"my zth qá trờiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii": 0,
"Tui vẫn xài 6s bình thường. Năm sau lên 7 lun 😂": 0,
"màn hình hơi lỗi cảm ứng, đôi khi bị liệt cảm ứng ở góc trên bên phải màn hình": 0,
"E ĐI HỌC Đây c ơi": 0,
"sản phẩm đóng gói rất đẹp, chuẩn hàng auth, dùng lên rất sảng khoái, sát khuẩn cao. Sản phẩm phù hợp với da dầu, nhạy cảm dễ mụn như mình, chữa mụn ẩn rất tốt. còn đối với mụn đầu đen thì ko có tác dụng lắm": 0,
"Cj thy ai địch nổi 🤔🤔🤗🤗": 0,
"Shop Kazen   cái này còn k ạ": 1,
"A đứng ngoài cửa rồi , e ra đi :))": 0,
"Giá tốt nhất shopee, đóng gói cẩn thận, mình mua hôm sale được giá rất rẻ, mặc dù ban đầu đặt nhầm nhưng may shop vẫn nhớ đơn của mình, 5* mọi ng nên mua của shop nha": 0,
"Nguyễn Thị Trang  chắc k hay lại mất tg": 0,
"Lan vừa đi về thì toang. May nhé": 0,
"mình mua một chiếc xe đạp để thân thiện với môi trường hơn. xe có thể gấp lại vô cùng nhỏ gọn, thao tác dễ dàng. xe đạp rất nhẹ, có ổ líp tăng giảm tiện dụng, bánh xe 20 inch, khá nhỏ nên chạy không nhanh lắm. xe nặng khoảng 15kg, khung khá cứng cáp. anh shipper dễ thương giúp mình đẩy thùng hàng vào thang máy luôn ạ. chiếc xe rất đáng mua cho các bạn yêu thích vận động hay bảo vệ môi trường nhé": 0,
"bó tay bà ngọc anh này": 0,
"Cuang hai hoặc cung phụng": 0,
"Đóng giường cho các cụ đẹp quá nên các cụ tặng ý mà :v": 0,
". Cuối cùng chỗ để ko quan trọng lắm. Mua về là mừng rồi đằng nào chả có chỗ ?": 0,
"Chuẩn rồi anh em ey :))))))": 0,
"Mây hợp tác quá điiii cưng e": 0,
"Thanh Thảo Nguyễn Văn Thảo  Hồng Phươngpải ko zạ": 0,
"Thứ 6: Tập thể dục, chơi một số môn thể thao. Đừng là E-Sport là được.": 0,
"8  Phiến Ngọc Huỳnh Hồng Như Phi Nguyen": 0,
"khum biếc ra đời làm gì nma nhìn thấy mấy ông nhả khói tôi chỉ nghĩ đến    mồm các ông chả khác j cái bát hương :))))))": 0,
"Đối với bạn nào muốn xem về người yêu cũ và muốn tìm cách quay trở lại inbox mình nhé": 1,
"Năm mới vv nha mẹ Mít": 0,
"hôm nay vé phim nhím ae ơiii )))": 0,
"Đc đi lại bình thường cũng hãy cẩn thận chứ bây giờ dịch chưa hết mà tràn ra đường ăn uống, vui chơi thì vài hôm ko đi đâu được lại đổ thừa này nọ.": 0,
"cái kết ?": 0,
"trời đụ thằng con trời đánh": 0,
"Thế này phải ra vào chục lần cho bõ🥲": 0,
"Chị ơi sao e ko thành FC": 0,
"Hero Xử Nữ vch cái zề 😌😌": 0,
"Nếu Bác còn  nào din mà hỏng dvd cũng đc bác ới e nhé. E chỉ cần usb với thẻ thôi": 0,
"Nguyễn Hồng đi cái này nữa": 0,
"Chốt áo xinh xịn mịn -&&&-&@❤❤hốt hốt &&-xanh Ngọc đi e": 1,
"mình bỏ tuýp này vào tủ lạnh để bảo  quả n. khi mình sử dụng thì nó có hơi rít và nóng mặt (vitamin chị thường như vậy). aPly xong thì hết rít và nóng. các bạn nên chờ nó khô se lại rồi hãy tiếp tục bước sp_dưỡng_da nhé. aPly liên tục dễ nóng mặt mà sp_dưỡng_da còn chẳng hiệu  quả  mấy": 0,
"Ok thầy ;v": 0,
"hom bua choi ak nhma m ko co thua hihi": 0,
"Cho chị xem mẫu dây nịt nam": 1,
"Những em k muốn thi thì chỉ có những em lười học ": 0,
"Dung r oi chi": 0,
"Anh em thất lạc nè Thầy Ba :))": 0,
"😔 nghe có vẻ hay ho": 0,
"per ly trà sen của anh đem qua có ngày nó thành như v đó 😭😭 hức hức ...": 0,
"Nguyễn Đạt lần đầu anh thấy em yếu lòng": 0,
"Nhin em thay themwa": 0,
"đi rank gặp anh Tùng là xác đinh": 0,
"Hà Trần lấy nhận ng khác à.k phải nhận cua mình a": 0,
"Máy khác hình như đúng thông số. Lắp đặt nhanh. Máy chạy tốt làm lạnh nhanh. Cục nóng k ồn như mọi ng thường nói về daikin": 0,
"có gói cứu trợ phát là bùng dịch nhỉ tội cho cô em tên Nhung, không hiểu mô tê gì thành vật thế thân chuyển hướng dư luận :))": 0,
"Liêm sỉ đáp thùng rác rồi": 0,
"Na Doan c lấy bé trai hay gái ạ": 0,
"- Sai khi ngay từ đầu chồng đã nói không thích có con mà vẫn cố gắng kết hôn để rồi sau này lại ép nó nhận  điều mà không hề thích, tất nhiên với người vợ và rất nhiều người khác thì đứa trẻ là sinh mệnh họ cực kì trân trọng nâng niu.": 0,
"Mới nhận hàng chưa ăn nên chưa biết có ngon ko nhưng thấy ngày sx còn mới,giao hàng đúng ngày dự kiến trong mùa dịch nên 5☆": 0,
":)) mày biết tại sao k thằng ngu vì đứng đấy ngăn ngừa phòng chống hiệu quả nhất cái loại ngu dốt như mày nên học lại đi đã k hiểu ý người khác đã vào cmt người ta chửi cho nhục mặt lắm thân ^^ còn tao chưa nói gì là văn hoá hay k văn hoá tao đang muốn nói ý thức của các ông say rượu khi tham gia giao thông": 0,
"Hi vong em được nhận quà của chị Na ạ": 1,
"Nhìn có nét giống Cẩm Ly": 0,
"Đồng đội chỵ sao ko nghe thấy gì hết": 0,
"Ít nhất nó có bím": 0,
"Đây là con " bong sắc " hả mng. Tui tưởng đâu là gái chứ , 🙏": 0,
"năm đổ lại đây ko động vào  miếng nào luôn :)))": 0,
"Đặc sản của Sài Gòn là gì thì mình hok biết ah. Chứ mình biết mình cũng ăn dồi, ăn món khác làm gì.": 0,
"Anh này nhìn kĩ ảnh ăn mặc vuốt keo các thứ lên thì cũng y vậy": 0,
"Hai chi em xinh lam.": 0,
"Nemo đang bận ôm chị khanh nha": 0,
"đáng nhẽ nên làm việc này từ sớm . chứ hàn quốc họ làm từ lâu rồi ạ": 0,
"Từ hôm qua đến giờ thấy các báo toàn giật tít tào lao. Do những năm gần đây lượng thí sinh tăng cao, các đơn vị tổ chức thi (BC & IDP) đã liên kết với các trường đại học, trường cấp  làm địa điểm tổ chức thi. Tuy nhiên Bộ GD yêu cầu các đơn vị liên kết tổ chức thi IELTS phải có đầy đủ giấy tờ của bộ. Bây giờ người ta đang tạm dừng thi để hoàn thiện thủ tục thôi. IELTS vẫn là chứng chỉ được công nhận toàn cầu và vẫn được yêu cầu bởi các trường đại học trên thế giới. Bỏ IELTS để đi lùi với thế giới à?": 0,
"Vẫn đang phân vân xem có nên đảo k. 🤣": 0,
"Cơ trưởng này lái ok lắm ấy, hôm bữa đi tra trang mà hạ cánh nhẹ quá làm tớ ko cảm nhận đc máy bay đang hạ cánh ấy": 0,
"Optimusthree với 99% sức mạnh": 0,
"phương thục đình  có đâu :v": 0,
"Em mình làm ở thẩm mỹ viên này đây": 1,
"Đăng ảnh đành phải xoá thôi": 0,
"tộc caaaa": 0,
"Chồng đưa tiền cho nhiều đi còn tất cả để vợ lo kkkkk": 0,
"TẠI DÂN THÍCH LÀM NÔ LỆ .CHƯA THÍCH LÀM NGƯỜI .ÔNG TRỜI CHO TỰ DO QUYẾT ĐỊNH .TẠI SAO KHÔNG CHỊU LÀM NGƯỜI MÀ LÀM CHÓ GIỮ CHẾ ĐỘ CSVN": 0,
"Thôi thì em vẫn ở bên Vùng ký ức vậy 🤦♀": 0,
"Mạ vàng để trong ví nhưng t làm rớt cái ví rồi giờ sao đây": 0,
"Trời ơi, con này là Threshershark , một loại cá mập đuôi dài cực kỳ quý hiếm. Sao có thể dạt vào sông dc.": 0,
"Thường thì đa phần họ ko chấp nhận cái mới ngay đâu, nó thuộc về trải nghiệm sống và thời đại của mỗi thế hệ lại ra một chất khác nhau. Có người cảm nhận đc và không. Thích hoặc không phù hợp": 0,
"Óc chó kiểu italy😅😆😄😃😂": 0,
"Mọi cái sai lầm đều phải trả giá băng tiền. Học phí cao nhưng may mắn ko phải nuôi con thằng khác": 0,
"Cơ hội cho côngn chúa sang ả rập đấu tranh nữ quyền rồi.": 0,
"Mua di t đọc ké :": 0,
"Anh châu cho em xin nich fb cua anh để em đt gặp cô kim nc 😂❤️": 0,
"Chào bạn Bạn vui lòng nộp hồ sơ ứng tuyển trực tuyến nhé": 1,
"Hai chi em xinh lam.": 0,
"Nemo đang bận ôm chị khanh nha": 0,
"đáng nhẽ nên làm việc này từ sớm . chứ hàn quốc họ làm từ lâu rồi ạ": 0,
"Từ hôm qua đến giờ thấy các báo toàn giật tít tào lao. Do những năm gần đây lượng thí sinh tăng cao, các đơn vị tổ chức thi (BC & IDP) đã liên kết với các trường đại học, trường cấp  làm địa điểm tổ chức thi. Tuy nhiên Bộ GD yêu cầu các đơn vị liên kết tổ chức thi IELTS phải có đầy đủ giấy tờ của bộ. Bây giờ người ta đang tạm dừng thi để hoàn thiện thủ tục thôi. IELTS vẫn là chứng chỉ được công nhận toàn cầu và vẫn được yêu cầu bởi các trường đại học trên thế giới. Bỏ IELTS để đi lùi với thế giới à?": 0,
"Vẫn đang phân vân xem có nên đảo k. 🤣": 0,
"Cơ trưởng này lái ok lắm ấy, hôm bữa đi tra trang mà hạ cánh nhẹ quá làm tớ ko cảm nhận đc máy bay đang hạ cánh ấy": 0,
"Optimusthree với 99% sức mạnh": 0,
"phương thục đình  có đâu :v": 0,
"Đăng ảnh đành phải xoá thôi": 0,
"tộc caaaa": 0,
"Chồng đưa tiền cho nhiều đi còn tất cả để vợ lo kkkkk": 0,
"TẠI DÂN THÍCH LÀM NÔ LỆ .CHƯA THÍCH LÀM NGƯỜI .ÔNG TRỜI CHO TỰ DO QUYẾT ĐỊNH .TẠI SAO KHÔNG CHỊU LÀM NGƯỜI MÀ LÀM CHÓ GIỮ CHẾ ĐỘ CSVN": 0,
"Thôi thì em vẫn ở bên Vùng ký ức vậy 🤦♀": 0,
"Mạ vàng để trong ví nhưng t làm rớt cái ví rồi giờ sao đây": 0,
"Trời ơi, con này là Threshershark , một loại cá mập đuôi dài cực kỳ quý hiếm. Sao có thể dạt vào sông dc.": 0,
"Thường thì đa phần họ ko chấp nhận cái mới ngay đâu, nó thuộc về trải nghiệm sống và thời đại của mỗi thế hệ lại ra một chất khác nhau. Có người cảm nhận đc và không. Thích hoặc không phù hợp": 0,
"Óc chó kiểu italy😅😆😄😃😂": 0,
"Mọi cái sai lầm đều phải trả giá băng tiền. Học phí cao nhưng may mắn ko phải nuôi con thằng khác": 0,
"Cơ hội cho côngn chúa sang ả rập đấu tranh nữ quyền rồi.": 0,
"Mua di t đọc ké :": 0,
"Anh châu cho em xin nich fb cua anh để em đt gặp cô kim nc 😂❤️": 1,
"Chào bạn Bạn vui lòng nộp hồ sơ ứng tuyển trực tuyến nhé": 1,
"Mua cho t hả😆": 0,
"Lập acc bản Trung khó quá c ơi": 0,
"Hà Minh Sơn đồng hương with love :))": 0,
"Xinh mà bé qá hạnh ưi": 0,
"Ăn sương sương chậu mì tôm vs vài ba con cua hoàng đế thôi :))": 0,
"Yêu thì yêu k yêu thì thôi sao mà phải độc ác nv . Đúng saii , ba mẹ ngt nuôi ngần ấy năm trời để cho lũ cm yêu đg xong giết chóc kinh vch -((( yêu lý trí nên các con ng máu lạnh ơii 😓😓": 0,
"De thuong qua Ah kk": 0,
"Xem my đc  tháng rồi sao ko đc lm phan cứng": 0,
"Hoc mat nhieu tgian k a": 0,
"mình mới tiếp xúc và bắt đầu tự học nhưng đã thấy ứng dụng rất tốt. mong admin sẽ phát triển ứng dụng tốt hơn nữa. cám ơn": 0,
"Zô đây camping là đúg bài lun ák m.n": 0,
"Cách đây 2 tuần qua streaming house gặp anh quang ngọc trinh đưa a grab ship áo còn a tú đụ cởi trần mở cửa cho e gái": 0,
"bxhhxbxbdbdn dbdbdnfnfjdjxnfjjdjxnxjdjzjdjjdjdjdhdhdhd": 1,
"Tôi không chơi GI nhưng nếu game lên anime thì làm theo cốt truyện trong game à (:?": 0,
"Mon Monn Cường Trần Tuc Vu Hoang  Hoàng Lâm  zi đi": 0,
"Ai chụp mẹ con t dẫy. 😂😂": 0,
"đông lào là gì vậy": 0,
"Ai cũng như vậy thì dịch gì đi nữa cũng ko sợ🤗": 0,
"Soi gương cũng không đến nỗi tệ nên tự tin cầm máy lên selfie xong tâm trạng buồn hẳn luôn.": 0,
"T1,tranh chấp ở 1 vùng chiến lược như biển đông không phải là chuyện đơn giản,mặc dù hàng ngày húc nhau ầm ầm,đặc công mình và người nhái của nó xin nhau tí tiết là chuyện thường ngày ở huyện nhưng trước mặt bàn dân thiên hạ việt nam đang đóng vai thằn": 0,
"Mới phủ bóng xong nhanh tay đi qc dove liền🤣": 1,
"tao thấy dễ thương màaaa": 0,
"Woa quả giọng nước cốt dừaaaaa": 0,
"Games sau kiểu ép chết làm quả top8 trừ 96đ đau ko chịu đk.kkk": 0,
"Bảo Ngọc  hỉu lun nè he 😂": 0,
"Pito Pito cứ chờ mood thế nào đã ": 0,
"Ok ngon bổ rẻ": 0,
"Tội nghiệp anh zai🤣": 0,
"Mình vẫn chưa biết dùng bật lửa và giờ thì 🙂": 0,
"Thôi với cái đà này là giáo sư sẽ chơi cùng anh Minh nhật rồi": 0,
"Mà cuốn sổ này kiểu phải tỷ năm rồi mình không đụng vào, quên luôn sự tồn tại của nó rồi. Chả biết ai xui mà tiện tay mở ra, nếu không mở ra xem chắc chả bao giờ phát hiện =)))": 0,
"ngta ko nói thì lo mà trả đi nha": 0,
"Mèo nhà nta ngoan vậy còn mèo nhà tôi 🤦♀😵💫": 0,
"Đen thôi đỏ là red🙂": 0,
"Thề thích xem a gáy vãi": 0,
"Minh luc co cai cam nhin vo la biet sua.  Nhung xau qua  nhon. Nhin la biet cam gia. Xau. Qua.": 0,
"Em học thử trên youtube xem. Nhiều kênh dạy massage cơ bản đó. Hướng dẫn cũng rất chi tiết, nếu em học để massage thư giãn thì tài liệu trên youtube khá đầy đủ rồi. Còn nếu ng yêu cần trị liệu j thêm thì chắc phải ra ngoài học. Gần rmit thì có nhà văn hoá phụ nữ cơ sở  cũng có khoá massage cơ bản ý.": 0,
"Ad khai thiệc đi": 0,
"Chốt quần xinb 48": 1,
"Sáng em ch ăn j lun": 0,
"Nguyễn Cẩm Hường thuyết phục🤣🤣": 0,
"Vưa mơi găp môt ông phat ngôn câu đây xong ma cau :)": 0,
"🐰mng nên mua nha ": 1,
"Tất cả chỉ là bất ngờ": 0,
"gia hàng nhanh,  sản phẩm  đúng mô tả chính hãng, da mụn nên mua xài lần đầu. hi vọng có kết  quả  tốt.  cám ơn   cửa hàng ": 1,
"idol em cj ưiiiiiiiiiiiiii": 0,
"Nguyễn Hương Giang hơ hơ": 0,
"về stream nhanh còn kịp anh ei😂": 0,
"Dương Kỳ hắt xì vẫn toang =))": 0,
"Bức ảnh cuối thấy hạnh phúc làm sao, giờ đất Sài Gòn hiếm có cảnh gđ con cháu quây quầng bên ánh lửa hồng như vậy lắm. Tuy họ nghèo tiền bạc nhưng mình cảm nhận họ rất hạnh phúc, điều mà có khi chúng ta mãi tìm mà chưa chắc đã được. Chúc gđ anh nhiều sức khỏe và ổn định cuộc sống.": 0,
"Gòi sao lấy chồng ta": 0,
"Bảo vệ trường ngta =)))": 0,
"Về vị: cá nhân mình thấy không cay mà là the, vị the của các loại gia vị ấm nóng chứ không phải the bạc hà. Vị chua rất nổi bật. Topping không nhiều. Lúc ăn tự thêm xúc xích/ trứng/ rong biển/... sẽ ngon hơn. Mình nghĩ tuỳ người sẽ thấy ngon hay không.": 0,
"Nam mo quan the am bo tat Nam mo quan the am bo tat Nam mo quan the am bo tat Nam mo quan the am bo tat Nam mo quan the am bo tat": 0,
"Loa zy898 phiên bản mới và thẻ bao nhiu tiền z?": 1,
"bà hà thâm hơn dái chó 🤣🤣": 0,
"Chốt áo đẹp ♥♥🍀": 1,
"Bộ Y tế đề xuất phương án chống lây nhiễm chéo trong khu cách ly tập trung tại TP.HCM": 0,
"ah á đâu chế ới": 0,
"Lê Minh Nhật mặc kệ miệng đời và ăn thật nhiều": 0,
"nhúng nước được k": 0,
"Tâm thần cho nghỉ việc ,không đề ảnh hưởng đến ngành CA!": 0,
"Đúng chất câu trong bài hát của a ý mà 😆😆": 0,
"Chưa từng thấy câu hỏi ứng xử nào xu như câu của Mai ngô, thở cái biết câu hỏi lạc đề liền": 0,
"Giac phuong bac ma": 0,
"ôi mùng hai tết r hã": 0,
"Khi lấy kem mình hơi ngạc nhiên vì chất kem màu hồng tím nhẹ, không phải dạng kem trắng như đa số kem chống nắng thông thường. Khi thoa lên mặt tiệp vào da và ráo mặt. Mình khá hài lòng.": 0,
"Thanh Ngân hợp lí nè": 0,
"Bún Lèo vây lại ngon ý anh": 0,
"Lưu Thị Tài vinh dự quá. Mình là  thiên thần rồi😂": 0,
"phải 7 t ko chị êy .": 0,
"nhìn mặt chó ông nhà báo là thấy hãm tài": 0,
"Đi làm nhớ mặc ấm vào bạn nhé. k rét cả ngày lại khổ 🤣": 0,
"lôi súng ra bắn 1 phát": 0,
"Mất bao lâu để quên đi  ng đây 😊": 0,
"Em được dạy từ cơ bản em nhé": 0,
"đúng rồi đấy , như thế là đang mắc chứng ảo tưởng sức mạnh": 0,
"@Phú Đại GMTV vậy mày vô đây sủa làm cc gì tml": 0,
"hôm đấy trên đường đi làm về": 0,
"Lê Văn Quật chịu luôn :>>": 0,
"Ảo thuật": 0,
"Kịch bản giống đoạn đường tăng trong đại thoại tây du của châu tinh trì v": 0,
"SP đẹp .. hợp túi tiền mọi ng nên ủng hộ shop nhe": 1,
"Ok hfsfjhggggtyyy 👍👍👍👍👍👍vvvvvvvvvvvvvccc": 1
}
{
"Quynh Anh giờ bay ra đà lạt còn kịp em": 0,
"Liênn tang con binh đat vao😂😂": 0,
"Hello anh thanh duy Hello chị khởi my": 0,
"Mai mới có phim ạ": 0,
"Đã đủ mẹ mít ạ 🤣🤣": 0,
"CUỘC ĐỜI E CÒN ĐANG DANG DỞ  A BƯỚC VÀO CHE CHỞ ĐC KO": 0,
"Chơi vs pro thy như boss": 0,
"Lúc trc nó lập trang này ra k vấn đề j , mỗi lần nó đăng là cả mấy nghìn lượt like , giờ bạn thấy đấy , theo con đường này giờ nhìn lại nó có 88 cái like , thôi thì mình cũng bố thí cho nó 1 like coi như công pót bài .. Haha": 0,
"Ra bài của wean đi ạ:))))": 0,
"Việt Hải chuẩn": 0,
"Ở nhà  mình vui mà": 0,
"Duy Ho thôi bỏ đi a :)))": 0,
"Phạm Kiên may mà có e đi chơi vs a nhé": 0,
"Gái rờ mích ny trường ngoài đây bạn :": 0,
"mua chiếc này đi xuyên việt ngon nè, tuổi trẻ nên đi  lần trong đời trước khi thoát vị đĩa đệm nhé": 0,
"Nước bao la như thế con không ở đâu": 0,
"còn yêu nhưng vẫn rời đi": 0,
"ngọt hơn cả người yêu haaaaa": 0,
"Gấu  gấu   chào  nha": 0,
"Mới nghe đc mấy tiếng trước 😂": 0,
"Nguyễn Ngọc Chung sẽ trân quý lắm": 0,
"Minh co gui so dt roi , goi minh tu van dc kg 94578": 1,
"Hôm ấy chắc đông bẹp dí. Tắc đường đi về": 0,
"e ở campuchia ma": 0,
"Năm nay Argentina có dàn cầu thủ quá hay và đồng đều ở các tuyến, các nhân tố chủ chốt biết toả sáng đúng lúc, đúng thời điểm tạo thành  đội có lối chơi công thủ rất đẹp. Tôi yêu Argentina và FANs M 🏆": 0,
"Ok thầy": 0,
"Những shop đồ của mị =)))": 0,
"Giang Bean bài ni hay này téng ơi 🙆": 0,
"Trần Dần bị đồng tính thôi": 0,
"Chào chị, dịch vụ xóa nốt ruồi có phí dao động từ ,  ,/  nốt, tùy theo kích cỡ nốt nếu nốt không lồi. Mời chị dành ít thời gian ghé qua mỹ viện chuyên viên sẽ kiểm tra và báo giá chính xác cho chị nhé.": 1,
"Kim Tiêm": 0,
"Tí có team code ko chị": 0,
"Có ai xứ nghệ không chị ơi": 0,
"Trịnh Văn Trường clm xem tấu hài thực sự 😂": 0,
"Tối nào cũng vậy nha 😆😆": 0,
"Tính dành dụm niềng mà s thấy nản quassss": 0,
"Ghé t xem nhiều phụ kiện xinh lắm nè": 1,
"Viet Huu": 0,
"Bát ăn dặm bé gái 9k": 1,
"Gần 4h rồi chị ơiii": 0,
"Giao hàng siêu nhanh, đóng gói cẩn thận, túi đẹp, giá sale rất ưng luôn": 1,
"Q còi vs Q nổ": 0,
"môn thấp điểm nhất đời hs =.=''" : 0,
"Nếu thời gian rảnh nhiều thì mua thóc về gieo rồi chăm cho nó lớn rồi gặt rồi xay vỏ ra lấy gạo nấu ăn luôn bạn ạ. Tha hồ ngon 😋😋😋": 0,
"game nhiều  quả người cáo thì tắt mạng đi thôi": 0,
"Không nghe thầy phán chỉ có chết thôi mọi người ạ": 0,
"Các em nên nhớ:trượt tốt nghiệp hay k đỗ đh chưa hẳn là hết hi vọng.biết đâu sau này mấy đứa nó đỗ đh nó lại là lính của các em": 0,
"Thế này mà vẫn còn anh à": 0,
"chơi chiêu nha ,ngày đầu vô làm là có lì xì nha 😄😄😄": 0,
"Dư Bùi hihi ☺☺☺": 0,
"không phải ai cũng vậy đâu nhé :))))": 0,
"Xin tránh nói thêm về tương lai": 0,
". Vào trễ huhu": 0,
"Tank bây giờ mạnh sát thương mà:))": 0,
"Giờ ra chơi của nhóc Nicholas.  Những ngày mệt mỏi, nhất là lúc áp lực, cứ đến với Nicholas là vui nhất. Đúng như tựa sách nói, trẻ con thì thấy giống quá mà người lớn thì thấy nhớ quá.": 0,
"Mi Lê có có. Mà sợ ông dôn Châu Tiến Dũng k chịu hợp tác nì": 0,
"Diep Pham Thi set như mẫu mặc đây a": 1,
"Nhung Kún kaka. A trả lời toàn câu độc mà nó ko chèn vào.": 0,
"Nguyễn Thị Mỹ Hằng. Tết về t tặng cho :)))": 0,
"Dm michos , làm t tạch rate bn lần": 0,
"Bó tay con này😞": 0,
"Thử động đến tao xem.": 0,
"Trồi má mí xinh vl": 0,
"Mặc nó hâm nóng ko sop áo khoác lông cuu": 1,
"Mất cái nhầm màu thì phải, nhìn đâu cũng không thấy giống 'NÂU'. Tks!": 0,
"Phan Hà My oh no u too": 0,
"Đây nữa nè ađ kkkk": 0,
"Lê Thị Cẩm Ly được chưa em": 0,
"Ma 5 mặt bấm nhạc sau cam dt  dc": 0,
"Chưa thử loz xách tay bao h,có thì ới 😂": 0,
"Trung lập xem bóng đá mà quay qua anti Cro luôn. Đá đấm cầm hoà nhìn phát bực.": 0,
"Í nay có fc lại r nèee": 0,
"Em cũng đang đau bụng nek T^T": 0,
"công nhận, bài này cuốn vãi nồi :))": 0,
"dịch vụ của Ninja Van ai đã dùng là đều ưng ý hết nên bạn cứ yên tâm nhé": 1,
"Giao hàng nhanh đóng gói cẩn thận chắc chắn": 1,
"có suất h với h5": 0,
"ối dồi ôi, u mê quá": 0,
"Ad đào Pi hay sao mà có Pi cuối câu dị :))))": 0,
"móc ngửi đấy má  dơ vcl 🤣 🤣": 0,
"giao rất nhanh, đóng gói kỷ và đẹp mắt, sản phẩm dùng ok phù hợp với giá tiền.": 1,
"Đáp án: có  chiếc quạt hút Panasonic trong hình. Thanh Hằng": 0,
"Chị nhà cháy quáaa": 0,
"Mịa apple làm loạn hết cả lên. Arm đang yên ở mobile lên lap làm loạn": 0,
"Sợ quá nhỉ mình cầu mong mọi người ko sao tốt lành cho dân Hb": 0,
"mùa thu năm ấy cta đã gặp nhau ?": 0,
"1 là jason statham": 0,
"9656979, mã số  xuồng 7": 1,
"Trailer phim gì mới đây?": 0,
"T chỉ cần cầm đt ck t ' thôi là có gì t lột ra hết .": 0,
"4A ny ơi :p": 0,
"goi e i c my": 0,
"Nếu,na, gỗ,huong, bao, tiền": 0,
"Mai Hoài Anh để bữa nào anh sáng rước em hé": 0,
"Hàng đẹp": 0,
"làm cái đéo gì mà lại lâm vào tình huống như thế này": 0,
"Lê Quỳnh Như mày nói chiện liên quan quá ha con khùm": 0,
"Đây mới là ứng cử viên sáng giá :))": 0,
"Truong van Trung  thế ở trần cho đỡ tốn e aii": 0,
"Mua ở đâu mấy ký": 0,
"thế đấy dkm": 0,
"dạ vâng ạ chị cho shop xin địa chỉ để đến nơi shop báo phát cho chị luôn ạ": 1,
"Đứa m6 đứa m75": 0,
"đồng nai mùa chim nướng yên xe kaka": 0,
"Anh Gi mà tổ chức show thì em cũng xin hứa đi xem không sót buổi nào": 0,
"hỏng rồi . dùng đồ như phá": 0,
"dàn pc này sau này cũng là của t ngu gì mà đập": 0,
"Dồn đon với đầm Nhung đỏ hôm wa nha em": 0,
"phút bốc đồng  đời bốc cức 🙂": 0,
"Dẻ yêu quá": 0,
"Mưa thì đắp chăn đi ngủ chơ yêu đương cái gì :))": 0,
"gởi địa chỉ củ cho chị": 0,
"Má cười chắc khùng luôn": 0,
"Chủ quán: Riết gòi tới cái bát cơm cũng báo trời thần đất lở:))": 0,
"lô Mít ô hô hô": 0,
"like nha anh": 0,
"Vincom thái bình có tổ chức k ạ": 0,
"Hàng đẹp nma chất vải hơi nóng tí hành may đẹp nha mn nên mua hâhhhadnjdjsnsnsnndndndbdbdbdhdhdbbdbxbxbxbbxbxbxbxbxbxbxbbxbxbxbxbxbbxbxbxbxbxbxbxbbxbxbxbbxbxbxbxbxbxbxbxbxbbxbxbxbhsbsbdbbdbdbdbdbdbdbdbdbdbdbbdbdbdbdbdbdbbdhgvhhhbbhbgggggvvvvgvcvvvvvvvvvvvvvvhdhdhhdhdhfhhhfhfbfb": 1,
"Omo này mình xài cũng lâu rồi, hàng tốt không cần review lại ha.": 1,
"thời gian khởi động lên đến 5'nhưng hoạt động mạnh mẽ chỉ 'hơn": 0,
"sản phẩm đúng mô tả, giao hàng nhanh, đóng gói cẩn thận, sẽ ủng hộ lần sau.": 1,
"Hàng Hàng đẹp giá rẻ Hàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹpHàng đẹp": 1,
"Thúy Ngân thấy khó quá m": 0,
":333333333333333333333333333333333333333333": 0,
"Ôg vào đây làm gì 😂": 0,
"Chị đó xinh vậy :": 0,
"Chach cao de  may duoc ko ạ": 0,
"các tiền bối dịch hộ em với ạ": 0,
"tôi một người xài 5 năm facebook lite rất là thích. rồi càng ngày càng tệ. mỗi lần bị đăng xuất hay lí do gì không biết vào ứng dụng rất là lâu. có khi một ngày trời vẫn không vào được cứ một chấm đang tải trong khi wifi rất mạnh. tốc độ thì vẫn vậy cải thiện đâu chả thấy, càng tệ thêm.": 0,
"Khẩu hiệu tuyệt vời": 0,
"Nói miết mà không sợ Trúc Văn Võ": 0,
"Phi Yến mở công viên hồi nào á 😝": 0,
"Đến cốc nước còn thiếu thì lương của tôi không biết là CTY có trả đủ ko": 0,
"lo làm kiếm tiền đi nha mấy con giời suốt ngày văn vở, ra dẻ biết điều này kia nhưng lại ko hề biết điều nha": 0,
"Có đk của đầu VTV Cap k": 0,
"túi có size cho các cỡ vali ko ạ? giá cái lớn bao nhiêu ạ. cám ơn shop": 1,
"Cường lực vừa, dùng mượt nhưng bị thiếu miếng dán bụi làm mình bị dính 1 hạt bụi vào mà không làm cách nào lấy ra được. Nói chung với giá này thì cũng ok không có đòi hỏi gì thêm nhưng mong shop cho thêm miếng dán bụi để dán được mượt hơn": 0,
"Chị sao chị xinh vậy": 0,
"Máy mỏng nhẹ, thêm chức năng so với bản cũ nên sử dụng rất thích. Mình đọc nhận xét thấy 1 số bạn báo máy hay bị lỗi cảm ứng nên cũng hơi lo. Để mình sử dụng 1 thời gian xem, nếu có vấn đề gì mình sẽ báo lại để các bạn tham khảo.": 0,
"Cân bằng  chuyện rất khó. Sẽ dễ dàng nếu gặp một người phụ nữ thấu hiểu, quan tâm mình.": 0,
"Cầu mấy thằng t ghét  đá thua lòi cl :))": 0,
"Tùng hỏng :))))": 0,
"sao k chào e hí hí": 0,
"Thì thôi a sẽ ở lại 😞 trộm nhìn theo e tháng năm dài 😭😭": 0,
"Thanh lý tr mua 6tr nha các mom": 1,
"hồng nhan bạc tỷ :)))) còn không hồng nhan thì bạc phận": 0,
"LÀM TẤT CẢ vì người mình yêu 🐧": 0,
"tôi chụp cho bạn đẹp bạn chụp cho tôi hết hồn": 0,
"Phượng Đỗ ghim lại này 🤣": 0,
"add xem ảnh ngoài đời chưa mà nói dume": 0,
"Nhưng ma cung không chê tranh cua Haixia Liu, thưc ma ky thuât cung điêu luyên nưa.": 0,
"Llongyfarchiadau Well done Xx": 0,
"Không nên mua đt trung quốc bạn ạ, dùng được mấy tháng phát ức": 0,
"#nhungtamchieumoi": 1,
"may mình xem rồi =))": 0,
"ba vì thì đi  ngày chơ  ngày mần ci trên á :": 0,
"Đj choi ha c": 0,
"Anh mà xấu như em xem để tóc nào cũng thế à :(": 0,
"Lở nói thì bán đi chị": 0,
"Chan Kim Đặng Luân của m": 0,
"THẨM MỸ VIỆN MAILISA cam on chi": 1,
"chưa bao giờ nghe ngươ ̀ i ta chửi thề mà đáng êu đến vậy lun =))!": 0,
"Tôi TIN TƯỞNG, TIN TƯỞNG": 0,
"Người Miền tây tôi, ăn chén cơm đầy, uống ly rượu đế, để tiền mua vàng đeo": 0,
"Xin lỗi shop vì hình mang tính chất nhận xu. Hàng chất liệu đẹp chưa mặc thử vì mua nhầm size nhưng rất vui vì shop hỗ trợ đổi size. Ủng hộ shop": 0,
"Mỗi ngày trôi qua cậu chỉ mong sao mở mắt ra đã thấy mình hoá vào thinh không, chẳng còn buồn khổ, chẳng còn đau đớn, chẳng là gánh nặng cho ai cả nữa.": 0,
"Mấy thằng Hack Cũng rất gì và này nọ =))))": 0,
"Yojan Soy yo o tiene un parentesco con el de Xbox": 1,
"Nguyễn Tiến Kim Thu có mặt k": 0,
"Coi phim mệt tim ds mệt": 0,
"Tìm anh chạy xe Winner X cùng về Gia Lai": 0,
"Nhật Quang tớ về cậu mua cho tớ": 0,
"Not e ch ah": 0,
"game như nagative khuyên mọi người không nên tải": 0,
"để em mua choa": 0,
"Mình ở cầu giấy bao nhiêu tiền síp": 0,
"Vẫn là chính mình. Tự tin bước tiếp": 0,
"Mạnh Thắng Trần Nguyễn Đức Hoàng ko đi tơi ở cầu giấy nha :((": 0,
"Anna Trang đừng làm vậy nha anh": 0,
"Giao hàng nhanhhhh": 0,
"Linh's Kun's lỗi lòng của ck": 0,
"Trai bắc đây :<<": 0,
"Chị gái tên L may mắn thoát được vài kiếp luôn 😢 nếu lúc đó hung thủ có ở nhà thì die, mà về lâu về dài có khi lại thành cô bạn gái cũ thứ , cũng die 😢": 0,
"Có b nhé. b hỏi màu nào ạ": 0,
"Khốn kiếp": 0,
". Anh vẫn rất tỉnh :))": 0,
"11 ngày mới nhận đc, nt hỏi giày ko tl, hỏi sao chưa có hàng cũng ko tl, hỏi quá thì dạ vs vâng cho có.": 0,
"Supi Trần hự hự như một cắt vào tâm hồn em.": 0,
"Công nhận team bạn ý chí kiên cường kkk": 0,
"máy rất tốt, dùng mượt": 0,
"Khi biết giá trị của chữ Tiền thì trong túi đã không còn một xu": 0,
"Thanh Xuân có ai mới sinh ra biết cấy đâu.lên chỉ cho là biết liền mà": 0,
"Hoi cho hồng ngu": 0,
"mày đừng bốc phét mày đừng mồm điêu": 0,
"Bag tuoi di hen kevin khanh ha": 0,
"thầy dạy lý thuyết kỹ .": 0,
"Hiện em còn size 5 và 9 chị nha": 0,
":(((( coi hong đã": 0,
"Sao trong mv không có sự góp mặt của mẹ nuj": 0,
"Bánh này nó lạ lắm": 0,
"Nghiêm Lê Hoàng Danh Vo Truc Ly Thiên Lee Tuấn Khải Ngọc Long Kim Son Phuc Nguyen": 0,
"Shop xứng đáng 1000* nhá": 1,
"Hôm nọ còn đòi cấm internet nga nữa mới ngáo :v thực ra  số thằng nó cũng chả muốn cấm đâu nhưng bị sức ép của phương tây quá nên nó bắt buộc phải cấm": 0,
"5k/ ngày là quá nhiều rồi :))))))": 0,
"Chương trình dự thưởng kết thúc vào / hay là /": 1,
"demain j'achète un camping car": 1,
"A lên ngọc gj mà đáng rừng hồi má vậy anh": 0,
"ủng hộ abe ủng hộ abe": 0,
"- Máy tiện lợi chỗ có mức max cho dễ canh chỉnh nè! Dễ vệ sinh lưỡi dao nè! Rồi có quay xách nữa. Mà nặng có 450gram thôi.": 0,
"Nhun thấy t k": 0,
"Gáy thì thua thôi": 0,
"Thua tàu hết Mấy năm nữa xe TQ sẽ tràn ngập": 0,
"Scarlett Pham Bánh Tráng BơNguyễn Vũ Phương Vy năm ngoái ăn ở lớp cô thương 😭😭": 0,
"Lê Phương cho mk xin 5k đi ăn xiên hàng loại  đi =)))))": 0,
"Chuyển tn? Có cần thẻ hay gì k": 0,
"Nhìn làn da muốn khóc thét": 0,
"i remember friend you": 0,
"Có bán hạt chia k c": 0,
"đm phí hết sức :(( ": 0,
"Giá rẻ, đóng hộp đẹp mắt.": 0,
"Thì đi với anh cho vui": 0,
"Nam chính MV kìa bóng dáng ai official :))": 0,
"Chuyển qua insta thì hack bằng lol,bằng niềm tin": 0,
"Nhìn anh Tân giống ninja quá": 0,
"e sắp thi chuyển cấp rr chị thy chúc e thi tốt đy <": 0,
"Ông HOÀ BINH ơi Sao lại ra đề thi này cho các GIÁO SƯ TIẾN SỸ SỬ HỌC? Các ông ấu khg phải vừa đâu. Còn tất cả dân VIỆT thấy rằng Chính quyền Dân bầu là của Dân và vì dân, là Đại diện cho một chinh thể Quôcs gia  Việt Nam chứ. Còn bộ máy Bù nhìn do ngoại bang dựng lên thì làm sao đại diện cho Dân tộc VIỆT NAM được?.": 0,
"Quá đúng lun ko sai chút nào ❤": 0,
"Bỏ chắc chắn nhiều hơn Thu": 0,
"bạn gái trong ảnh lưu hình gửi cho công an xã chết con mẹ bọn nó đi :)))": 0,
"Thế là cuối cũng cũng ko thể vạch mặt hết ra đc. Bày đàn quá mà": 0,
"Set 6 và Set 6.5": 0,
"Hí =)) thế sao lúc mng chê bạn ko nhắc lại lời mng =)) chê cái j nấu cho mà ăn là tốt rồi=)))": 0,
"Lần đầu tiên comment bài chị Na, hi vọng chút may mắn và xinh đẹp từ c 😍": 0,
"Nguyễn Hoàng Đức Huy Thanh Hoàng Hoa Hieu Nguyen Hồng Vân": 0,
"Jodbekeodbdkekdbdkfkf": 0,
"Má 5 có ăn chay hk c": 0,
"Tuy rằng mua hàng online, nhưng hàng đúng chuẩn cà phê highlands mình yêu thích nhất, mùa dịch chỉ cần ở đặt hàng tiki giao tận nơi, còn gì bằng nữa cả nhà ơi 💖": 1,
"Có e nek chi my": 0,
"Còn đây là tôi đang dọn nhà thì gặp  bạn tổ buôn. Nên cũng tiếp chuyện  lúc tầm 45 phút": 0,
"9 - đt 8975768": 1,
"Thanh Tung Dam ja": 0,
"Đang bận sáng tác nhạc ⚜👻⚜": 0,
"Nhii Tâm rất xinh xắn đáng yêu 😍": 0,
"Làm mẹ của  bé rồi, bạn hãy vững tâm lên. Mỗi người có ý chí tự do của riêng mình, nên ông bà nội có nói gì thì quan trọng vẫn ở phía bạn. Nếu ô bà già k trông dc cháu thì bạn hãy nghỉ thêm một time nữa cho con cứng cáp hơn rùi hãy đi làm. Giờ con vẫn nhỏ bạn đi làm k chăm dc con bệnh lại vào viện(đồng lương của bạn hay chồng bạn chi trả cho chi phí đó).": 0,
"Nguyen Que Huong Bạch Quân Lãng Ty Trần": 0,
"Đúng ý t ghê 🤣": 0,
"Hay haz, hay mụ giải vờ chơi les crush thời vụ tôi cho tôi hít hở tí không khí có ny đi nào, vã lắm r :)": 0,
"Cho tao xin  xuất đi 😀": 0,
"Thiên Bình 7/ đi làm xong về royal ngồi ăn uống chơi bời đợi đến khung giờ vàng luôn =))": 0,
"Là tui đó, cả  người, vừa design, vừa marketing, vừa sx content 😞": 0,
"996 hoặc 997  thời điểm ấy Ba Mẹ tôi khoẻ mạnh và đẹp nhất": 0,
"bây giờ dịch bệnh phức tạp, ai muốn thư giã đầu óc và hạn chế tiếp xúc với mọi người thì lên GoJoy book phòng nha.": 1,
"Phương Mỹ Lê, nếu được may mắn phần quà này , tặng người iu set dưỡng da và nước dưỡng da nha.Cho ng iu da đẹp lại như xưa 😊": 1,
"Quả màn hình phụ không ngon bằng flip": 0,
"D 4 Nhã Xuân Lâm Long Nghi Hồ Trần Trần Long Nghi": 0,
"Chưa kịp bắt nó bay lại kẹp bay trym m luôn đấy :)))": 0,
"năm trước từng lặn lội gió mưa đến sân bay tân sơn nhất để đón, idol vẫn trẻ mà mình thì sắp có con dâu rồi": 0,
"Mỗi lần nghe nhạc của Vũ t lại nhớ đến  ng khi t và bạn ấy đều rất thích nhạc của Vũ": 0,
"Nói chung là chất lượng sản phẩm ok, đóng gói đẹp và chắc chắn, giao hàng nhanh !!": 0,
"cam có ngon bằng  cam ko nhể": 0,
"Ngọc Tùng phải chi cái áo nó màu tím :)": 0,
"Giọng vẫn trêu đùa vl , nghiêm túc tẹo đê :)))": 0,
"em chờ được mà 🙂": 0,
"Màu son: mình order số 27, màu Hồng tím kén người dùng quá.": 0,
"Nguyễn Nhật Linh hay sau ce mình cx học tập để đc gầy =)))": 0,
"Tuận Đình Nguyễn vâng bác ạ đúng là con sâu bỏ dầu nồi canh": 0,
"Tóc c nhuộm màu gì đp thee k biết ☺": 0,
"Mùi Hoàng chuan bj giau roi": 0,
"Vui mừng khôn siết, má dùng từ đúng và đủ luôn.": 0,
"Có kẻ mắt ko chị?": 0,
"Ao dai eo 74 nha": 0,
"Nhớ nghỉ hẳn luôn nhé ☺️": 0,
"lọ nhỏ hơn mình nghĩ nhưng chất lượng tốt, mùi vani rất thơm luôn.": 0,
"ngân acp nick pubg em với được không😢": 0,
"Mang Các giáo án từ việt nam ra thế giới": 0,
"ko khám cho kỹ, rồi người ta bị sida cũng lấy máu luôn.😑😑": 0,
"Đổi bài khác đi chị": 0,
"Giải lao quay simple, hải, híp đi chị :v": 0,
"BỘ ĐỒ Y CHANG E =))": 0,
"Hk chs ff hả cj": 0,
"t cũng là fan ruột phim hollywood , đam mê truyện trinh thám, cũng sở hữu  trí tưởng tượng phong phú. thế nhưng việc t làm được là làm cách nào để mai có thể xin nghỉ làm . Thật sự hổ thẹn": 0,
"Maria Eduarda Silva querroo veeer😍": 1,
"Ở Việt Nam thế là tuyệt vời lắm rồi": 0,
"Honggg bé oii, honggg lên đouu nha🐒": 0,
"Cháu cũng làm con cặc j.": 0,
"VPBank da sd nhung Dk online k dc": 0,
"Thông minh quá cũng tự hại mình": 0,
"văn toàn xem kìa :))": 0,
"Phan Nguyệt Trần Huyền Trân thân gửi": 0,
"Với mua cho a thêm  gói BH nhân thọ có kèm chống thương tật đi ạ.": 1,
"Mimon Hóc Số họ bày ăn chuối tiêu mà hồi giờ c toàn ăn chuối mốc. 😂😂😂": 0,
"Quả cuối nhìn tưởng abe": 0,
"Sản phẩm dùng ổn , cho mình hỏi miếng trên màng hình là miếng dán cường lực hay chi là miếng dán thường thôi": 0
}

